In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Usage</a></span></li><li><span><a href="#Watermark" data-toc-modified-id="Watermark-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Watermark</a></span></li></ul></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Imports-/-load-data" data-toc-modified-id="Imports-/-load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports / load data</a></span><ul class="toc-item"><li><span><a href="#Load/extract-sanity-checking-data" data-toc-modified-id="Load/extract-sanity-checking-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load/extract sanity-checking data</a></span><ul class="toc-item"><li><span><a href="#Segmental-lexicon,-prefixes,-inventory,-and-triphones" data-toc-modified-id="Segmental-lexicon,-prefixes,-inventory,-and-triphones-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Segmental lexicon, prefixes, inventory, and triphones</a></span></li><li><span><a href="#Triphone-channel-distribution-and-channel-alphabet" data-toc-modified-id="Triphone-channel-distribution-and-channel-alphabet-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Triphone channel distribution and channel alphabet</a></span></li><li><span><a href="#Preview-and-postview-distributions-and-alphabets" data-toc-modified-id="Preview-and-postview-distributions-and-alphabets-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Preview and postview distributions and alphabets</a></span></li><li><span><a href="#Corpus-contexts" data-toc-modified-id="Corpus-contexts-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Corpus contexts</a></span></li><li><span><a href="#Conversion-to-one-hot-vectors-/-sequences-thereof" data-toc-modified-id="Conversion-to-one-hot-vectors-/-sequences-thereof-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Conversion to one-hot vectors / sequences thereof</a></span></li></ul></li><li><span><a href="#Load-segmental-sequence-channel-matrices-$p(Y_0^k|X_0^k)$-and-$p(Y_0^f|X_0^f)$" data-toc-modified-id="Load-segmental-sequence-channel-matrices-$p(Y_0^k|X_0^k)$-and-$p(Y_0^f|X_0^f)$-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Load segmental sequence channel matrices $p(Y_0^k|X_0^k)$ and $p(Y_0^f|X_0^f)$</a></span></li><li><span><a href="#Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$" data-toc-modified-id="Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load contextual distribution on segmental wordforms $p(W|C)$</a></span></li><li><span><a href="#Load-lexicon-metadata" data-toc-modified-id="Load-lexicon-metadata-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Load lexicon metadata</a></span></li></ul></li><li><span><a href="#Slow,-but-sanity-checking-calculations" data-toc-modified-id="Slow,-but-sanity-checking-calculations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Slow, but sanity-checking calculations</a></span><ul class="toc-item"><li><span><a href="#$p(C_0^i-|-X_0^i;-X_{i+1})$" data-toc-modified-id="$p(C_0^i-|-X_0^i;-X_{i+1})$-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>$p(C_0^i | X_0^i; X_{i+1})$</a></span></li><li><span><a href="#$p(Y_0^i|X_0^k)$" data-toc-modified-id="$p(Y_0^i|X_0^k)$-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>$p(Y_0^i|X_0^k)$</a></span></li><li><span><a href="#$p(X_0^f|C)$" data-toc-modified-id="$p(X_0^f|C)$-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>$p(X_0^f|C)$</a></span></li><li><span><a href="#$p(Y_0^i-|-c)$" data-toc-modified-id="$p(Y_0^i-|-c)$-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>$p(Y_0^i | c)$</a></span></li><li><span><a href="#$p(Y_0^i|x_0^k)$-sampler" data-toc-modified-id="$p(Y_0^i|x_0^k)$-sampler-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>$p(Y_0^i|x_0^k)$ sampler</a></span></li><li><span><a href="#$p(\widehat{X_0^f}|Y_0^{i},-C)$" data-toc-modified-id="$p(\widehat{X_0^f}|Y_0^{i},-C)$-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>$p(\widehat{X_0^f}|Y_0^{i}, C)$</a></span></li><li><span><a href="#$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1},-C)$" data-toc-modified-id="$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1},-C)$-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1}, C)$</a></span></li></ul></li><li><span><a href="#Definition-of-core-calculation" data-toc-modified-id="Definition-of-core-calculation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Definition of core calculation</a></span><ul class="toc-item"><li><span><a href="#Channel-sampler" data-toc-modified-id="Channel-sampler-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Channel sampler</a></span></li><li><span><a href="#Sample-parallel-posterior-estimator" data-toc-modified-id="Sample-parallel-posterior-estimator-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Sample-parallel posterior estimator</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Parallelization-over-samples-and-contexts" data-toc-modified-id="Parallelization-over-samples-and-contexts-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Parallelization over samples and contexts</a></span></li><li><span><a href="#Exact/full-wordform-length-variant-functions" data-toc-modified-id="Exact/full-wordform-length-variant-functions-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Exact/full wordform-length variant functions</a></span></li><li><span><a href="#Calculation-volume-organization-for-Case-1" data-toc-modified-id="Calculation-volume-organization-for-Case-1-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Calculation volume organization for Case 1</a></span><ul class="toc-item"><li><span><a href="#Slices:-Calculate-results-for-groups-of-reconstructed-wordforms-by-(input-prefix,-context)" data-toc-modified-id="Slices:-Calculate-results-for-groups-of-reconstructed-wordforms-by-(input-prefix,-context)-5.6.1"><span class="toc-item-num">5.6.1&nbsp;&nbsp;</span>Slices: Calculate results for groups of reconstructed wordforms by (input prefix, context)</a></span></li><li><span><a href="#Blocks:-Calculate-results-for-groups-of-input-prefixes-by-context" data-toc-modified-id="Blocks:-Calculate-results-for-groups-of-input-prefixes-by-context-5.6.2"><span class="toc-item-num">5.6.2&nbsp;&nbsp;</span>Blocks: Calculate results for groups of input prefixes by context</a></span></li></ul></li><li><span><a href="#Calculation-volume-organization-2" data-toc-modified-id="Calculation-volume-organization-2-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Calculation volume organization 2</a></span></li><li><span><a href="#Calculation-volume-organization-3" data-toc-modified-id="Calculation-volume-organization-3-5.8"><span class="toc-item-num">5.8&nbsp;&nbsp;</span>Calculation volume organization 3</a></span><ul class="toc-item"><li><span><a href="#Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$W$-and-for-a-single-given-$c$" data-toc-modified-id="Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$W$-and-for-a-single-given-$c$-5.8.1"><span class="toc-item-num">5.8.1&nbsp;&nbsp;</span>Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $W$ and for a single given $c$</a></span></li><li><span><a href="#Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$c$" data-toc-modified-id="Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$c$-5.8.2"><span class="toc-item-num">5.8.2&nbsp;&nbsp;</span>Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $c$</a></span></li></ul></li></ul></li><li><span><a href="#Calculate-distribution" data-toc-modified-id="Calculate-distribution-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Calculate distribution</a></span><ul class="toc-item"><li><span><a href="#Number-of-calculations-necessary-under-different-calculation-criteria" data-toc-modified-id="Number-of-calculations-necessary-under-different-calculation-criteria-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Number of calculations necessary under different calculation criteria</a></span></li><li><span><a href="#Determine-dimensions-and-cells-to-be-calculated-vs.-ignored" data-toc-modified-id="Determine-dimensions-and-cells-to-be-calculated-vs.-ignored-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Determine dimensions and cells to be calculated vs. ignored</a></span><ul class="toc-item"><li><span><a href="#Option:-Only-calculate-full-source-wordforms-rather-than-all-source-prefixes?" data-toc-modified-id="Option:-Only-calculate-full-source-wordforms-rather-than-all-source-prefixes?-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Option: Only calculate full source wordforms rather than all source prefixes?</a></span></li><li><span><a href="#Suboption:-Given-$\neg-r$-and-a-full-source-wordform-$w^*$,-only-calculate-posterior-probability-$p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)$,-$\forall-w^*$?" data-toc-modified-id="Suboption:-Given-$\neg-r$-and-a-full-source-wordform-$w^*$,-only-calculate-posterior-probability-$p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)$,-$\forall-w^*$?-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Suboption: Given $\neg r$ and a full source wordform $w^*$, only calculate posterior probability $p(\widehat{W} = w^* | W = w^*, C = c)$, $\forall w^*$?</a></span></li><li><span><a href="#Suboption:-Restrict-calculations-only-to-(source-sequence,-reconstruction-sequence)-pairs-within-edit-distance-$k$?" data-toc-modified-id="Suboption:-Restrict-calculations-only-to-(source-sequence,-reconstruction-sequence)-pairs-within-edit-distance-$k$?-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Suboption: Restrict calculations only to (source sequence, reconstruction sequence) pairs within edit distance $k$?</a></span></li><li><span><a href="#Altogether..." data-toc-modified-id="Altogether...-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Altogether...</a></span></li></ul></li><li><span><a href="#Case-1:-$r-\land-(0-<-k-\leq-4)$" data-toc-modified-id="Case-1:-$r-\land-(0-<-k-\leq-4)$-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Case 1: $r \land (0 &lt; k \leq 4)$</a></span></li><li><span><a href="#Case-2:-$\neg-r-\land-(0-<-k-\leq-4)-\land-\neg-e$" data-toc-modified-id="Case-2:-$\neg-r-\land-(0-<-k-\leq-4)-\land-\neg-e$-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Case 2: $\neg r \land (0 &lt; k \leq 4) \land \neg e$</a></span></li><li><span><a href="#Case-3:-$\neg-r-\land-e$" data-toc-modified-id="Case-3:-$\neg-r-\land-e$-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Case 3: $\neg r \land e$</a></span></li></ul></li></ul></div>

# Overview

Given a choice of parameters $\epsilon$ and $n$, and given
 - wordform channel matrices $p(Y_0^f|X_0^f)$
 - a contextual distribution on segmental wordforms $p(X_0^f|C)$
 - segmental lexicon metadata pre-calculating $k$-cousins/$k$-spheres up to $k=4$
 
Calculate

$$\hat{p}(\hat{X}_0^f = x_0^{'f}|X_0^f = x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where 
  - edit distance $d(x_0^{'f}, x_0^{*f}) \leq 4$
  - $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
  - $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
  - $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$

## Requirements

#FIXME

## Usage

#FIXME

## Watermark

In [2]:
import watermark

In [3]:
%load_ext watermark

In [4]:
%watermark -ihmuvg

last updated: 2019-08-10T22:05:38-07:00

CPython 3.7.3
IPython 7.7.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-51-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 12
interpreter: 64bit
host name  : kotoba
Git hash   : 9d300bff690d71c35cc28349b42bba9e7e61a15d


In [5]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              12
On-line CPU(s) list: 0-11
Thread(s) per core:  2
Core(s) per socket:  6
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               158
Model name:          Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz
Stepping:            10
CPU MHz:             800.368
CPU max MHz:         4600.0000
CPU min MHz:         800.0000
BogoMIPS:            6384.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            12288K
NUMA node0 CPU(s):   0-11
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dtes64 monitor ds_cpl vmx 

In [6]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        3.4G         27G        8.2M        430M         27G
Swap:          2.0G        1.3G        723M


In [7]:
!modinfo nvidia | grep version

version:        410.104
srcversion:     3B812B02678A6B43A294F17


In [8]:
%pwd

'/home/AD/emeinhar/wr'

# Parameters

In [9]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

In [10]:
from boilerplate import *

In [11]:
repo_dir = getcwd(); repo_dir

'/home/AD/emeinhar/wr'

In [12]:
# Parameters

#p(Y_0^k|X_0^k) - from step 4e
c = ''
# c = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle'
c = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_CMs_by_length_by_wordform_index.pickle'

#p(Y_0^f|X_0^f) - from step 4e
f = ''
f = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_exact_CMs_by_length_by_wordform_index.pickle'

#p(X_0^f|C) - from step 4c
w = ''
# w = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C.npy'
w = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C.npy'

# LTR metadata directory - contains pre-computed k-cousin/k-sphere information
m = ''
# m = 'LTR_Buckeye_aligned_w_GD_AmE_destressed'
m = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

# output filepath prefix for pW_WC
o = ''
# o = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_WC'
# o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC'
o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

# for sanity checking...

# the sorted prefixes of W here correspond to the W dimension of file 'w'
p = ''
# p = 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json'
p = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json'

# the sorted conditioning triphones and outcome response uniphones correspond to
# the dimensions of file 'c'
t = ''
# t = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'
t = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p3Y1X012.json'

# the sorted projected contexts here correspond to the C dimension of file 'w'
s = ''
# s = 'LD_Fisher_vocab_in_Buckeye_contexts/LM_filtered_buckeye_contexts.txt'
s = 'LD_Fisher_vocab_in_swbd2003_contexts/LM_filtered_swbd2003_contexts.txt'


# if True, then run notebook 'benchmark' cells
x = ''

# if True, then don't recalculate anything that's been calcualted already...
d = ''
d = 'True'
# d = 'False'

# calculation parameters

#samples per (reconstructed target wordform, source prefix, context) triple
n = ''
n = '50'

#maximum edit distance for reconstructed target, source prefix 'cousins' to calculate
k = ''
k = '2'

#batch size - currently only interpretable/used for case 3 (see below)
# if using a GPU with around 8GB of memory, use a batch size < 300
# if using a CPU and parallelizing, use a larger batch size (500-750).
# In either case, using all available memory will *not* in general
# maximize performance.
b = ''
b = '250'

#parallelize batches over processes?
l = ''
l = 'True'

#use GPU? (overrides parallelize option)
g = ''
g = 'True'

#if 'False', only calculate p(\hat{W}|W = w, c), i.e. don't calculate p(\hat{W}|P = p, c)
r = ''
r = 'False' 

#if r='False' and e='True', only calculate p(\hat{W} = w*| W = w*, c) ∀w ∈ W
e = ''
e = 'True' 

#the (optional) beginning and end of the block of complete wordforms this notebook should calculate
wStart = ''
wEnd = ''
# wStart = '0'
# wEnd = '2'

#the (optional) beginning and end of the block of contexts this notebook should calculate
# cStart = ''
# cEnd = ''
# cStart = '0'
# cEnd = '10'

In [13]:
file_params = (c, f, w, m, p, t, s)
for fp in file_params:
    if not path.exists(fp):
        print(f"Parameter path does not exist: {fp}")

In [14]:
if x == '' or x == 'False' or x == False:
    benchmark = False
elif x == 'True' or x == True:
    benchmark = True
else:
    raise Exception(f"x arg must be one of {'', 'True', 'False'}, got {x} instead, with type ={type(x)}.")

In [15]:
if d == '' or d == 'True' or d == True:
    d = True
elif d == 'False' or d == False:
    d = False
else:
    raise Exception(f"d arg must be one of {'', 'True', 'False'}, got {d} instead.")

In [16]:
output_dir = path.dirname(o)
if not path.exists(output_dir):
    print('Making output path {0}'.format(output_dir))
    makedirs(output_dir)

In [17]:
if n == '':
    n = 50
else:
    n = int(n)

In [18]:
if k == '':
    k = 2
else:
    k = int(k)

In [19]:
if b == '':
    b = None
else:
    b = int(b)
    
assert 0 < b, f"Batch size argument b = {b} must be greater than 0."

In [20]:
if l == '' or l == 'True' or l == True:
    l = True
elif l == 'False' or l == False:
    l = False
else:
    raise Exception(f"l must be one of {'', 'True', 'False'}, got {l} instead.")

In [21]:
if g == '' or g == 'True' or g == True:
    g = True
elif g == 'False' or g == False:
    g = False
else:
    raise Exception(f"g must be one of {'', 'True', 'False'}, got {g} instead.")

In [22]:
if r == '' or r == 'False' or r == False:
    r = False
#     raise Exception('Assuming only full wordforms have been produced is not supported currently.')
elif r == 'True' or r == True:
    r = True
else:
    raise Exception(f"r must be one of {'','True','False'}, got '{r}' instead.")

In [23]:
if not r and (e == '' or e == 'True' or e == True):
    e = True
#     raise Exception('Only calculating the posterior probability of the actual (correct) full intended wordform is not supported currently.')
elif r and (e == '' or e == 'True' or e == True):
    raise Exception("e can only be True if r is False")
elif e == 'False' or e == False:
    e = False
else:
    raise Exception(f"e must be one of {'','True','False'}, got '{e}'")

In [24]:
if b is not None:
    if r:
        raise Exception(f'batch size b = {b} only currently interpretable and supported when r is False and e is True.')

In [25]:
# postview_fp = path.join(path.dirname(t), "p6Y0X01.json"); postview_fp

# preview_fp = path.join(path.dirname(t), "p3Y1X01.json"); preview_fp

In [26]:
# cStart = int(cStart)
# cEnd = int(cEnd)

# assert cStart <= cEnd, f"First context id of block to calculate be <= the context id of the end of the block, got {cStart} and {cEnd} instead."

if wStart == '':
    wStart = 0
else:
    wStart = int(wStart)
if wEnd == '':
    wEnd = None
    print('No wEnd provided. Will be set to 1 less than the length of the number of contexts in the metadata.')
else:
    wEnd = int(wEnd)

if wEnd is not None:
    assert wStart <= wEnd, f"First word id of block to calculate must be <= the word id of the end of the block, got {wStart} and {wEnd} instead."

No wEnd provided. Will be set to 1 less than the length of the number of contexts in the metadata.


In [27]:
consolidation_enabled = True

# Imports / load data

In [28]:
import pickle

In [29]:
import numpy as np
import cupy
import torch
import sparse

In [30]:
import tiledb

In [31]:
from probdist import *

In [32]:
from tqdm import tqdm, tqdm_gui, tqdm_notebook

In [33]:
from joblib import Parallel, delayed

J = -1
# J = 16
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def identity(x):
    return x

def par(gen_expr):
    return Parallel(n_jobs=J, backend=BACKEND, verbose=V, prefer=PREFER)(gen_expr)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    if g and l:
        print("Disabling 'parallelize' flag...")
        l = False
    
    print(torch.cuda.get_device_name(0))
    total_mem_MB = torch.cuda.get_device_properties(device).total_memory / 1e6
    print('Total Memory: {0}'.format(total_mem_MB) )
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    if torch.cuda.device_count() > 1:
        print(torch.cuda.get_device_name(1))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(1)/1024**3,1), 'GB')
elif g:
    print("g set to 'True', but torch cannot find a GPU. Setting g to 'False'.")
    g = False
else:
    pass
#     raise Exception(f"g set to 'True' but torch cannot find a GPU.")

Using device: cuda

Disabling 'parallelize' flag...
GeForce GTX 1080
Total Memory: 8513.978368
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [35]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')

my_device = cpu

In [36]:
cuda_ft = torch.cuda.FloatTensor
cuda_dt = torch.cuda.DoubleTensor

ft = torch.FloatTensor
dt = torch.DoubleTensor

my_ft = ft
my_dt = dt

my_type = my_ft
# my_type = my_dt

torch.set_default_tensor_type(my_type)

my_tt = torch.float32
# my_tt = torch.float64

## Load/extract sanity-checking data

We want to be able to make queries, spot checks, and sanity checks. That means we want to be able to reference
 1. the set of strings constituting segmental wordforms and prefixes
 2. the source and channel alphabets
 3. the channel distribution's conditioning triphones ∩ the triphones in the lexicon = the triphones in the lexicon
 4.  

Segmental wordforms were necessary for 
 - the lexicon metadata calculation (step 4b)
 - the contextual distribution on segmental wordforms (step 4c)
 - the definition of the segmental sequence channel matrices (step 4d)
 
What does each use as input? (Pass that to this notebook.)

Each notebook uses ...pW_V.json (or something slightly downstream of that)

### Segmental lexicon, prefixes, inventory, and triphones

In [37]:
pW_V = condDistsAsProbDists(importProbDist(p))

Extract 
 - `orthographic vocabulary`
 - `segmental vocabulary`

In [38]:
Vs = set(pW_V.keys())
Ws = union(mapValues(lambda dist: set(conditions(dist)), 
                     pW_V).values())
len(Vs)
len(Ws)

9411

9172

In [39]:
wordlengthsInclEdges = set(len(ds2t(w)) for w in Ws)
wordlengthsInclEdges
numWordsOfExactlyLength = {l:len(wordformsOfLength(l, Ws, True)) for l in wordlengthsInclEdges}
numWordsOfExactlyLength

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}

{4: 5,
 5: 173,
 6: 1266,
 7: 1790,
 8: 1646,
 9: 1328,
 10: 1052,
 11: 828,
 12: 508,
 13: 329,
 14: 150,
 15: 66,
 16: 24,
 17: 6,
 19: 1}

In [40]:
wordLengthRange = sorted(list(range(min(wordlengthsInclEdges),
                                    max(wordlengthsInclEdges)+1)))
wordLengthRange

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [41]:
wordlengthsNotIncludingEdges = {each-2 for each in wordlengthsInclEdges}
wordlengthsNotIncludingEdges

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17}

In [42]:
lengthFreqs = {l:len(wordformsAtLeastLlong(l, Ws, True)) for l in wordlengthsInclEdges}
lengthFreqs

{4: 9172,
 5: 9167,
 6: 8994,
 7: 7728,
 8: 5938,
 9: 4292,
 10: 2964,
 11: 1912,
 12: 1084,
 13: 576,
 14: 247,
 15: 97,
 16: 31,
 17: 7,
 19: 1}

Create mappings between `segmental wordforms` and `segmental prefixes`.

In [43]:
prefix_relation = set(union({(w,p) for p in getPrefixes(w)} for w in tqdm(Ws)))
len(prefix_relation)

100%|██████████| 9172/9172 [00:06<00:00, 1343.26it/s] 


79727

Extract 
 - `segmental prefixes`

In [44]:
Ps = set(map(lambda pair: pair[1],
             prefix_relation))

Create 
 - `sorted` versions of the `segmental vocabulary` and `segmental prefixes`

In [45]:
Ws_t = tuple(sorted(list(Ws)))
Ps_t = tuple(sorted(list(Ps)))
num_wordforms = len(Ws_t)
num_prefixes = len(Ps_t)

num_wordforms
num_prefixes

9172

42231

In [46]:
wordformsByLength_t = {l:tuple(sorted([w for w in wordformsOfLength(l, Ws, True)]))
                       for l in wordlengthsInclEdges}

In [47]:
Ws_l_t = wordformsByLength_t

Create mappings between `orthographic wordforms` and `segmental wordforms`.

In [48]:
v_to_Ws = mapValues(lambda dist: set(conditions(dist)),
                    pW_V)
V_W_relation = {(v,w) 
                for v in v_to_Ws 
                for w in v_to_Ws[v]}
w_to_Vs = {w:{v for v in Vs if (v,w) in V_W_relation}
           for w in Ws}

In [49]:
if not benchmark: 
    del pW_V
    del Vs
    del v_to_Ws
    del V_W_relation
    del w_to_Vs

Extract
 - `segmental inventory`
 - `triphones` in the `segmental lexicon`

In [50]:
source_alphabet = lexiconToInventory(Ws)
Xs = source_alphabet
len(Xs)
Xs

41

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ə',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ',
 '⋉',
 '⋊'}

In [51]:
Xs_t = tuple(sorted(Xs))

In [52]:
lexicon_triphones = lexiconTo3factors(Ws)
len(lexicon_triphones)
list(lexicon_triphones)[:10]

7412

['p.ʌ.k',
 '⋊.s.p',
 'dʒ.u.v',
 'ʒ.m.ɪ',
 'i.tʃ.ɚ',
 'k.s.⋉',
 'ɪ.n.v',
 'b.ʊ.ʃ',
 'ɑ.ŋ.z',
 'w.oʊ.l']

In [53]:
X012s = lexicon_triphones
X012s_t = tuple(sorted(list(X012s)))

In [54]:
triphs_with_LE = {triph for triph in lexicon_triphones if leftEdge in triph}
len(triphs_with_LE)

489

In [55]:
triphs_with_RE = {triph for triph in lexicon_triphones if rightEdge in triph}
len(triphs_with_RE)

541

In [56]:
lexicon_diphones = lexiconTo2factors(Ws)
len(lexicon_diphones)
list(lexicon_diphones)[:10]

1003

['z.u', 'ʊ.t', 'ɹ.ʊ', 'ɚ.ʃ', 'n.u', 'ə.t', 'v.ɹ', 'θ.æ', 'æ.l', 'i.d']

In [57]:
X01s = lexicon_diphones
X01s_t = tuple(sorted(list(X01s)))

In [58]:
X01s_with_LE = {diph for diph in X01s if leftEdge in diph}
len(X01s_with_LE)

36

In [59]:
X01s_with_RE = {diph for diph in X01s if rightEdge in diph}
len(X01s_with_RE)

32

### Triphone channel distribution and channel alphabet

In [60]:
p3Y1X012 = condDistsAsProbDists(importProbDist(t))

assert uniformOutcomes(p3Y1X012)

In [61]:
channel_triphones = conditions(p3Y1X012)
len(channel_triphones)

assert all(triph in channel_triphones for triph in lexicon_triphones)

7412

In [62]:
{triph for triph in lexicon_triphones if triph not in channel_triphones}

set()

In [63]:
channel_alphabet = outcomes(p3Y1X012)
len(channel_alphabet)
channel_alphabet

39

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ',
 '⋉'}

In [64]:
Y1s = channel_alphabet
Y1s_t = tuple(sorted(Y1s))

In [65]:
in_lexicon_inventory_but_not_in_channel_inventory = source_alphabet - channel_alphabet
in_lexicon_inventory_but_not_in_channel_inventory

assert in_lexicon_inventory_but_not_in_channel_inventory == {leftEdge, rightEdge} or in_lexicon_inventory_but_not_in_channel_inventory == {'ə', leftEdge, rightEdge} or in_lexicon_inventory_but_not_in_channel_inventory == {'ə', leftEdge}

{'ə', '⋊'}

### Preview and postview distributions and alphabets

In [66]:
# preview_fp
# postview_fp

In [67]:
# p3Y1X01 = condDistsAsProbDists(importProbDist(preview_fp))

# p6Y0X01 = condDistsAsProbDists(importProbDist(postview_fp))

In [68]:
# assert uniformOutcomes(p3Y1X01)

# assert uniformOutcomes(p6Y0X01)

In [69]:
# preview_outcomes = outcomes(p3Y1X01)

# postview_outcomes = outcomes(p6Y0X01)

# assert preview_outcomes == Y1s

# assert postview_outcomes == Y1s

In [70]:
# preview_channel_diphones = conditions(p3Y1X01)
# len(preview_channel_diphones)

# # missing_preview_conditions = {diph for diph in lexicon_diphones if (diph not in preview_channel_diphones) and (leftEdge not in diph)}
# # missing_preview_conditions

# # p3Y1X01 = condProbDistAsDicts(p3Y1X01)

# # #add 𝚺⋉ to preview conditions
# # # for each 𝛔⋉ ∈ 𝚺⋉, ensure p(Y_1 = ⋉|𝛔⋉) = 1.0
# # for c in missing_preview_conditions:
# #     p3Y1X01[c] = {o:1.0 if ds2t(o)[1] == rightEdge else 0.0 for o in Y1s}

# # # add ⋉ to preview outcomes
# # for c in preview_channel_diphones
    
# # p3Y1X01 = condDistsAsProbDists(p3Y1X01)

# assert all(diph in preview_channel_diphones for diph in lexicon_diphones if not (leftEdge in diph or rightEdge in diph))

In [71]:
# postview_channel_diphones = conditions(p6Y0X01)
# len(postview_channel_diphones)

# assert all(diph in postview_channel_diphones for diph in lexicon_diphones if not (leftEdge in diph or rightEdge in diph))

In [72]:
# p3Y1X01 = condProbDistAsDicts(p3Y1X01)
# p6Y0X01 = condProbDistAsDicts(p6Y0X01)

# p3Y1X01 = {c:p3Y1X01[c] for c in lexicon_diphones if not (leftEdge in c or rightEdge in c)}
# p6Y0X01 = {c:p6Y0X01[c] for c in lexicon_diphones if not (leftEdge in c or rightEdge in c)}

# assert areNormalized(p3Y1X01)
# assert areNormalized(p6Y0X01)
# assert uniformOutcomes(p3Y1X01)
# assert uniformOutcomes(p6Y0X01)

# p3Y1X01 = condDistsAsProbDists(p3Y1X01)
# p6Y0X01 = condDistsAsProbDists(p6Y0X01)

In [73]:
# len(conditions(p3Y1X01))
# len(conditions(p6Y0X01))

In [74]:
# Y012s = set(map(t2ds, sigmaK(set.union(Y1s, edgeSymbols), 3)))
# len(set.union(Y1s, edgeSymbols))
# len(Y012s)
# Y012s_t = tuple(sorted(list(Y012s)))

### Corpus contexts

In [75]:
Cs = importSeqs(s, list)
len(Cs)

Cs_t = tuple(sorted(Cs))
Cs_t[123:129]

assert tuple(Cs) == Cs_t

106295

('<rem> and doctor',
 '<rem> and electric',
 '<rem> and especially',
 '<rem> and even',
 '<rem> and for',
 '<rem> and he')

In [76]:
if not benchmark:
    del Cs

In [77]:
if wEnd is None:
    print(f'wEnd not provided; setting wEnd to 1 less than the number of segmental wordforms in the metadata = {len(Ws_t)}')
    wEnd = len(Ws_t) - 1

wEnd not provided; setting wEnd to 1 less than the number of segmental wordforms in the metadata = 9172


In [78]:
if wEnd > len(Ws_t):
    raise Exception(f'wEnd = {wEnd} must be less than the number of segmental wordforms ({len(Ws_t)})')

In [79]:
if wEnd < wStart:
    raise Exception(f"wStart must be <= wEnd, got {wStart} and {wEnd} instead")

### Conversion to one-hot vectors / sequences thereof

In [80]:
# construct what you need to convert to/from one-hot representations

# look at segment sequence channel matrix notebook

In [81]:
Xmap = seqsToIndexMap(Xs)
XOHmap = seqsToOneHotMap(Xs)

In [82]:
X012map = seqsToIndexMap(X012s)
# X012OHs = seqMapToOneHots(X012map)
X012OHmap = seqsToOneHotMap(X012s)

In [83]:
Y1map = seqsToIndexMap(Y1s)

In [84]:
def dsToUniphoneIndices(ds, uniphoneToIndexMap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToIndexMap[uniphone] for uniphone in uniphoneSeq])

def dsToUniphoneOHs(ds, uniphoneToOHmap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToOHmap[uniphone] for uniphone in uniphoneSeq])

def dsToTriphoneSeq(ds):
    return dsToKfactorSequence(3, ds)

def dsToTriphoneIndices(ds, triphoneToIndexMap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToIndexMap[triphone] for triphone in triphoneSeq])

def dsToTriphoneOHs(ds, triphoneToOHmap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToOHmap[triphone] for triphone in triphoneSeq])

dsToUniphoneIndices('t.i.f.l', Xmap)
dsToUniphoneOHs('t.i.f.l', XOHmap)
dsToTriphoneSeq('t.i.f.l')
dsToTriphoneIndices('t.i.f.l', X012map)
dsToTriphoneOHs('t.i.f.l', X012OHmap)
dsToTriphoneOHs('t.i.f.l', X012OHmap).shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0].shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0][5528]
dsToTriphoneOHs('t.i.f.l', X012OHmap)[1][5352]

array([18,  9,  6, 12])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

('t.i.f', 'i.f.l')

array([3756, 1449])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(2, 7412)

(7412,)

0.0

0.0

In [85]:
# Y1s_RE = outcomes(p3Y1X01)
# len(Y1s_RE)
# Y1s_RE_list = sorted(list(Y1s_RE))

# print(Y1s_RE - Y1s)

# Y1REmap = seqsToIndexMap(Y1s_RE)

# Y1REOHs = seqMapToOneHots(Y1REmap)
# Y1REOHmap = seqsToOneHotMap(Y1s_RE)
Y1OHmap = seqsToOneHotMap(Y1s)
# OHY1REmap = oneHotToSeqMap(Y1s_RE)
OHY1map = oneHotToSeqMap(Y1s)

def ymap(y):
#     return Y1REmap[y]
    return Y1map[y]

def y0kMap(y0k):
    return np.array(list(map(ymap, ds2t(y0k)[1:])))

def channelSeqOHs2ds(y0k_OHs, addLeftEdge = False):
    if not addLeftEdge:
#         return t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
        return t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))
#     return leftEdge + '.' + t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
    return leftEdge + '.' + t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))

def channelSeqds2OHs(y0k):
    y0k_t = ds2t(y0k)
    if leftEdge == y0k_t[0]:
        y1k_t = y0k_t[1:]
    y1k_t = y0k_t[1:]
#     return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (_, 39)
    return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (_, 38)

def y0kOHmap(y0k):
##     y0k_t = ds2t(y0k)
## #     y0k_indices = y0kMap(y0k) #np.array(list(map(lambda y1: Y1map[y1], y0k_t[1:])))
##     y1k = t2ds(y0k_t[1:])
##     y0k_OHs = dsToUniphoneOHs(y1k, Y1REOHmap)
#     return dsToUniphoneOHs(t2ds(ds2t(y0k)[1:]), Y1REOHmap)
    y0k_t = ds2t(y0k) #let l = len(y0k_t)
    if y0k_t[0] == leftEdge:
        y1k_t = y0k_t[1:]
#         return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 39)
        return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 38)
#     return np.array([Y1REOHmap[ yj ] for yj in y0k_t]) #shape should be (l, 39)
    return np.array([Y1OHmap[ yj ] for yj in y0k_t]) #shape should be (l, 38)


# list(Y1s_RE)[0]
list(Y1s)[0]
# OHY1REmap( Y1REOHmap[ list(Y1s_RE)[0] ] )
OHY1map( Y1OHmap[ list(Y1s)[0] ] )

channelSeqds2OHs(leftEdge + '.'+ 't.u.p')

channelSeqOHs2ds( channelSeqds2OHs(leftEdge + '.'+ 't.u.p') , True)

channelSeqOHs2ds(y0kOHmap(leftEdge + '.' + 't.u.p'), True)
channelSeqOHs2ds(y0kOHmap('t.u.p'))
len(ds2t(leftEdge + '.' + 't.u.p'))
len(ds2t('t.u.p'))
y0kOHmap('t.u.p').shape
y0kOHmap(leftEdge + '.' + 't.u.p').shape

'k'

'k'

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]])

'⋊.t.u.p'

'⋊.t.u.p'

't.u.p'

4

3

(3, 39)

(3, 39)

In [86]:
p3Y1X012_np = condDistFamilyToNP(p3Y1X012)
p3Y1X012_np.shape

(39, 7412)

In [87]:
from random import choice

In [88]:
random_source_wordform = choice(list(Ws_t))
random_source_wordform

random_source_prefix = choice(list(Ps_t))
random_source_prefix

'⋊.s.ɛ.n.s.⋉.⋉'

'⋊.l.ɛ.t.d'

In [89]:
def randomPrefix(l, alphabet=Xs):
    return randomString(alphabet, l, hasLeftEdge=True)

In [90]:
random_channel_prefix2 = randomPrefix(len(ds2t(random_source_wordform))-1, alphabet=Y1s)
random_channel_prefix2

'⋊.ŋ.oʊ.ʊ.b.æ.h'

In [91]:
# random_source_prefix = getRandomKey(pX0i)
random_source_prefix = choice(list(Ps_t))
random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)
while ds2t(random_source_prefix)[-1] == rightEdge or len(ds2t(random_source_prefix)) > len(ds2t(random_source_wordform)) or len(ds2t(random_channel_prefix)) + 1 > len(ds2t(random_source_wordform)):
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps_t))
    random_source_wordform = choice(list(Ws_t))
    random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)
# while len(ds2t(random_source_prefix)) > len(ds2t(random_source_wordform)):
# #     random_source_prefix = getRandomKey(pX0i)
#     random_source_prefix = choice(list(Ps))
# # random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)
# while len(ds2t(random_channel_prefix)) + 1 > len(ds2t(random_source_wordform)):
#     random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)

random_source_wordform
random_source_prefix
random_channel_prefix

'⋊.b.ɹ.u.h.ɑ.h.ɑ.⋉.⋉'

'⋊.m.ɪ.s.k.ɪ'

'⋊.p.ʊ.f.t.d'

In [92]:
len(ds2t(random_channel_prefix))
len(ds2t(random_source_prefix))
len(ds2t(random_source_wordform))

6

6

10

In [93]:
def sourcePrefixToTriphones(x0k):
    xp_t = ds2t(x0k) #"x prefix"
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    
#     xik_ds = t2ds((xi, xk))
#     preview_dist = p3Y1X01[xik_ds]
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
    return x012s

random_triphoneSeq = sourcePrefixToTriphones(random_source_prefix)
random_triphoneSeq
threeFactorSequenceToDS(random_triphoneSeq)

('⋊.m.ɪ', 'm.ɪ.s', 'ɪ.s.k', 's.k.ɪ')

'⋊.m.ɪ.s.k.ɪ'

In [94]:
def sourcePrefixToTriphoneIndices(x0k):
    triphoneSequence = sourcePrefixToTriphones(x0k)
    return tuple(map(lambda x012: X012map[x012], triphoneSequence))

sourcePrefixToTriphoneIndices(random_source_prefix)

(7120, 2523, 6010, 3450)

## Load segmental sequence channel matrices $p(Y_0^k|X_0^k)$ and $p(Y_0^f|X_0^f)$

Index $i$ of `pY0k_X0ks` is a tensor of shape $|W| \times |Y_1| \times (i-2)$: for each wordform $w$ in $W$, there is a length $i-2$ stack of vectors giving the channel distribution for the length $i$ prefix (including the left word edge symbol and the upcoming segment) of $w$. (The stack is of length $i-2$ because there are $i-2$ triphones associated with a length $i$ source prefix.) Where necessary and appropriate (viz. when the wordform's length is incongruent w.r.t. the channel string), a wordform's channel matrix has been padded with columns of 0s to reflect the fact that the relevant source prefix could never have given rise to a channel string of that length.

`pY0f_X0fs` is an analogous tensor with a similar shape, but intended for use in contexts where only wordforms whose length exactly matches the length of a channel string is worth considering as a cause of the channel string. Accordingly, its first dimension is $|W_{i+2}|$, where $W_{i+2}$ is the set of all wordforms exactly of length $i+2$ (including the left word edge and the upcoming symbol).

In [95]:
pY0k_X0ks = pickle.load(open(c, 'rb'))
len(pY0k_X0ks)
pY0k_X0ks[10].dtype

20

dtype('float64')

In [96]:
pY0k_X0ks[10].shape

(9172, 39, 8)

In [97]:
pY0f_X0fs = pickle.load(open(f, 'rb'))
len(pY0f_X0fs)
pY0f_X0fs[10].dtype

20

dtype('float64')

In [98]:
list(map(lambda stack: stack.nbytes / 1e9 if stack is not None else None,
         pY0k_X0ks))
sum(list(map(lambda stack: stack.nbytes / 1e9 if stack is not None else None,
         pY0k_X0ks)))
list(map(lambda stack: stack.nbytes / 1e9 if stack is not None else None,
         pY0f_X0fs))
sum(list(map(lambda stack: stack.nbytes / 1e9 if stack is not None else None,
         pY0f_X0fs)))

[0.0,
 0.0,
 0.0,
 0.0,
 0.005723328,
 0.008584992,
 0.011446656,
 0.01430832,
 0.017169984,
 0.020031648,
 0.022893312,
 0.025754976,
 0.02861664,
 0.031478304,
 0.034339968,
 0.037201632,
 0.040063296,
 0.04292496,
 0.045786624,
 0.048648288]

0.434972928

[0.0,
 0.0,
 0.0,
 0.0,
 3.12e-06,
 0.000161928,
 0.001579968,
 0.0027924,
 0.003081312,
 0.002900352,
 0.002625792,
 0.002325024,
 0.00158496,
 0.001129128,
 0.0005616,
 0.000267696,
 0.000104832,
 2.808e-05,
 0.0,
 5.304e-06]

0.019151496

In [99]:
pY0k_X0ks[0].dtype
pY0f_X0fs[0].dtype

dtype('float64')

dtype('float64')

In [100]:
wordlengthsNotIncludingEdges
wordlengthsInclEdges
lengthFreqs

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17}

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}

{4: 9172,
 5: 9167,
 6: 8994,
 7: 7728,
 8: 5938,
 9: 4292,
 10: 2964,
 11: 1912,
 12: 1084,
 13: 576,
 14: 247,
 15: 97,
 16: 31,
 17: 7,
 19: 1}

In [101]:
len(Ws_t)
len(Y1s_t)

9172

39

In [102]:
pY0k_X0ks[0].shape
pY0k_X0ks[1].shape
pY0k_X0ks[2].shape
pY0k_X0ks[3].shape
pY0k_X0ks[4].shape

pY0f_X0fs[0].shape
pY0f_X0fs[1].shape
pY0f_X0fs[2].shape
pY0f_X0fs[3].shape
pY0f_X0fs[4].shape

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(9172, 39, 2)

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(5, 39, 2)

In [103]:
pY0k_X0ks[4][0]
pY0f_X0fs[4][0]

array([[0.65264711, 0.02141693],
       [0.00700978, 0.00855777],
       [0.02545656, 0.00855777],
       [0.02545656, 0.58122414],
       [0.00682531, 0.00821919],
       [0.06235012, 0.00855777],
       [0.00682531, 0.00833257],
       [0.00700978, 0.02552273],
       [0.00682531, 0.00844442],
       [0.00700978, 0.00855777],
       [0.0051651 , 0.00734595],
       [0.00682531, 0.00844442],
       [0.00700978, 0.00855777],
       [0.00700978, 0.00844442],
       [0.00700978, 0.01630829],
       [0.00700978, 0.00855777],
       [0.00700978, 0.00844442],
       [0.00682531, 0.00844442],
       [0.00700978, 0.05787536],
       [0.00700978, 0.00855777],
       [0.00700978, 0.00855777],
       [0.00682531, 0.00844442],
       [0.00664084, 0.00832952],
       [0.00682531, 0.00833257],
       [0.00700978, 0.00855777],
       [0.00682531, 0.00821919],
       [0.00700978, 0.00734595],
       [0.00700978, 0.00855777],
       [0.00700978, 0.00855777],
       [0.00682531, 0.00844442],
       [0.

array([[0.64012878, 0.        ],
       [0.00864939, 0.        ],
       [0.00714712, 0.        ],
       [0.00710919, 0.        ],
       [0.00597457, 0.        ],
       [0.09519946, 0.        ],
       [0.00632972, 0.        ],
       [0.00649346, 0.        ],
       [0.00735496, 0.        ],
       [0.00743004, 0.        ],
       [0.00693599, 0.        ],
       [0.00614232, 0.        ],
       [0.00622218, 0.        ],
       [0.00613831, 0.        ],
       [0.00649346, 0.        ],
       [0.0064649 , 0.        ],
       [0.00613831, 0.        ],
       [0.00614232, 0.        ],
       [0.00676474, 0.        ],
       [0.00622218, 0.        ],
       [0.00637828, 0.        ],
       [0.00614232, 0.        ],
       [0.00606245, 0.        ],
       [0.00605844, 0.        ],
       [0.00992452, 0.        ],
       [0.00621729, 0.        ],
       [0.00565472, 0.        ],
       [0.0220417 , 0.        ],
       [0.00967323, 0.        ],
       [0.00654113, 0.        ],
       [0.

In [104]:
p3Y1X012_np.shape

(39, 7412)

In [105]:
pY0k_X0ks[4].shape
pY0f_X0fs[4].shape

(9172, 39, 2)

(5, 39, 2)

In [106]:
def retrieve_CM_for(x0f_WE, total_prefix_length, exact = False):
    '''
    total_prefix_length includes 
     - the "⋊" symbol,
     - the actually and completely produced prefix,
     - the next (i.e. upcoming) segment.
     
    Ex:
      w = '⋊.aɪ.d.i.l.aɪ.z.⋉'
      |'⋊.aɪ.d.i.l.aɪ.z.⋉'| = 8
      l = 5
    =>
      'total_prefix' = '⋊.aɪ.d.i.l'
      |'⋊.aɪ.d.i.l'| = 5
      The associated channel matrix is of shape (|Y1s|, l-2 = 5-2 = 3).
    '''
    if not exact:
        return pY0k_X0ks[total_prefix_length][Ws_t.index(x0f_WE)]
    else:
        return pY0f_X0fs[total_prefix_length][Ws_l_t[total_prefix_length].index(x0f_WE)]

In [107]:
w_0 = Ws_t[0]; w_0
w_0_produced_prefix_noLE = t2ds(ds2t(Ws_t[0])[1:3+1]); w_0_produced_prefix_noLE, len(ds2t(w_0_produced_prefix_noLE))
w_0_produced_prefix = t2ds(ds2t(Ws_t[0])[0:3+1]); w_0_produced_prefix, len(ds2t(w_0_produced_prefix))
w_0_produced_prefix_plus_next = t2ds(ds2t(Ws_t[0])[0:3+2]); w_0_produced_prefix_plus_next, len(ds2t(w_0_produced_prefix_plus_next))

pY0k_X0ks[5][0].shape
# pY0k_X0ks[3][0] #assume the produced prefix is of length i+1=3+1=4 (not including left word edge)
pY0k_X0ks[5][0] # assume the produced prefix is of length i-2=3 (not including left word edge)
                # assume the produced prefix is of length i-1=4 (including left word edge)
                # assume the produced prefix + upcoming segment is of length i=5 (including left word edge)

w_0_whole_prefix = t2ds(ds2t(Ws_t[0])[0:3+2]); w_0_whole_prefix
w_0_whole_prefix_3factors = sourcePrefixToTriphones(w_0_whole_prefix); w_0_whole_prefix_3factors
w_0_whole_prefix_3factor_indices = sourcePrefixToTriphoneIndices(w_0_whole_prefix); w_0_whole_prefix_3factor_indices
w_0_prefix_CM = np.vstack([p3Y1X012_np[:,source_triph_idx] for source_triph_idx in w_0_whole_prefix_3factor_indices]).T; w_0_prefix_CM
w_0_prefix_CM.shape

assert np.allclose( w_0_prefix_CM, pY0k_X0ks[5][0] )
assert np.allclose( pY0k_X0ks[5][0], retrieve_CM_for(Ws_t[0], 5))

'⋊.aɪ.d.i.l.aɪ.z.⋉.⋉'

('aɪ.d.i', 3)

('⋊.aɪ.d.i', 4)

('⋊.aɪ.d.i.l', 5)

(39, 3)

...

'⋊.aɪ.d.i.l'

('⋊.aɪ.d', 'aɪ.d.i', 'd.i.l')

(6923, 5, 498)

array([[0.65264711, 0.02141693, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.02545656, 0.00855777, 0.00703146],
       [0.02545656, 0.58122414, 0.00693832],
       [0.00682531, 0.00821919, 0.00675326],
       [0.06235012, 0.00855777, 0.00703146],
       [0.00682531, 0.00833257, 0.00684642],
       [0.00700978, 0.02552273, 0.00703146],
       [0.00682531, 0.00844442, 0.00693832],
       [0.00700978, 0.00855777, 0.73787478],
       [0.0051651 , 0.00734595, 0.00603577],
       [0.00682531, 0.00844442, 0.00693832],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00700978, 0.00844442, 0.00693832],
       [0.00700978, 0.01630829, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00700978, 0.00844442, 0.00693832],
       [0.00682531, 0.00844442, 0.00693832],
       [0.00700978, 0.05787536, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00682531, 0.00844442, 0.00693832],
       [0.

(39, 3)

**Notes:**

 - `|pY0k_X0ks|` = $max_l = |w|. ∄w'. |w'| > |w|$ (around 16-18, usually) 
 - `pY0k_X0ks[i]` = the lexical channel matrices for 'total prefixes' of length `i` (i.e. incl. word edges and upcoming segment)
 - `pY0k_X0ks[i]` =  the lexical channel matrices for prefixes of length `i-1` (i.e. *not* incl. word edges, but incl. upcoming segment)
 - `|pY0k_X0ks[i]|` = `(|W|, |Y1|, i-2)` = (segmental wordforms, Y1s, number of triphones)
 
(Similar statements hold wrt `pY0fX0f`, mod earlier notes.)

## Load contextual distribution on segmental wordforms $p(W|C)$

In [108]:
pW_C = np.load(w)
pW_C.shape
pW_C.dtype
pW_C.nbytes / 1e9

(9172, 106295)

dtype('float64')

7.79950192

In [109]:
if pW_C.shape[1] != len(Cs_t):
    raise Exception(f'# of contexts in metadata file does not match context dimension of pW_C matrix: {len(Cs_t)} vs. {pW_C.shape[1]}\n\t{s}\n\t{w}')

In [110]:
len(Ws_t)
len(Cs_t)

9172

106295

In [111]:
Cs_t[Cs_t.index('a couple of')]
pW_C[:,Cs_t.index('a couple of')]
isNormalized_np(pW_C[:,Cs_t.index('a couple of')])

'a couple of'

array([2.52036749e-10, 1.50634197e-08, 2.52036749e-10, ...,
       5.54693797e-10, 4.15967189e-08, 6.22123621e-07])

True

In [112]:
def pW_C_lookup(w=None,c=None):
    if w is None and c is None:
        raise Exception('Must specify at least one of a context string or segmental wordform string')
    if w is None:
        my_pW_c = pW_C[:,Cs_t.index(c)]
        my_pW_c_as_dict = dict(zip(Ws_t, my_pW_c))
        assert isNormalized(my_pW_c_as_dict, epsilon = 1e-6)
        return ProbDist(my_pW_c_as_dict)
    if w is not None and c is not None:
        my_pw_c = pW_C[Ws_t.index(w), Cs_t.index(c)]
        return my_pw_c
    if c is None:
        my_pw_C = pW_C[Ws_t.index(w), :]
        my_pw_C_as_dict = dict(zip(Cs_t, my_pw_C))
        return my_pw_C_as_dict

In [113]:
my_dist = pW_C_lookup(c='a couple of')

In [114]:
my_dist.most_common(10)

[('⋊.t.aɪ.m.z.⋉.⋉', 0.7017068824069426),
 ('⋊.ð.ɛ.m.⋉.⋉', 0.058586970250377776),
 ('⋊.ð.ə.⋉.⋉', 0.05467041905649899),
 ('⋊.ʌ.ð.ɚ.⋉.⋉', 0.03664541602803625),
 ('⋊.m.aɪ.⋉.⋉', 0.028159696907432537),
 ('⋊.ð.oʊ.z.⋉.⋉', 0.020634293629461325),
 ('⋊.h.ʌ.n.d.ɹ.ə.d.⋉.⋉', 0.01748920205729206),
 ('⋊.l.aɪ.k.⋉.⋉', 0.014006091693604115),
 ('⋊.g.ʊ.d.⋉.⋉', 0.005842215226429354),
 ('⋊.m.ɪ.l.i.ə.n.⋉.⋉', 0.005304906857139398)]

In [115]:
random_context = choice(Cs_t); random_context
Cs_t.index(random_context)
pW_C_lookup(c=random_context).most_common(10)

'right yeah well'

59734

[('⋊.aɪ.⋉.⋉', 0.8038854945877595),
 ('⋊.j.u.⋉.⋉', 0.05961778775433437),
 ('⋊.w.i.⋉.⋉', 0.022756717198857943),
 ('⋊.ð.ə.⋉.⋉', 0.020517328336076567),
 ('⋊.m.aɪ.⋉.⋉', 0.01917511991263336),
 ('⋊.ɪ.t.⋉.⋉', 0.007016147837544381),
 ('⋊.æ.n.d.⋉.⋉', 0.00516328623558489),
 ('⋊.ð.eɪ.⋉.⋉', 0.004734345378218211),
 ('⋊.ð.æ.t.⋉.⋉', 0.004231753849583734),
 ('⋊.h.w.ɑ.t.⋉.⋉', 0.004064795540676732)]

## Load lexicon metadata

In [116]:
cousin_fn_map = {i:'{0}cousinsOf.npz'.format(i) 
                 for i in range(5)}
sphere_fn_map = {i:'{0}spheresOf.npz'.format(i) 
                 for i in range(5)}

In [117]:
cousin_fn_map

{0: '0cousinsOf.npz',
 1: '1cousinsOf.npz',
 2: '2cousinsOf.npz',
 3: '3cousinsOf.npz',
 4: '4cousinsOf.npz'}

In [118]:
sphere_fn_map

{0: '0spheresOf.npz',
 1: '1spheresOf.npz',
 2: '2spheresOf.npz',
 3: '3spheresOf.npz',
 4: '4spheresOf.npz'}

In [119]:
assert all(fn in listdir(m) for fn in cousin_fn_map.values())
assert all(fn in listdir(m) for fn in sphere_fn_map.values())

In [120]:
chdir(m)

In [121]:
cousin_mats = mapValues(sparse.load_npz, cousin_fn_map)

In [122]:
sphere_mats = mapValues(sparse.load_npz, sphere_fn_map)

In [123]:
chdir(repo_dir)

The $k$th-cousin/sphere matrix $m$ is a matrix of shape $|P| \times |W|$, where $m_{i,j}$ = 1 iff prefix $p_i$ is in a $k$-cousin/sphere relation to wordform $w_j$.

In [124]:
len(Ps_t)
len(Ws_t)

42231

9172

In [125]:
cousin_mats
mapValues(lambda m: m.nbytes / 1e9,
          cousin_mats)

{0: <COO: shape=(42231, 9172), dtype=uint8, nnz=79727, fill_value=0>,
 1: <COO: shape=(42231, 9172), dtype=uint8, nnz=1000076, fill_value=0>,
 2: <COO: shape=(42231, 9172), dtype=uint8, nnz=9307972, fill_value=0>,
 3: <COO: shape=(42231, 9172), dtype=uint8, nnz=39821468, fill_value=0>,
 4: <COO: shape=(42231, 9172), dtype=uint8, nnz=71449640, fill_value=0>}

{0: 0.001355359, 1: 0.017001292, 2: 0.158235524, 3: 0.676964956, 4: 1.21464388}

In [126]:
sphere_mats
mapValues(lambda m: m.nbytes / 1e9,
          sphere_mats)

{0: <COO: shape=(42231, 9172), dtype=uint8, nnz=9172, fill_value=0>,
 1: <COO: shape=(42231, 9172), dtype=uint8, nnz=30652, fill_value=0>,
 2: <COO: shape=(42231, 9172), dtype=uint8, nnz=370148, fill_value=0>,
 3: <COO: shape=(42231, 9172), dtype=uint8, nnz=2062370, fill_value=0>,
 4: <COO: shape=(42231, 9172), dtype=uint8, nnz=3207990, fill_value=0>}

{0: 0.000155924, 1: 0.000521084, 2: 0.006292516, 3: 0.03506029, 4: 0.05453583}

In [127]:
# c1.coords

In [128]:
c1 = cousin_mats[1]
(c1.coords[0][-2], c1.coords[1][-2])
Ps_t[c1.coords[0][-2]]
Ws_t[c1.coords[1][-2]]

(42229, 6949)

'⋊.θ.ʌ.n.d.ɚ.⋉'

'⋊.w.ʌ.n.d.ɚ.⋉.⋉'

In [129]:
m

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

In [130]:
# segmental_wordforms = importSeqs(path.join(m, 'LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V_Transcriptions.txt'))
# len(segmental_wordforms)

In [131]:
# list(segmental_wordforms)[0]

In [132]:
# Ws_t = tuple(sorted(list(map(padInputSequenceWithBoundaries,
#                              segmental_wordforms))))
# len(Ws_t)

In [133]:
# Ps = union(map(lambda w: getPrefixes(padInputSequenceWithBoundaries(w)), segmental_wordforms))
# len(Ps)

In [134]:
# Ps_t = tuple(sorted(list(Ps)))
# len(Ps_t)

# Slow, but sanity-checking calculations

## $p(C_0^i | X_0^i; X_{i+1})$

$C_i = (Y_{i-1}^{x_i}, Y_i^{x_i}, Y_{i+1}^{x_i})$

$p(y_{i-1}, y_i; y_{i+1} | x_{i-1}^i; x_{i+1}) = p(y_{i-1} | x_{i-1}, x_i;) p(y_i | x_{i-1}^i ; x_{i+1}) p(y_{i+1} | x_i ; x_{i+1})$

$p(c_0^i | x_0^i; x_{i+1}) = \prod\limits_{j=0}^{j=i} p(c_i | x_{i-1}^i ; x_{i+1})$

In [135]:
# def pC1_X012(y012, x012):
#     x012_t = ds2t(x012)
#     y012_t = ds2t(y012)
    
#     x0, x1, x2 = x012_t[0], x012_t[1], x012_t[2]
#     x01 = (x0, x1)
#     x12 = (x1, x2)
    
#     y0, y1, y2 = y012_t[0], y012_t[1], y012_t[2]
    
#     if x0 == leftEdge:
#         if y0 == leftEdge:
#             left_term = 1.0
#         else:
#             left_term = 0.0
#     else:
#         if y0 == leftEdge:
#             left_term = 0.0
#         else:
#             left_term = p6Y0X01[x01][y0]
    
#     center_term = p3Y1X012[x012][y1]
    
#     if x1 == rightEdge:
#         if y1 == rightEdge:
#             right_term = 1.0
#         else:
#             right_term = 0.0
#     else:
#         if y1 == rightEdge:
#             right_term = 0.0
#         else:
#             right_term = p3Y1X01[x12][y2]
    
#     terms = (left_term, center_term, right_term)
    
#     result = prod(terms)
    
#     return result

# def pC0iX0k(c0i, x0k):
#     xp_t = ds2t(x0k)
#     three_factor_seq = dsToKfactorSequence(3, x0k)
    
#     inputs = zip(c0i, three_factor_seq)
    
#     terms = starmap(pC1_X012, inputs)
    
#     result = prod(terms)
    
#     return result

In [136]:
# s = leftEdge + '.i.p.i.' + rightEdge; s
# dsToKfactorSequence(3, s)
# threeFactorSequenceToDS(dsToKfactorSequence(3, s))

In [137]:
# len(Y012s)
# len(X012s)
# len(Y012s) * len(X012s)

In [138]:
# # takes 5.5m on sidious and 45GB
# square = set(product(Y012s, X012s))

In [139]:
# def pY012_X012_calc(y012, x012):
#     return (x012, pC1_X012(y012, x012))

# pY012_X012 = dict(par(delayed(pY012_X012_calc)(y012, x012) for y012, x012 in product(Y012s, X012s)))

# # def pY012_X012_calc(y012_x012_pair):
# #     y012, x012 = y012_x012_pair
# #     return (x012, pC1_X012(y012, x012))

# # pY012_X012 = dict(par(delayed(pY012_X012_calc)(y012_x012_pair) for y012_x012_pair in product(Y012s, X012s)))

## $p(Y_0^i|X_0^k)$

In [140]:
len(pY0k_X0ks)
sorted(wordlengthsInclEdges)
assert len(pY0k_X0ks)-1 == sorted(wordlengthsInclEdges)[-1]

20

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19]

In [141]:
len(Ws)
len(Ps)

9172

42231

In [142]:
pY0k_X0ks[3].shape

(0, 0)

In [143]:
def pY0iX0k(y0i, x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
    yp_t = ds2t(y0i) #"y prefix"
#     assert xp_t[0] == leftEdge
#     assert yp_t[0] == leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3.')
    if (len(yp_t)+1 != len(xp_t)) and not ((yp_t[-1] == rightEdge) and len(yp_t) == len(xp_t)):
        raise Exception('|y0i| must = |x0k|-1, or |y0k| == |x0k| and y_k == ⋉.')
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0k must match.')
    # if len(xp_t) == 1:
    #     if xp_t[0] != leftEdge:
    #         raise Exception('First symbol of x0k must be left word edge symbol.')    
    #     if yp_t[0] == leftEdge:
    #         return 1.0
    #     else:
    #         return 0.0
    if yp_t[-1] == rightEdge:
        #if y_{f+1} == ⋉, then 
        # the only strings x_0^f that could generate it
        # have exactly the same length and also end with ⋉
        if len(yp_t) != len(xp_t):
            return 0.0
        
        x_k = xp_t[-1]
        if x_k != rightEdge:
            return 0.0
        
        #if |x_0^f| == |y_0^f| ∧ (y_{f+1} == x_{f+1} == ⋉),
        # then remove y_{f+1} and proceed with the calculation
        # as normal...
        y_k = rightEdge
        yp_t = yp_t[:-1]
    elif xp_t[-1] == rightEdge:
        # if x_k = ⋉ and y_k != ⋉, then the probability of that channel string is 0.0...
        return 0.0
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # yk = yp_t[-1]
    # xik_ds = t2ds((xi, xk))
    # preview_dist = p3Y1X01[xik_ds]
#     if yk not in preview_dist:
#         print('y0i: {0}\nx0k: {1}\nxi: {2}\nxk: {3}\nyk: {4}\nxik: {5}'.format(y0i, x0k, xi, xk, yk, xik_ds))
#     preview_term = preview_dist[yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
    # y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#         x0i_ds = t2ds(x0i_t)
#         y0i_ds = t2ds(y0i_t)
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
#     x012s = dsToKfactorSequence(3, x0i_ds)
#     x012s = dsToKfactorSequence(3, t2ds(xp_t))
    x012s = dsToKfactorSequence(3, x0k)
    # y1s = y0i_t[1:]
    y1s = yp_t[1:]
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    pairs = zip(x012s, y1s)
    produced_terms = (p3Y1X012[x012][y1] for x012, y1 in pairs)
    
#     result = preview_term * prod(produced_terms)
    result = prod(produced_terms)

#     if debug:
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(y1s))
#         print('main terms: {0}'.format(produced_terms))
#         print('end result = {0}'.format(result))
    
    if debug:
#         return result, [p3Y1X012[x012] for x012 in x012s]
        return result, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T


    return result

In [144]:
w_0
# w_0_prefix = 
w_0_3 = t2ds(ds2t(w_0)[:3+2]); w_0_3# + 1
w_0_4 = t2ds(ds2t(w_0)[:4+2]); w_0_4# + 1
pY0k_X0ks[5][0]

'⋊.aɪ.d.i.l.aɪ.z.⋉.⋉'

'⋊.aɪ.d.i.l'

'⋊.aɪ.d.i.l.aɪ'

array([[0.65264711, 0.02141693, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.02545656, 0.00855777, 0.00703146],
       [0.02545656, 0.58122414, 0.00693832],
       [0.00682531, 0.00821919, 0.00675326],
       [0.06235012, 0.00855777, 0.00703146],
       [0.00682531, 0.00833257, 0.00684642],
       [0.00700978, 0.02552273, 0.00703146],
       [0.00682531, 0.00844442, 0.00693832],
       [0.00700978, 0.00855777, 0.73787478],
       [0.0051651 , 0.00734595, 0.00603577],
       [0.00682531, 0.00844442, 0.00693832],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00700978, 0.00844442, 0.00693832],
       [0.00700978, 0.01630829, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00700978, 0.00844442, 0.00693832],
       [0.00682531, 0.00844442, 0.00693832],
       [0.00700978, 0.05787536, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00700978, 0.00855777, 0.00703146],
       [0.00682531, 0.00844442, 0.00693832],
       [0.

In [145]:
pY0iX0k(w_0_3, w_0_4)
pY0iX0k(w_0_3, w_0_4, True)[1]
assert np.allclose( pY0k_X0ks[6][0], pY0iX0k(w_0_3, w_0_4, True)[1] )

0.1983147788326225

array([[0.65264711, 0.02141693, 0.00703146, 0.007226  ],
       [0.00700978, 0.00855777, 0.00703146, 0.007226  ],
       [0.02545656, 0.00855777, 0.00703146, 0.007226  ],
       [0.02545656, 0.58122414, 0.00693832, 0.00713029],
       [0.00682531, 0.00821919, 0.00675326, 0.00694011],
       [0.06235012, 0.00855777, 0.00703146, 0.007226  ],
       [0.00682531, 0.00833257, 0.00684642, 0.00703584],
       [0.00700978, 0.02552273, 0.00703146, 0.007226  ],
       [0.00682531, 0.00844442, 0.00693832, 0.00713029],
       [0.00700978, 0.00855777, 0.73787478, 0.007226  ],
       [0.0051651 , 0.00734595, 0.00603577, 0.00620276],
       [0.00682531, 0.00844442, 0.00693832, 0.00713029],
       [0.00700978, 0.00855777, 0.00703146, 0.70851712],
       [0.00700978, 0.00844442, 0.00693832, 0.00713029],
       [0.00700978, 0.01630829, 0.00703146, 0.007226  ],
       [0.00700978, 0.00855777, 0.00703146, 0.02155085],
       [0.00700978, 0.00844442, 0.00693832, 0.00713029],
       [0.00682531, 0.00844442,

In [146]:
pY0iX0k('⋊.k.ɑ.n.t', '⋊.k.ɑ.n.t.⋉')
pY0iX0k('⋊.k.ɑ.n.t.⋉', '⋊.k.ɑ.n.t.⋉')

0.0

0.14201210183923127

In [147]:
pY0iX0k('⋊.k.ɑ.n', '⋊.k.ɑ.n.t')
pY0iX0k('⋊.k.ɑ.m', '⋊.k.ɑ.n.t')

0.25068286913932425

0.0034938697256087312

In [148]:
def trimToMatch(y0i, x0f):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1:
        raise Exception('|x0f| must ≥ |y0i| + 1.')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
    return x0k

def trimToLength(l, x0f, pad=False, padSymbol='?'):
    x0f_t = ds2t(x0f)
#     y0i_t = ds2t(y0i)
#     l = len(y0i_t)
    if len(x0f_t) < l and not pad:
        raise Exception('x0f must be at least as long as l.')
    if len(x0f_t) < l and pad:
        x0k_t = x0f_t[:l]
        x0k_t_padded = tuple( list(x0k_t) + [padSymbol] * (l - len(x0k_t)) )
        return t2ds(x0k_t_padded)
    else:
        x0k_t = x0f_t[:l]
        x0k = t2ds(x0k_t)
        return x0k
    
# trimToMatch('t.u.p', 't.u.p')
trimToMatch('t.u.p', 't.u.p.l')
trimToLength(3, 't.u.p')
trimToLength(3, 't.u.p.l')
trimToLength(6, 't.u.p.l', True)

't.u.p.l'

't.u.p'

't.u.p'

't.u.p.l.?.?'

In [149]:
def pY0iX0f(y0i, x0f, debug = False):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1 and not ((y0i_t[-1] == rightEdge) and len(y0i_t) == len(x0f_t)):
        raise Exception('|x0f| must ≥ |y0i| + 1 or |x0f| = |y0k| ∧ y_k = ⋉')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
#     print(y0i)
#     print(x0k)
    return pY0iX0k(y0i, x0k, debug)

In [150]:
pY0iX0f('⋊.k.ɑ.n', '⋊.k.ɑ.n.t')
pY0iX0f('⋊.k.ɑ.n', '⋊.k.ɑ.n.t.⋉')
pY0iX0f('⋊.k.ɑ.n.t', '⋊.k.ɑ.n.t.⋉')
pY0iX0f('⋊.k.ɑ.n.t', '⋊.k.ɑ.n.t.⋉')
pY0iX0f('⋊.k.ɑ.n.t.⋉', '⋊.k.ɑ.n.t.⋉.⋉')

0.25068286913932425

0.25068286913932425

0.0

0.0

0.0

In [151]:
random_x0f = choice(Ws_t); random_x0f
lw = len(ds2t(random_x0f))
while lw-1 <= 2:
    random_x0f = getRandomKey(pW); random_x0f
    lw = len(ds2t(random_x0f))

random_y0i = randomPrefix(choice(range(2,lw-1)), alphabet=Y1s); random_y0i
random_x0f
random_y0i
# pY0iX0f(random_y0i, random_x0f)

'⋊.oʊ.n.⋉.⋉'

'⋊.ʌ.h.ɪ'

'⋊.oʊ.n.⋉.⋉'

'⋊.ʌ.h.ɪ'

## $p(X_0^f|C)$

In [152]:
pW_C.shape
(len(Ws_t), len(Cs_t))
assert pW_C.shape[0] == len(Ws_t)
assert pW_C.shape[1] == len(Cs_t)

(9172, 106295)

(9172, 106295)

In [153]:
if benchmark:
    Cs_t[Cs_t.index('a couple of')]

    pX0f = pW_C[:,Cs_t.index('a couple of')]

    pW = pW_C_lookup(c = Cs_t[Cs_t.index('a couple of')])

In [154]:
def pX0f_C(w, c):
    return pW_C[:, Cs_t.index(c)]

## $p(Y_0^i | c)$

In [155]:
def pXhat0fY0i_unnormalized(xhat0f, y0i, c):
    likelihood = pY0iX0f(y0i, xhat0f)
#     prior = pW[xhat0f]
    prior = pW_C_lookup(xhat0f, c)
    return likelihood * prior

def pY0i(y0i, c):
#     l = len(ds2t(y0i))
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if y0i_t[-1] == rightEdge:
        possible_source_wordforms = {w for w in Ws if len(ds2t(w)) == l}
    else:
        possible_source_wordforms = {w for w in Ws if len(ds2t(w)) >= l+1}
#     x0ksWithLengthl = prefixesWithLength(l)
#     py0kx0ks = tuple(pXhat0fY0k_unnormalized(x0k, y0k) for x0k in x0ksWithLengthl)

#     sufficiently_long_words = {w for w in Ws if len(ds2t(w)) >= l+1}
#     print(len(sufficiently_long_words))
    #line below is slower by about a third (pypy/kotoba)
#     sufficiently_long_words = wordformsAtLeastLlong(l, True)

    py0ix0fs = (pXhat0fY0i_unnormalized(x0f, y0i, c) for x0f in possible_source_wordforms)
#     def foo(x0f):
#         return pXhat0fY0k_unnormalized(x0f, y0k)
    # there's no apparent benefit to using foo + generator
#     py0kx0fs = (foo(x0f) for x0f in sufficiently_long_words)
# Parallel(n_jobs=-1, verbose=50, batch_size=4, prefer="processes", backend="multiprocessing")
#     py0kx0fs = Parallel(n_jobs=8, batch_size=4, verbose=50, prefer="threads", backend="multiprocessing")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
#     py0kx0fs = Parallel(n_jobs=8, prefer="threads")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
    return sum(py0ix0fs)

In [156]:
random_context = choice(Cs_t); random_context
pY0i('⋊.k.ɑ.n.t', random_context)

'and stress disorders'

2.6368955055803817e-06

In [157]:
# {w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}

In [158]:
random_context = choice(Cs_t); random_context
pXhat0fY0i_unnormalized('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context)

'take their own'

6.889596570332141e-07

## $p(Y_0^i|x_0^k)$ sampler

In [159]:
def sample_pY0iX0k(x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
#     yp_t = ds2t(y0i) #"y prefix"
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0i must match.')
    if len(xp_t) == 1:
        if xp_t[0] != leftEdge:
            raise Exception('First symbol of x0k must be left word edge symbol.')
        else:
            return leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3')
    
    # xi = xp_t[-2] #just-completed segment
    # xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # xik = t2ds((xi, xk))
    # yk = sampleFrom(p3Y1X01[xik])
#     yk = yp_t[-1]
#     preview_term = p3Y1X01[t2ds((xi, xk))][yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#     y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
#     print('x012s: {0}'.format(x012s))
#     y1s = y0i_t[1:]
    y1s = [sampleFrom(p3Y1X012[x012]) for x012 in x012s]
#     print('y1s: {0}'.format(y1s))
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    y0i_t = tuple( [leftEdge] + y1s )
#     print('y0i_t = {0}'.format(y0i_t))
    y0i_ds = t2ds(y0i_t)
#     print('y0i_ds = {0}'.format(y0i_ds))
#     assert len(y0i_t) == len(x0i_t)
    # yp_t = tuple([leftEdge] + y1s + [yk])
    # y0i = t2ds(yp_t)
    
    if xp_t[-1] == rightEdge:
        y0i_ds = y0i_ds + '.' + rightEdge
    
    if debug:
#         sorted_outcomes = Y1s_t
        return y0i_ds, np.array([[p3Y1X012[x012][k] for k in Y1s_t] for x012 in x012s]).T
#         return y0i_ds, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(tuple(y1s)))
#         print('main terms: {0}'.format(produced_terms))

    return y0i_ds
#     return preview_term * prod(produced_terms)

In [160]:
'⋊.æ.k.t.ɪ.v.ə.t.i.⋉.⋉'
activity_samples1 = [sample_pY0iX0k('⋊.æ.k.t.ɪ.v.ə.t.i.⋉.⋉') for each in range(20)]; activity_samples1
activity_samples2 = [sample_pY0iX0k('⋊.æ.k.t.ɪ.v.ə.t.i.⋉') for each in range(20)]; activity_samples2
activity_samples3 = [sample_pY0iX0k('⋊.æ.k.t.ɪ.v.ə.t.i') for each in range(5)]; activity_samples3
del activity_samples1, activity_samples2, activity_samples3

'⋊.æ.k.t.ɪ.v.ə.t.i.⋉.⋉'

['⋊.t.ʌ.t.f.v.l.ɑ.i.⋉.⋉',
 '⋊.aɪ.s.aɪ.ɪ.v.ɔɪ.t.i.⋉.⋉',
 '⋊.ɑ.ɪ.t.h.v.ɪ.t.j.⋉.⋉',
 '⋊.oʊ.ʊ.t.ɪ.v.ɑ.t.ʃ.⋉.⋉',
 '⋊.ŋ.n.t.eɪ.v.ɪ.t.h.⋉.⋉',
 '⋊.ɑ.w.t.ŋ.v.ʌ.t.i.⋉.⋉',
 '⋊.k.k.ʃ.ɪ.v.ɑ.t.i.⋉.⋉',
 '⋊.n.æ.t.ɪ.ɪ.ʌ.ɛ.i.⋉.⋉',
 '⋊.v.k.eɪ.h.p.ɔɪ.n.i.⋉.⋉',
 '⋊.aɪ.k.t.ɪ.v.ʒ.t.i.⋉.⋉',
 '⋊.ɑ.k.t.ʌ.j.ɛ.t.ɑ.⋉.⋉',
 '⋊.ɑ.k.t.ɪ.z.k.t.i.⋉.⋉',
 '⋊.ʒ.k.t.ʊ.v.ɔɪ.z.i.⋉.⋉',
 '⋊.ɑ.ʃ.t.u.f.ʌ.t.i.⋉.⋉',
 '⋊.æ.k.t.ɪ.v.ʃ.t.w.⋉.⋉',
 '⋊.ɛ.k.tʃ.ɪ.æ.ɛ.t.i.⋉.⋉',
 '⋊.æ.æ.d.v.v.ʊ.h.i.⋉.⋉',
 '⋊.eɪ.k.t.ʌ.v.ʌ.dʒ.i.⋉.⋉',
 '⋊.ʒ.k.ð.ɔɪ.v.ɚ.t.i.⋉.⋉',
 '⋊.aɪ.k.t.ɪ.v.aʊ.t.i.⋉.⋉']

['⋊.æ.k.t.f.v.ɪ.t.i.⋉',
 '⋊.g.k.tʃ.eɪ.v.b.z.i.⋉',
 '⋊.ɑ.k.ʊ.ɪ.v.h.t.ð.⋉',
 '⋊.æ.ɔɪ.ʌ.ɪ.k.t.oʊ.i.⋉',
 '⋊.b.k.t.d.v.n.t.i.⋉',
 '⋊.æ.k.t.ʊ.ʊ.t.t.p.⋉',
 '⋊.ɑ.k.t.ɪ.tʃ.ɔɪ.ɔɪ.ɛ.⋉',
 '⋊.aɪ.k.t.aʊ.v.ʌ.dʒ.i.⋉',
 '⋊.aɪ.k.t.ɪ.v.f.u.i.⋉',
 '⋊.aɪ.k.t.ɪ.v.ʊ.eɪ.d.⋉',
 '⋊.æ.k.t.ɪ.j.eɪ.t.i.⋉',
 '⋊.æ.θ.t.ɪ.v.ʌ.t.t.⋉',
 '⋊.ŋ.k.t.ɪ.v.ɪ.ɚ.i.⋉',
 '⋊.ɑ.d.d.ɛ.ɚ.eɪ.b.i.⋉',
 '⋊.ɑ.j.eɪ.ɪ.ɹ.p.ʊ.i.⋉',
 '⋊.ɑ.k.ŋ.ɪ.v.ʌ.ʊ.i.⋉',
 '⋊.ɑ.k.ʌ.eɪ.v.h.t.i.⋉',
 '⋊.b.ŋ.t.ɪ.ʃ.aɪ.θ.i.⋉',
 '⋊.aɪ.f.t.ɪ.v.ɛ.t.i.⋉',
 '⋊.æ.ɚ.t.ɪ.v.d.t.i.⋉']

['⋊.oʊ.æ.ɪ.ɪ.v.eɪ.dʒ',
 '⋊.æ.ɔɪ.t.ɛ.u.ɪ.t',
 '⋊.æ.eɪ.t.ɪ.v.p.t',
 '⋊.æ.tʃ.t.ɛ.v.æ.t',
 '⋊.ɑ.n.t.ɪ.v.g.t']

In [161]:
random_source_prefix
random_channel_samples = [sample_pY0iX0k(random_source_prefix) for each in range(50)]; random_channel_samples
del random_channel_samples

'⋊.m.ɪ.s.k.ɪ'

['⋊.m.ɪ.s.k',
 '⋊.m.ɪ.s.k',
 '⋊.m.k.s.k',
 '⋊.m.ɪ.s.t',
 '⋊.m.k.s.u',
 '⋊.m.θ.ɔɪ.ʒ',
 '⋊.dʒ.ʃ.b.k',
 '⋊.m.eɪ.s.k',
 '⋊.m.ɪ.s.f',
 '⋊.m.ɪ.s.d',
 '⋊.m.ɛ.s.k',
 '⋊.m.ɪ.s.ɔɪ',
 '⋊.n.ð.s.k',
 '⋊.ɪ.ɪ.s.k',
 '⋊.m.ʒ.ʃ.k',
 '⋊.ʒ.z.s.k',
 '⋊.p.ɪ.z.k',
 '⋊.m.j.s.k',
 '⋊.m.l.ʒ.k',
 '⋊.b.ɪ.s.k',
 '⋊.m.ɛ.s.k',
 '⋊.m.eɪ.s.ʌ',
 '⋊.m.ɛ.p.ʊ',
 '⋊.m.w.s.k',
 '⋊.m.ɪ.s.k',
 '⋊.m.ɪ.s.k',
 '⋊.f.m.s.k',
 '⋊.m.ɛ.l.k',
 '⋊.ʌ.eɪ.ʃ.k',
 '⋊.m.ɪ.p.ʌ',
 '⋊.m.l.æ.j',
 '⋊.m.i.s.g',
 '⋊.m.ɪ.s.k',
 '⋊.n.ʒ.l.k',
 '⋊.m.ʒ.ŋ.k',
 '⋊.m.m.aʊ.θ',
 '⋊.ʃ.ɪ.s.g',
 '⋊.m.ɪ.s.n',
 '⋊.u.eɪ.s.k',
 '⋊.m.s.ɪ.k',
 '⋊.m.ʒ.p.k',
 '⋊.n.ɪ.b.g',
 '⋊.m.ɪ.s.k',
 '⋊.m.eɪ.v.k',
 '⋊.m.ɪ.m.v',
 '⋊.m.ɛ.ɹ.aʊ',
 '⋊.ɛ.ɪ.s.k',
 '⋊.m.t.s.k',
 '⋊.m.ɪ.s.aʊ',
 '⋊.n.eɪ.v.eɪ']

In [162]:
cont_samples = [sample_pY0iX0k('⋊.k.ɑ.n.t') for each in range(50)]; cont_samples
del cont_samples

['⋊.k.ɑ.n',
 '⋊.l.ɑ.n',
 '⋊.k.ɹ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.ʌ',
 '⋊.aɪ.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.oʊ',
 '⋊.k.ɑ.n',
 '⋊.k.ʌ.n',
 '⋊.k.ɑ.n',
 '⋊.p.ɑ.n',
 '⋊.k.ɑ.aɪ',
 '⋊.k.ʌ.n',
 '⋊.k.ʌ.aɪ',
 '⋊.l.ɑ.n',
 '⋊.z.ɑ.tʃ',
 '⋊.k.ʌ.n',
 '⋊.k.ʌ.aʊ',
 '⋊.k.f.n',
 '⋊.k.ɑ.ɑ',
 '⋊.g.ɑ.n',
 '⋊.k.tʃ.tʃ',
 '⋊.k.tʃ.u',
 '⋊.k.ɑ.ʃ',
 '⋊.b.oʊ.n',
 '⋊.eɪ.ʊ.n',
 '⋊.k.ɑ.ɛ',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.g.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.b.tʃ.n',
 '⋊.k.ɹ.ʌ',
 '⋊.k.ɑ.m',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.θ',
 '⋊.k.ʌ.v',
 '⋊.k.ɑ.ʌ',
 '⋊.k.ɑ.n',
 '⋊.aɪ.ɑ.j',
 '⋊.k.ɑ.ɑ',
 '⋊.ʒ.ɑ.ɔɪ',
 '⋊.ʒ.ɑ.æ',
 '⋊.k.p.n',
 '⋊.k.ɑ.n',
 '⋊.aɪ.ɑ.n',
 '⋊.k.ɹ.n']

In [163]:
sample_pY0iX0k('⋊.k.ɑ.n.t', debug=True)[1].shape
sample_pY0iX0k('⋊.k.ɑ.n.t', debug=True)[1]

(39, 3)

array([[0.04390334, 0.01805178, 0.00911065],
       [0.00700978, 0.01374583, 0.00911065],
       [0.02545656, 0.00721312, 0.00911065],
       [0.00700978, 0.00711758, 0.00898997],
       [0.00682531, 0.00692774, 0.01700733],
       [0.00700978, 0.00721312, 0.00911065],
       [0.00682531, 0.0070233 , 0.0088709 ],
       [0.00700978, 0.00721312, 0.00911065],
       [0.00682531, 0.00711758, 0.00898997],
       [0.00700978, 0.00721312, 0.00911065],
       [0.0051651 , 0.00619171, 0.00782054],
       [0.65246265, 0.00711758, 0.00898997],
       [0.02545656, 0.00721312, 0.00911065],
       [0.00700978, 0.00711758, 0.00898997],
       [0.00700978, 0.00721312, 0.6450247 ],
       [0.00700978, 0.01374583, 0.00911065],
       [0.00700978, 0.00711758, 0.00898997],
       [0.00682531, 0.00711758, 0.00898997],
       [0.00700978, 0.00721312, 0.00911065],
       [0.00700978, 0.00721312, 0.00911065],
       [0.00700978, 0.00721312, 0.00911065],
       [0.00682531, 0.00711758, 0.00898997],
       [0.

## $p(\widehat{X_0^f}|Y_0^{i}, C)$

$p(\widehat{x_0^f}|y_0^{i}, c) = \frac{p(y_0^{i} | \widehat{x_0^f})p(\widehat{x_0^f} | c)}{p(y_0^{i})}$

In [164]:
def pXhat0fY0i_old(xhat0f, y0i, c):
    numerator = pXhat0fY0i_unnormalized(xhat0f, y0i, c)
    denominator = pY0i(y0i, c)
    if denominator == 0.0:
#         if numerator == 0.0:
#             print('both numerator and denominator are 0')
#         else:
#             print('just the denominator is 0')
        return 0
    return numerator / denominator

random_context = choice(Cs_t); random_context
(random_source_wordform, random_channel_prefix, random_context)
pXhat0fY0i_old(random_source_wordform, random_channel_prefix, random_context)

"that's the half"

('⋊.b.ɹ.u.h.ɑ.h.ɑ.⋉.⋉', '⋊.p.ʊ.f.t.d', "that's the half")

8.333147900706623e-07

## $\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1}, C)$

$p(\widehat{x_0^f} | x_0^i;x_{i+1}, c) = \sum\limits_{y_0^{i}} p(\widehat{x_0^f} | y_0^{i}, c)p(y_0^{i}|x_0^{i};x_{i+1}, c)$

$\widehat{p}(\widehat{x_0^f}|x_0^i;x_{i+1}, c) = \frac{1}{m}\sum\limits_{m \text{ sampled } y_0^{i}\text{s from } p(Y_0^{i} | x_0^{i};x_{i+1})} p(\widehat{x_0^f} | y_0^{i}, c)$

In [165]:
# BASELINE = OLD
def phatXhat0fX0k_baseline(xhat0f, x0k, c, m = 50, my_j=None):
#     y0ks = tuple(sample_pY0kX0k(x0k) for each in range(n))
#     terms = (pXhat0fY0k(xhat0f, y0k) for y0k in y0ks)

#     foo = partial(pXhat0fY0i, xhat0f)
#     def foo(y0i):
#         return pXhat0fY0i(xhat0f, y0i)
#     terms = (foo(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in y0is)
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=5, verbose=50, batch_size=10, prefer="threads")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=-1, verbose=50, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))

#     terms = Parallel(n_jobs=-1, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0k_old)(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
#     terms = Parallel(n_jobs=-1, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
    if my_j is None:
        my_j = -1
    terms = Parallel(n_jobs=my_j, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i, c) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
#     terms = (pXhat0fY0k_old(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
    s = sum(terms)
    sbar = (1.0 * s) / m
    return sbar

In [166]:
random_context = choice(Cs_t); random_context

'you know another'

In [167]:
if benchmark:
    phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 50)

In [168]:
if benchmark:
    phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 200)

In [169]:
if benchmark:
    {w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}
    [f"p(W' = {w}) → {pW[w]}"
     for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}),
                     key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

    [f"p(W' = {w}|r = '⋊.k.ɑ.n.t') → {phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1)}"
     for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}),
                     key=lambda w:phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1), reverse=True)]

In [170]:
if benchmark:
    {w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}

    [f"p(W' = {w}) → {pW_C_lookup(w=w, c=random_context)}"
     for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}),
                     key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

    [f"p(W' = {w}|r = '⋊.k.ɑ.n.t') → {phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1)}"
     for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}),
                     key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

In [171]:
'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 1
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 4)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 4), 
             pY0k_X0ks[4][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')])

'⋊.k.ɑ.n.t'

5

4

array([[0.04390334, 0.01805178],
       [0.00700978, 0.01374583],
       [0.02545656, 0.00721312],
       [0.00700978, 0.00711758],
       [0.00682531, 0.00692774],
       [0.00700978, 0.00721312],
       [0.00682531, 0.0070233 ],
       [0.00700978, 0.00721312],
       [0.00682531, 0.00711758],
       [0.00700978, 0.00721312],
       [0.0051651 , 0.00619171],
       [0.65246265, 0.00711758],
       [0.02545656, 0.00721312],
       [0.00700978, 0.00711758],
       [0.00700978, 0.00721312],
       [0.00700978, 0.01374583],
       [0.00700978, 0.00711758],
       [0.00682531, 0.00711758],
       [0.00700978, 0.00721312],
       [0.00700978, 0.00721312],
       [0.00700978, 0.00721312],
       [0.00682531, 0.00711758],
       [0.00664084, 0.00702074],
       [0.00682531, 0.0070233 ],
       [0.00700978, 0.01805178],
       [0.00682531, 0.00692774],
       [0.00700978, 0.00619171],
       [0.00700978, 0.59565208],
       [0.00700978, 0.00721312],
       [0.00682531, 0.00711758],
       [0.

True

# Definition of core calculation

In [172]:
from random import choice

In [173]:
random_wordform = choice(Ws_t)
random_wordform_length = len(ds2t(random_wordform))
random_wordform_idx = Ws_t.index(random_wordform)
random_wordform, random_wordform_idx, random_wordform_length

('⋊.ɹ.ɪ.d.i.m.⋉.⋉', 8715, 8)

In [174]:
type(pW_C)
pW_C.shape
pW_C[:,0].shape

numpy.ndarray

(9172, 106295)

(9172,)

In [175]:
Cs_t[Cs_t.index('a couple of')]
random_context = 'a couple of'

'a couple of'

In [176]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G         13G        9.6G        8.2M        8.5G         17G
Swap:          2.0G        1.3G        723M


In [177]:
# pX0f = pW_C[:,Cs_t.index('a couple of')]
# pX0f_torch = torch.from_numpy(pX0f)
pW_C_torch = torch.from_numpy(pW_C).type(my_tt)
pW_C_torch.dtype

torch.float32

In [178]:
def get_pW_t(c_idx):
    return pW_C_torch[:,c_idx]

In [179]:
# if r:
CMsByLengthByWordformIndex = pY0k_X0ks
CMsByLengthByWordformIndex_torch = [torch.from_numpy(each).type(my_tt) 
                                    for each in CMsByLengthByWordformIndex]
exactCMsByLengthByWordformIndex = pY0f_X0fs
exactCMsByLengthByWordformIndex_torch = [torch.tensor(each).type(my_tt) 
                                         for each in exactCMsByLengthByWordformIndex]

In [180]:
# CMsByLengthByWordformIndex_cupy = cupy.array(CMsByLengthByWordformIndex)
# CMsByLengthByWordformIndex_cupy.device

In [181]:
# CMsByLengthByWordformIndex = [cupy.array(each.numpy()).get()
#                               for each in CMsByLengthByWordformIndex_torch]
# type(CMsByLengthByWordformIndex[4])

In [182]:
CMsByLengthByWordformIndex[10].dtype
CMsByLengthByWordformIndex_torch[10].dtype

exactCMsByLengthByWordformIndex[10].dtype
exactCMsByLengthByWordformIndex_torch[10].dtype

dtype('float64')

torch.float32

dtype('float64')

torch.float32

In [183]:
len(CMsByLengthByWordformIndex_torch)
len(exactCMsByLengthByWordformIndex_torch)

20

20

In [184]:
random_wordform
random_wordform_idx
random_wordform_length
torch.equal(CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx],
            exactCMsByLengthByWordformIndex_torch[random_wordform_length][Ws_l_t[random_wordform_length].index(random_wordform)])
CMsByLengthByWordformIndex_torch[random_wordform_length-1][random_wordform_idx]
random_wordform in Ws_l_t[random_wordform_length-1]
# exactCMsByLengthByWordformIndex_torch[random_wordform_length-1][Ws_l_t[random_wordform_length-1].index(random_wordform)]

'⋊.ɹ.ɪ.d.i.m.⋉.⋉'

8715

8

True

tensor([[0.0070, 0.0140, 0.0082, 0.0071, 0.0070],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070],
        [0.0070, 0.0073, 0.0244, 0.0134, 0.0070],
        [0.0070, 0.0073, 0.5550, 0.0070, 0.0068],
        [0.0068, 0.0071, 0.0078, 0.0068, 0.0066],
        [0.0255, 0.1365, 0.0082, 0.0071, 0.0070],
        [0.0068, 0.0072, 0.0080, 0.0069, 0.0068],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070],
        [0.0068, 0.0073, 0.0081, 0.0070, 0.0070],
        [0.0070, 0.0073, 0.0082, 0.7307, 0.0070],
        [0.0052, 0.0063, 0.0070, 0.0061, 0.0070],
        [0.0068, 0.0073, 0.0081, 0.0070, 0.0070],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070],
        [0.0070, 0.0073, 0.0081, 0.0133, 0.7078],
        [0.0070, 0.0073, 0.0156, 0.0071, 0.0070],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070],
        [0.0070, 0.0138, 0.0081, 0.0070, 0.0068],
        [0.0068, 0.0073, 0.0081, 0.0070, 0.0070],
        [0.0070, 0.0073, 0.1053, 0.0071, 0.0070],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070],


False

In [185]:
exactCMsByLengthByWordformIndex_torch[5].shape

torch.Size([173, 39, 3])

In [186]:
CMsByLengthByWordformIndex_torch[10].dtype
exactCMsByLengthByWordformIndex_torch[10].dtype

torch.float32

torch.float32

In [187]:
random_wordform
ds2t(random_wordform)[1:-1]

'⋊.ɹ.ɪ.d.i.m.⋉.⋉'

('ɹ', 'ɪ', 'd', 'i', 'm', '⋉')

In [188]:
random_wordform
ds2t(random_wordform)
random_wordform_idx
random_wordform_length
random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx]
'--'
retrieve_CM_for(random_wordform, len(ds2t(random_wordform)))
np.allclose(retrieve_CM_for(random_wordform, len(ds2t(random_wordform))),
            CMsByLengthByWordformIndex[random_wordform_length][random_wordform_idx])
'--'
CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx].shape
random_wordform_CM

'⋊.ɹ.ɪ.d.i.m.⋉.⋉'

('⋊', 'ɹ', 'ɪ', 'd', 'i', 'm', '⋉', '⋉')

8715

8

'--'

array([[0.00700978, 0.01400384, 0.00817117, 0.00705364, 0.00700978,
        0.        ],
       [0.00700978, 0.00734851, 0.00817117, 0.00705364, 0.00700978,
        0.        ],
       [0.00700978, 0.00734851, 0.02436973, 0.01344191, 0.00700978,
        0.        ],
       [0.00700978, 0.00725118, 0.55496706, 0.00696021, 0.00682531,
        0.        ],
       [0.00682531, 0.00705778, 0.00784789, 0.00677457, 0.00664084,
        0.        ],
       [0.02545656, 0.13652874, 0.00817117, 0.00705364, 0.00700978,
        0.        ],
       [0.00682531, 0.00715513, 0.00795614, 0.00686802, 0.00682531,
        0.        ],
       [0.00700978, 0.00734851, 0.00817117, 0.00705364, 0.00700978,
        0.        ],
       [0.00682531, 0.00725118, 0.00806294, 0.00696021, 0.00700978,
        0.        ],
       [0.00700978, 0.00734851, 0.00817117, 0.73074413, 0.00700978,
        0.        ],
       [0.0051651 , 0.00630793, 0.00701409, 0.00605481, 0.00700978,
        0.        ],
       [0.00682531, 0

True

'--'

torch.Size([39, 6])

tensor([[0.0070, 0.0140, 0.0082, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.0244, 0.0134, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.5550, 0.0070, 0.0068, 0.0000],
        [0.0068, 0.0071, 0.0078, 0.0068, 0.0066, 0.0000],
        [0.0255, 0.1365, 0.0082, 0.0071, 0.0070, 0.0000],
        [0.0068, 0.0072, 0.0080, 0.0069, 0.0068, 0.0000],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070, 0.0000],
        [0.0068, 0.0073, 0.0081, 0.0070, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.0082, 0.7307, 0.0070, 0.0000],
        [0.0052, 0.0063, 0.0070, 0.0061, 0.0070, 0.0000],
        [0.0068, 0.0073, 0.0081, 0.0070, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.0081, 0.0133, 0.7078, 0.0000],
        [0.0070, 0.0073, 0.0156, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0073, 0.0082, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0138, 0.0081, 0.0070, 0.0068, 0.0000],
        [0.006

In [189]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G         17G        5.7G        8.2M        8.5G         13G
Swap:          2.0G        1.3G        723M


In [190]:
if not benchmark:
    del pY0k_X0ks
    del pY0f_X0fs
    del CMsByLengthByWordformIndex
    del exactCMsByLengthByWordformIndex
    del pW_C
    del pX0f_C
    del p3Y1X012
    if not r and e:
        del cousin_mats
        del sphere_mats

In [191]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        7.5G         15G        8.2M        8.5G         23G
Swap:          2.0G        1.3G        723M


In [192]:
# del exactCMsByLengthByWordformIndex_torch

In [193]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        7.5G         15G        8.2M        8.5G         23G
Swap:          2.0G        1.3G        723M


In [194]:
# del Ws_l_t

In [195]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        7.5G         15G        8.2M        8.5G         23G
Swap:          2.0G        1.3G        723M


## Channel sampler

In [196]:
# if not r:
def depthSampler2a_t(CM, m=1):
    '''
    Given 
     - a sequence of channel distributions (i.e. a pytorch tensor)
       associated with some prefix x_0^i of the lexicon
     - a number of samples m
    this returns m samples from the channel distribution p(Y_0^i|x_0^i),
    where each 'sample' (i.e. channel string) is a sequence of one hot vectors.

    Let l = i+1:
    |depthSampler2a_t(|CM| = (|Y1|, l), m)| = (m, l, |Y1|)
    '''
    stack = torch.zeros((m, CM.shape[1], CM.shape[0]))
#         print(f'{stack.dtype}')
    for eachStack in torch.arange(m):
        for i in torch.arange(CM.shape[1]):
            stack[eachStack, i] = torch.distributions.Multinomial(1, CM[:,i]).sample()
    return stack
#         return torch.squeeze(stack)
# else:
#     def depthSampler2a_t(xCM, m=1):
#         stack = torch.zeros((m, xCM.shape[1], xCM.shape[0]))
#         for eachStack in torch.arange(m):
#             for i in torch.arange(xCM.shape[1]):
#                 stack[eachStack, i] = torch.distributions.Multinomial(1, xCM[:,i]).sample()
#         return stack

In [197]:
random_wordform_CM.shape

torch.Size([39, 6])

In [198]:
random_wordform
sampled_channel_sequences = depthSampler2a_t(random_wordform_CM, m=2); sampled_channel_sequences
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], False)
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], True)
sampled_channel_sequences.shape
del sampled_channel_sequences

'⋊.ɹ.ɪ.d.i.m.⋉.⋉'

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

'i.eɪ.n.z.m.⋉'

'⋊.i.eɪ.n.z.m.⋉'

torch.Size([2, 6, 39])

In [199]:
random_wordform
sampled_channel_sequences2 = depthSampler2a_t(random_wordform_CM, m=100)
[channelSeqOHs2ds(each.numpy(), True) for each in sampled_channel_sequences2]
del sampled_channel_sequences2

'⋊.ɹ.ɪ.d.i.m.⋉.⋉'

['⋊.eɪ.ɪ.b.i.ŋ.⋉',
 '⋊.ɛ.ɪ.u.i.m.⋉',
 '⋊.ɹ.ɪ.d.ʃ.m.⋉',
 '⋊.ɹ.ɪ.d.i.m.⋉',
 '⋊.ɹ.ɔɪ.t.i.m.⋉',
 '⋊.ɹ.ʊ.t.i.m.⋉',
 '⋊.ɹ.ɪ.d.i.u.⋉',
 '⋊.ɛ.eɪ.d.ɛ.m.⋉',
 '⋊.ɹ.ɪ.d.i.m.⋉',
 '⋊.ɹ.z.t.i.ð.⋉',
 '⋊.ɹ.ɛ.d.tʃ.ŋ.⋉',
 '⋊.ɹ.ɪ.t.ɔɪ.m.⋉',
 '⋊.ɹ.ɪ.d.i.ŋ.⋉',
 '⋊.ɹ.ɪ.t.i.m.⋉',
 '⋊.i.dʒ.d.i.m.⋉',
 '⋊.ɹ.ɪ.eɪ.w.m.⋉',
 '⋊.ɹ.ɪ.ɹ.oʊ.m.⋉',
 '⋊.d.ʌ.d.i.m.⋉',
 '⋊.ɹ.l.d.k.m.⋉',
 '⋊.ʌ.aʊ.d.ɚ.m.⋉',
 '⋊.ɹ.ʌ.θ.aʊ.m.⋉',
 '⋊.ɹ.ɪ.d.i.ɚ.⋉',
 '⋊.ɹ.ɛ.d.i.m.⋉',
 '⋊.l.ɪ.d.θ.m.⋉',
 '⋊.z.ɪ.d.m.u.⋉',
 '⋊.p.ɪ.d.i.m.⋉',
 '⋊.ɹ.ɪ.d.i.m.⋉',
 '⋊.ɹ.ɪ.d.ɹ.ɛ.⋉',
 '⋊.ɹ.ɪ.ʊ.i.m.⋉',
 '⋊.s.ɑ.d.i.l.⋉',
 '⋊.ɑ.ɪ.t.ð.m.⋉',
 '⋊.ɹ.ɪ.g.i.m.⋉',
 '⋊.ɹ.eɪ.n.i.m.⋉',
 '⋊.ɹ.ɪ.æ.i.m.⋉',
 '⋊.ŋ.p.æ.i.m.⋉',
 '⋊.g.p.aʊ.i.m.⋉',
 '⋊.s.ɪ.k.tʃ.ɪ.⋉',
 '⋊.ɹ.ɪ.ɑ.m.u.⋉',
 '⋊.ɹ.s.d.i.ɑ.⋉',
 '⋊.v.ɪ.d.i.u.⋉',
 '⋊.ɹ.ɪ.d.i.m.⋉',
 '⋊.u.ɪ.d.v.m.⋉',
 '⋊.aɪ.eɪ.d.oʊ.m.⋉',
 '⋊.ɹ.ɛ.n.i.m.⋉',
 '⋊.ɹ.ɛ.d.i.m.⋉',
 '⋊.ɹ.ɛ.ɑ.ʒ.ŋ.⋉',
 '⋊.ɹ.ʒ.d.ʌ.m.⋉',
 '⋊.g.ɪ.d.i.m.⋉',
 '⋊.ɹ.j.oʊ.i.ŋ.⋉',
 '⋊.n.ɛ.t.i.m.⋉',
 '⋊.ɹ.eɪ.l.u.ð.⋉',
 '⋊.b.ɛ.d.i.f.⋉',
 '⋊.h.oʊ.ʃ.i.k.⋉',
 '⋊.ɹ.ɪ.d.dʒ.m.⋉',
 '⋊.ɹ.ɪ.

In [200]:
if benchmark:
    random_wordform
    sampled_channel_sequences3 = [sample_pY0iX0k(random_wordform) for each in range(50)]; sampled_channel_sequences3

In [201]:
if benchmark:
    len(ds2t('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h'))
    y0kOHmap('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h')
    y0kOHmap('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h').shape

    np.array(list(map(y0kOHmap, sampled_channel_sequences3))).shape
    np.array(list(map(y0kOHmap, [sample_pY0iX0k(random_wordform) for each in range(50)])))
    del sampled_channel_sequences3

In [202]:
if benchmark:
    random_wordform_CM.dtype
    type(sample_pY0iX0k(random_wordform, debug=True)[1])
    sample_pY0iX0k(random_wordform, debug=True)[1].dtype

    torch.equal(torch.tensor(sample_pY0iX0k(random_wordform, debug=True)[1]).type(my_tt), 
                random_wordform_CM)

In [203]:
random_wordform
random_wordform_length
random_wordform_CM.shape
depthSampler2a_t(random_wordform_CM, m=50).shape

'⋊.ɹ.ɪ.d.i.m.⋉.⋉'

8

torch.Size([39, 6])

torch.Size([50, 6, 39])

In [204]:
if benchmark:
    %timeit depthSampler2a_t(random_wordform_CM, m=50)

In [205]:
if benchmark:
    %timeit depthSampler2a_t(random_wordform_CM, m=1000)

In [206]:
def depthSamplerB_t(CM, m=1):
    '''
    Given 
     - a sequence of channel distributions (i.e. a pytorch tensor)
       associated with some prefix x_0^i of the lexicon
     - a number of samples m
    this returns m samples from the channel distribution p(Y_0^i|x_0^i),
    where each 'sample' (i.e. channel string) is a sequence of one hot vectors.

    Let l = i+1:
    |depthSamplerB_t(|CM| = (|Y1|, l), m)| = (m, l, |Y1|)
    '''
    return torch.stack([torch.stack([torch.distributions.Multinomial(1, CM[:,i]).sample()
                                      for i in torch.arange(CM.shape[1])]).type(torch.uint8) 
                        for eachStack in torch.arange(m)])
#     stack = torch.zeros((m, CM.shape[1], CM.shape[0]))
# #         print(f'{stack.dtype}')
#     for eachStack in torch.arange(m):
#         for i in torch.arange(CM.shape[1]):
#             stack[eachStack, i] = torch.distributions.Multinomial(1, CM[:,i]).sample()
#     return stack

In [207]:
if benchmark:
    %timeit depthSamplerB_t(random_wordform_CM, m=50)

In [208]:
if benchmark:
    %timeit torch.distributions.Multinomial(100, random_wordform_CM[:,0]).sample().type(torch.uint8)

In [209]:
if benchmark:
    %timeit torch.distributions.Multinomial(1, random_wordform_CM[:,0]).sample(sample_shape=(100,)).type(torch.uint8)

In [210]:
if benchmark:
    %timeit random_wordform_CM[:,0].multinomial(100, replacement=True) #already shuffled

In [211]:
def getBatchSizes(size_to_batch, num_batches):
    elements_per_batch = size_to_batch // num_batches
    batch_sizes = elements_per_batch * torch.ones((num_batches,), dtype=torch.int64)
    final_adjustment_batch_size = size_to_batch % num_batches
    if final_adjustment_batch_size != 0:
        batch_sizes = list(batch_sizes)
        batch_sizes = batch_sizes + [final_adjustment_batch_size]
        batch_sizes = torch.tensor(batch_sizes)
    return batch_sizes

getBatchSizes(100, 6)

tensor([16, 16, 16, 16, 16, 16,  4])

In [212]:
def depthSamplerC_t(CM, m=1):
# def depthSamplerC_t(CM, m=1, parallel=False, numBatches=-1):
    '''
    Given 
     - a sequence of channel distributions (i.e. a pytorch tensor)
       associated with some prefix x_0^i of the lexicon
     - a number of samples m
    this returns m samples from the channel distribution p(Y_0^i|x_0^i),
    where each 'sample' (i.e. channel string) is a sequence of one hot vectors.

    Let l = i+1:
    |depthSamplerC_t(|CM| = (|Y1|, l), m)| = (m, l, |Y1|)    
    '''
#     If parallel is True and numBatches is an int > 0, then the m samples will be 
#     generated in parallel, in numBatches batches via joblib. 
    
    s = CM.shape[0]
    l = CM.shape[1]
    
#     stampedNote('Generating samples...')
    #each of the l rows contains m sampled Y1 indices
    # where the samples in each row are drawn from the l'th column of CM
    stack_of_sampled_Y1_indices = torch.multinomial(CM.t(), m, replacement=True) #:: (l, m)

#     if not parallel:
#         #each of the l rows contains m sampled Y1 indices
#         # where the samples in each row are drawn from the l'th column of CM
#         stack_of_sampled_Y1_indices = torch.multinomial(CM.t(), m, replacement=True) #:: (l, m)
#     elif parallel and numBatches > 0:
#         CM_t = CM.t()
#         batch_sizes = getBatchSizes(m, numBatches)
        
#         stack_of_sampled_Y1_indices = torch.cat(par(delayed(torch.multinomial)(CM_t, batch_size, replacement=True)
#                                                     for batch_size in batch_sizes), 
#                                                 dim=1) #:: (l, m)
#     else:
#         raise Exception(f'If parallel is True, numbatches must be > 0, got {numBatches} instead.')
#     print(f'sample dtype: {stack_of_sampled_Y1_indices.dtype}')
#     print(f'sample shape: {stack_of_sampled_Y1_indices.shape}')
    
#     stampedNote('Converting to OH vectors...')
    #convert each of the l rows (of length m) into a cat'd sequence (of length s * m) of OH vectors
    deck_of_stacks_of_OHs = torch.zeros((l, s * m), dtype=torch.uint8) # :: (l, s * m)
    
#     stampedNote('Scattering...')
    # stack_of_sampled_Y1_indices[i] is a sequence of m Y1 indices, but
    # WRT deck_of_stacks_of_OHs, stack_of_sampled_Y1_indices[i] * offset represents 
    #   a set of column indices to set to 1
    offset = s * torch.arange(m)
    deck_of_stacks_of_OHs.scatter_(1, # we are specifying m *columns* (hence 1) for each row
                                   stack_of_sampled_Y1_indices + offset, # each row is a sequence of columns to be filled with...
                                   1) #...a 1 at all column indices
#     return deck_of_stacks_of_OHs

    # PERFORMANCE NOTE: *this* is by far the slowest part of sample generation when split+stack is used rather than reshape
#     stampedNote('Splitting...')
    #split along axis=1 into s-sized chunks
    deck_of_stacks_of_OHs = torch.stack( torch.split( deck_of_stacks_of_OHs, s, 1) ) # :: = (m, l, s) => each of the m (l,s) matrices is a channel sequence represented as l (s,) OH vectors
    #2-6x faster at scale and simplistitc testing shows the result is the same as stack->split, but downstream results are *clearly wrong*
#     deck_of_stacks_of_OHs2 = torch.reshape(deck_of_stacks_of_OHs, (m, l, s)) 
#     return deck_of_stacks_of_OHs, deck_of_stacks_of_OHs2
#     stampedNote('Done.')
#     deck_of_stacks_of_OHs = torch.reshape(deck_of_stacks_of_OHs, (m, l, s))
    
    return deck_of_stacks_of_OHs

# @torch.jit.script
def depthSamplerD_t(CM, m=1, numBatches=1):
    '''
    Wrapper call for using joblib to parallelize sampling using
    depthSamplerC_t. 
    
    Performance gain comes from two properties of depthSamplerC_t
        - the single-process nature of the pytorch sampling calls
        - the poor scaling of the last step of depthSamplerC_t
    and a balance of properties of m and numBatches
        - m must be fairly large (>> 10^6)
        - the number of batches must be appropriately sized.
    
    E.g. For 
      (machine, input data, m, numBatches) = (Quine, newdic, 5e6, 10)
    depthSamplerD_t is about 1.5x faster than depthSamplerC_t.
    '''
    sample_batch_sizes = getBatchSizes(m, numBatches)
    
    sample_batches = tuple(par(delayed(depthSamplerC_t)(CM, m=batch_size)
                               for batch_size in sample_batch_sizes))
    combined_samples = torch.cat(sample_batches)
    return combined_samples

def depthSamplerE_t(CM, m=1):
    return torch.distributions.Multinomial(1, CM.t()).sample(sample_shape=(m,)).type(torch.uint8)

In [213]:
if benchmark:
    %timeit depthSamplerC_t(random_wordform_CM, m=50)

In [214]:
if benchmark:
    %timeit depthSamplerE_t(random_wordform_CM, m=50)

In [215]:
if benchmark:
    %timeit depthSamplerC_t(random_wordform_CM, m=100000)

In [216]:
if benchmark:
    %timeit depthSamplerE_t(random_wordform_CM, m=100000)

In [217]:
if benchmark:
    #42.6s / quine
    #15.1s / kotoba
    exampleSample = depthSamplerC_t(random_wordform_CM, m=5000000).shape
    del exampleSample

In [218]:
if benchmark:
    #28.1s / quine
    #6.92s / kotoba
    exampleSample = depthSamplerD_t(random_wordform_CM, m=5000000, numBatches=10).shape
    del exampleSample

In [219]:
# 30.7s / quine, random_wordform_CM.shape = (39,6), using stack+split at end instead of reshape
if benchmark:
    %timeit depthSamplerC_t(random_wordform_CM, m=5000000)

In [220]:
def make_OH(size, hot_bit_idx):
    locations = torch.tensor([hot_bit_idx], dtype=torch.long)
    values = torch.ones(1, dtype=torch.uint8)
    return torch.zeros(size, dtype=torch.uint8).scatter_(0,
                                                         locations,
                                                         values)

def make_n_OHs(n, size, hot_bit_idx):
    index_vector = torch.arange(n)
    offsets = (size * index_vector)
    locations = hot_bit_idx + offsets
    # line below makes function 10-12x slower
#     locations = torch.tensor([hot_bit_idx + i * size for i in torch.arange(n)], dtype=torch.long)
    values = torch.ones(n, dtype=torch.uint8)
    vstacked = torch.zeros(size*n, dtype=torch.uint8).scatter_(0,
                                                               locations,
                                                               values)

    #two lines below are basically the same time/memory wise
    OHs = torch.stack(torch.split(vstacked, size)) #mildly more memory efficient
#     OHs = torch.reshape(vstacked, (n, size)) #slightly faster, but may risk weird copy vs. view behavior downstream
    return OHs

def make_OHs_helper(types_and_counts, size, m):
    '''
    types_and_counts :: (m, 2)
    types_and_counts[i] = a column vector representing (type_idx, type_count), where
                           - 0 <= type_idx <= size-1
    '''
    tc = types_and_counts
#     index_vector = torch.arange(n)
#     locations = hot_bit_idx + (size * index_vector)
    num_types = tc.shape[0]
    
    #each index appears only in a 'chunk' of copies of itself...
    # 'type_index' = hot_bit_idx in simpler function above...
    type_index_of_each_OH_token = torch.cat([tc[i,0] * torch.ones(tc[i,1], dtype=torch.uint8) 
                                             for i in torch.arange(num_types)])
    
    shuffled_type_index_of_each_OH_token = type_index_of_each_OH_token
    
    OH_token_index_vector = torch.arange(m)
    offsets = (size * OH_token_index_vector)
    locations = shuffled_type_index_of_each_OH_token.type(torch.long) + offsets
    
#     locations = []
#     for type_idx in tc[:,0]: #first column is the set of type indices
#         for each in range(tc):
#             locations.append(type_idx)
#     locations = torch.tensor([tc[i,0] + j * size for j in torch.arange(tc[i,1])], dtype=torch.long)
#     locations = torch.tensor([hot_bit_idx + i * size for i in torch.arange(m)], dtype=torch.long)
    values = torch.ones(m, dtype=torch.uint8)
    vstacked = torch.zeros(size*m, dtype=torch.uint8).scatter_(0,
                                                               locations,
                                                               values)
    OHs = torch.stack(torch.split(vstacked, size)) #mildly more memory efficient
#     OHs = torch.reshape(vstacked, (n, size)) #slightly faster, but may risk weird copy vs. view behavior downstream
    return OHs

def makeTypeCountMatrix(sampleVector):
    type_indices = sampleVector.nonzero().squeeze().type(torch.uint8)
    type_counts = torch.gather(sampleVector, 0, type_indices).type(torch.uint8)
#     type_counts = torch.index_select(sampleVector, 0, type_indices)
    return torch.stack((type_indices, type_counts)).t()

def make_OHs(sampleVector):
    '''
    sampleVector is a sparse non-negative integer vector of shape (|Y1s|,).
    
    It represents a sample of size m from a distribution over Y1s, where 
     - the value at index i indicates the count of sampled objects associated with index i.
    
    This returns a uniformly shuffled stack of one-hot vectors representing the sample.
    '''
    my_size = sampleVector.shape[0]
    m = sampleVector.sum().item()
    sample_type_indices = sampleVector.nonzero().squeeze().type(torch.long)
    sample_counts = torch.gather(sampleVector, 0, sample_type_indices).type(torch.long)
#     sample_counts = torch.index_select(sampleVector, 0, sample_type_indices)
    types_and_counts = torch.stack((sample_type_indices, sample_counts)).type(torch.uint8)
    return types_and_counts

# make_OH(39, 6).shape
# make_n_OHs(2, 39, 6).shape
make_n_OHs(20, 39, 6).shape
# print('----')
# make_n_OHs_alt(20, 39, 6).shape
# make_OHs(ten_samples).shape

torch.Size([20, 39])

In [221]:
if benchmark:
    %timeit make_n_OHs(100000, 39, 6)

In [222]:
if benchmark:
    %timeit depthSamplerB_t(random_wordform_CM, m=50)

In [223]:
if benchmark:
    %timeit depthSamplerB_t(random_wordform_CM, m=1000)

In [224]:
if benchmark:
    %timeit depthSamplerC_t(random_wordform_CM, m=50)

In [225]:
if benchmark:
    %timeit depthSamplerC_t(random_wordform_CM, m=1000)

## Sample-parallel posterior estimator

In [226]:
# import opt_einsum as oe

In [227]:
# if not r:
#     def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM=None, m = 50, x0k=None):
def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, c_idx, m = 50):
    '''
    If ⋊ x_0 x_1 ... x_i x_k denotes
    the produced prefix r of the speaker's intended wordform w,
     where
       x_i = the last fully produced segment
       k = i+1
       x_k = the next segment
       |⋊ x_0 x_1 ... x_i x_k| = l + 2
       l = i+1 = the length of the produced prefix (not incl. 
                 the left word edge or the next segment)

    then

    x0k_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
     where x0k_CM[:,j] is the distribution p(Y_j|X0^k has been produced)

    xhat0f_idx is the index of a wordform w' s.t. 

    pXhat0fX0i_pxt(xhat0f_idx, x0k_CM) ≈ p(w'|r)
    '''
    shape_info = False
    pW = get_pW_t(c_idx)
#         if x0k_CM is None and x0k is None:
#             raise Exception('Must specify one of x0k_CM or x0k.')

    # Computation proceeds in two steps:
    # calculate the denominator p(y0i) for m = 50 sampled y0i's
    # calculate the numerator p(y0i|w')p(w') for m = 50 sampled y0i's


#     xhat0f_idx = Wmap[xhat0f]

#     l = len(ds2t(x0i))
#     x0k_CM = CMsByPrefixIndex[prefixMap[x0i]]

#     my_Q_l = CMsByLengthByWordformIndex[l - 2]

    #Collect m samples from p(Y0i|X0k) = p(Y0i|r), 
    # where each sample is an l-length stack of one-hot vectors
    # where each one-hot vector corresponds to a channel symbol:
#         Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
    Y_prime = depthSamplerC_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#         if x0k_CM is not None:
#             Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#             if x0k is not None:
# #                 print('foo')
#                 print(type(x0k_CM))
#                 print(x0k_CM.shape)
#                 generated_CM = sample_pY0iX0k(x0k, debug=True)[1]
#                 print(type(generated_CM))
#                 print(generated_CM.shape)
#                 generated_CM_t = torch.tensor(generated_CM)
#                 assert torch.allclose(x0k_CM, torch.tensor(sample_pY0iX0k(x0k, debug=True)[1]))
# #                 print('bar')
#         else:
#             Y_prime = torch.tensor(list(map(y0kOHmap, [sample_pY0iX0k(x0k) for each in range(m)])))
#         print('baz')
    l = Y_prime.shape[1]# + 1
    if shape_info:
        print(f'|Y_prime| = {Y_prime.shape} = (m, l, |Y1|)')

    #Grab ALL the relevant-prefix-length channel matrices 
    # for every segmental wordform in the lexicon:
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 3]#.float()
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 1]#.float() #:: (|W|, |Y1|, l)
    my_Q_l = CMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W|, |Y1|, l)
    if shape_info:
        print(f'|Q_l| = {my_Q_l.shape} = (|W|=n, |Y1|, l)')

#         print(f'{Y_prime.dtype}, {my_Q_l.dtype}')

    # NORMALIZATION
#         print(Y_prime.dtype)
#         print(my_Q_l.dtype)
#         V_prime = oe.contract('mli,kil->mkl', Y_prime.type(my_tt), my_Q_l, backend='torch')  # :: (m,n,l)
    V_prime = torch.einsum('mli,kil->mkl', Y_prime.type(my_tt), my_Q_l)  # :: (m,n,l)
#         print(V_prime.dtype)
#         print(f"|V'| = {V_prime.shape} = (m, n, l)")
    M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         print(f"|M'| = {M_prime.shape} = (m, n)")
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, ) <- prior probabilities of each of the m sampled channel prefixes
    N_prime = torch.matmul(M_prime, pW)
#         print(f"|N'| = {N_prime.shape} = (m, )")
#         Z_prime = 1.0 / N_prime # :: (m, )
    Z_prime = torch.ones(N_prime.shape, dtype=my_tt)
    torch.div(input=Z_prime,
              other=N_prime,
              out=Z_prime)
#         print(f"|Z'| = {Z_prime.shape} = (m, )")
    if shape_info:
        print(f"|V'| = {V_prime.shape} = (m, n, l)")
        print(f"|M'| = {M_prime.shape} = (m, n)")
        print(f"|N'| = {N_prime.shape} = (m, )")
        print(f"|Z'| = {Z_prime.shape} = (m, )")

    # NUMERATOR
    L_w = my_Q_l[xhat0f_idx]#.float()
#         print(L_w.dtype)
    V_prime_w = torch.einsum('mij,ji->mi',Y_prime.type(my_tt), L_w)
#         print(V_prime_w.dtype)
#         print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
    O_w = torch.prod(V_prime_w, 1) # :: (m,) likelihoods of each of the m sampled channel prefixes
#         print(f"|O_w| = {O_w.shape} = (m, )")
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,) joint probabilities of xhat0f with each of the m sampled channel prefixes
    U_w = pW[xhat0f_idx] * O_w ## :: (m,) joint probabilities of xhat0f with each of the m sampled channel prefixes
#         print(f"|U_w| = {U_w.shape} = (m, )")
    if shape_info:
        print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
        print(f"|O_w| = {O_w.shape} = (m, )")
        print(f"|U_w| = {U_w.shape} = (m, )")


    if shape_info:
        E = torch.dot(Z_prime, U_w) / m
        print(f"|E| = scalar: type(E) = {type(E)}")
    return torch.dot(Z_prime, U_w) / m

#     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
#        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m
# else:
#     def pXhat0fX0k_pxt(xhat0f_idx, x0i_xCM, m = 50):
#     #     xhat0f_idx = Wmap[xhat0f]

#     #     l = len(ds2t(x0i))
#     #     x0i_xCM = xCMsByPrefixIndex[prefixMap[x0i]]

#     #     my_Q_l = xCMsByLengthByWordformIndex[l - 2]

#         Y_prime = depthSampler2a_t(x0i_xCM, m)#.float()
#         l = Y_prime.shape[1] + 1

#         my_Q_l = xCMsByLengthByWordformIndex_torch[l - 3]#.float()

#         # NORMALIZATION
#         V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
#         Z_prime = 1.0 / N_prime # :: (m, 1)

#         # NUMERATOR
#         L_w = my_Q_l[xhat0f_idx]#.float()
#         V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes

#         return torch.dot(Z_prime, U_w) / m

#     #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
#     #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m

In [228]:
def pXhat0fX0i_pxtn(xhat0f_idx, x0k_CM, c_idx, m = 50):
    return pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, c_idx, m = 50).numpy()

In [229]:
def prefixTox0kCM(r):
    source_wf = list(wordsWithPrefix(r, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    r_t = ds2t(r)
    total_x0k_length = len(r_t)# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0k_cm = CMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    return x0k_cm

In [230]:
CMsByLengthByWordformIndex_torch[3].shape
CMsByLengthByWordformIndex_torch[4].shape
len(CMsByLengthByWordformIndex_torch)

torch.Size([0, 0])

torch.Size([9172, 39, 2])

20

In [231]:
len(Ws_t)

9172

In [232]:
def wordformTox0kCM(w=None, w_idx=None):
#     if w is None and w_idx is None:
#         raise Exception('At least one argument must be specified.')
    if w is None:
        w = Ws_t[w_idx]
        w_t = ds2t(w)
    else:
        w_t = ds2t(w)
        w_idx = Ws_t.index(w)
    
    total_x0f_length = len(w_t) #includes both edge symbols
#     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
#     word_length_noRE = word_length_noLE - 1
#     offset = 2
#     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
#     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
#     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
#     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
    x0f_cm = CMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
    return x0f_cm

# def wordformTox0fCM(w=None, w_idx=None):
# #     if w is None and w_idx is None:
# #         raise Exception('At least one argument must be specified.')
#     if w is None:
#         w = Ws_t[w_idx]
#         w_t = ds2t(w)
#     else:
#         w_t = ds2t(w)
#         w_idx = Ws_t.index(w)
    
#     total_x0f_length = len(w_t) #includes both edge symbols
# #     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
# #     word_length_noRE = word_length_noLE - 1
# #     offset = 2
# #     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
# #     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
# #     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
# #     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
#     x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
#     return x0f_cm

In [233]:
len(ds2t('⋊.m.ɪ.n.ɪ.s.t'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉.⋉'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉.⋉'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉.⋉').shape

7

5

torch.Size([39, 5])

8

6

...

7

torch.Size([39, 7])

10

8

torch.Size([39, 8])

In [234]:
random_source_prefix
len(ds2t(random_source_prefix)) - 2
prefixTox0kCM(random_source_prefix).shape
prefixTox0kCM(random_source_prefix)

'⋊.m.ɪ.s.k.ɪ'

4

torch.Size([39, 4])

tensor([[0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0105, 0.0089, 0.0088],
        [0.0068, 0.0102, 0.0087, 0.0086],
        [0.0070, 0.0950, 0.0090, 0.0089],
        [0.0068, 0.0103, 0.0088, 0.0087],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0068, 0.0105, 0.0089, 0.0170],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0052, 0.0091, 0.0077, 0.0077],
        [0.0068, 0.0105, 0.0089, 0.6665],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.6711, 0.0105, 0.0089, 0.0088],
        [0.0808, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0105, 0.0170, 0.0088],
        [0.0068, 0.0105, 0.6474, 0.0088],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0068, 0.0105, 0.0089, 0.0088],
        [0.0066, 0.0103, 0.0088, 0.0087],
        [0.0068, 0.0103, 0.0169, 0

In [235]:
# for convenience
def pXhat0fX0i_pxtn_conv(w, r, c, m = 50):
    w_idx = Ws_t.index(w)

#     r_idx = Ps_t.index(r)
    source_wf = list(wordsWithPrefix(r, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    total_x0k_length = len(ds2t(r))# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0k_cm = CMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    
    c_idx = Cs_t.index(c)
    
    return pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m)

In [236]:
random_source_prefix
len(ds2t(random_source_prefix)) - 2
CMsByLengthByWordformIndex_torch[len(ds2t(random_source_prefix))][Ws_t.index(list(wordsWithPrefix(random_source_prefix, Ws))[0])].shape
CMsByLengthByWordformIndex_torch[len(ds2t(random_source_prefix))][Ws_t.index(list(wordsWithPrefix(random_source_prefix, Ws))[0])]

'⋊.m.ɪ.s.k.ɪ'

4

torch.Size([39, 4])

tensor([[0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0105, 0.0089, 0.0088],
        [0.0068, 0.0102, 0.0087, 0.0086],
        [0.0070, 0.0950, 0.0090, 0.0089],
        [0.0068, 0.0103, 0.0088, 0.0087],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0068, 0.0105, 0.0089, 0.0170],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0052, 0.0091, 0.0077, 0.0077],
        [0.0068, 0.0105, 0.0089, 0.6665],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.6711, 0.0105, 0.0089, 0.0088],
        [0.0808, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0105, 0.0170, 0.0088],
        [0.0068, 0.0105, 0.6474, 0.0088],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0070, 0.0106, 0.0090, 0.0089],
        [0.0068, 0.0105, 0.0089, 0.0088],
        [0.0066, 0.0103, 0.0088, 0.0087],
        [0.0068, 0.0103, 0.0169, 0

In [237]:
(random_source_wordform, random_source_prefix, random_context)
pXhat0fX0i_pxtn_conv(random_source_wordform, random_source_prefix, random_context)

('⋊.b.ɹ.u.h.ɑ.h.ɑ.⋉.⋉', '⋊.m.ɪ.s.k.ɪ', 'a couple of')

array(5.3868027e-10, dtype=float32)

## Testing

In [238]:
# random_context = choice(Cs_t); random_context
random_context = 'a couple of'
random_context_idx = Cs_t.index(random_context); random_context_idx

1753

In [239]:
random_wordform, random_wordform, random_context
pXhat0fX0i_pxt(random_wordform_idx, 
               random_wordform_CM,
               random_context_idx)

('⋊.ɹ.ɪ.d.i.m.⋉.⋉', '⋊.ɹ.ɪ.d.i.m.⋉.⋉', 'a couple of')

tensor(0.0109)

In [240]:
random_wordform, random_wordform, random_context
pXhat0fX0i_pxt(random_wordform_idx, 
               random_wordform_CM,
               random_context_idx)

('⋊.ɹ.ɪ.d.i.m.⋉.⋉', '⋊.ɹ.ɪ.d.i.m.⋉.⋉', 'a couple of')

tensor(0.0166)

In [241]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')

'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 2
# retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 5)

# np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 5), 
#              CMsByLengthByWordformIndex[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')])

2773

'⋊.k.ɑ.n.t'

5

3

In [242]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 50)

tensor(0.0032)

In [243]:
if benchmark:
    f"m = 10    → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 10) for each in range(10)])/10}"
    f"m = 50    → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 50) for each in range(10)])/10}"
    f"m = 100   → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 100) for each in range(10)])/10}"
    f"m = 250   → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 250)}"
    f"m = 500   → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 500)}"
    f"m = 1000  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 1000)}"
    f"m = 2500  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 2500)}"
    f"m = 5000  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 5000)}"
    f"m = 10000 → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 10000)}"

In [244]:
if benchmark:
    f"m = 50   → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 50)}"
    f"m = 100  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 100)}"
    f"m = 200  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 200)}"
    f"m = 500  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 500)}"
    f"m = 1000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 1000)}"
    f"m = 2000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 2000)}"
    f"m = 5000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 5000)}"

In [245]:
# formerly mean 75.9ms / quine
# formerly mean 39.7ms / quine
# mean 27.6 ms / quine
# mean 3.73 ms / kotoba
if benchmark:
    %timeit pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], Cs_t.index('a couple of'),m = 50)

In [246]:
# formerly mean 80ms on quine
# formerly mean 41.8ms on quine
# mean 27.3ms / quine
# mean 3.35ms / kotoba
if benchmark:
    %timeit pXhat0fX0i_pxtn(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], Cs_t.index('a couple of'), m = 50)

In [247]:
#formerly mean 221ms / quine
# formerly mean 122ms / quine
# formerly mean 69.4ms
# mean 83ms / quine
# mean 7.27ms / kotoba
if benchmark:
    %timeit pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], Cs_t.index('a couple of'), m = 200)

Proportionally, what parts of the calculation take the most time?

In [248]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')
Ps_t.index('⋊.k.ɑ.n.t')

2773

12129

In [249]:
contact_idx = Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')
cont_idx = Ps_t.index('⋊.k.ɑ.n.t')
cont_l = len(ds2t('⋊.k.ɑ.n.t')) # 5
cont_triph_l = cont_l - 2 # 3
cont_CM = CMsByLengthByWordformIndex_torch[cont_l][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')]
assert cont_triph_l == cont_CM.shape[1] # 3
acoupleof_idx = Cs_t.index('a couple of')
acouple_of_pW = get_pW_t(acoupleof_idx)
cont_Q_l = CMsByLengthByWordformIndex_torch[cont_triph_l+2]#.float() #:: (|W|, |Y1|, l)

In [250]:
if benchmark:
    %timeit -r 10 -n 10 depthSampler2a_t(cont_CM, m=50)

In [251]:
if benchmark:
    %timeit -r 10 -n 10 depthSamplerC_t(cont_CM, m=50)

In [252]:
cont_Y_prime = depthSamplerC_t(cont_CM, m=50)

In [253]:
%%timeit -r 10 -n 10

#denominator profiling

cont_Q_l = CMsByLengthByWordformIndex_torch[cont_triph_l+2]#.float() #:: (|W|, |Y1|, l)
cont_V_prime = torch.einsum('mli,kil->mkl', cont_Y_prime.type(my_tt), cont_Q_l)  # :: (m,n,l)
cont_M_prime = torch.prod(cont_V_prime, 2) # :: (m,n)
cont_N_prime = torch.matmul(cont_M_prime, acouple_of_pW) # :: (m, )
# cont_N_prime = torch.einsum('mn,n->m', cont_M_prime, acouple_of_pW) #pretty much the same as two cells below
cont_Z_prime = torch.ones(cont_N_prime.shape, dtype=my_tt)
torch.div(cont_Z_prime, cont_N_prime, out=cont_Z_prime)
# cont_Z_prime = 1.0 / cont_N_prime # :: (m, )

3.69 ms ± 820 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [254]:
# %%timeit -r 10 -n 10

# cont_Q_l = CMsByLengthByWordformIndex_torch[cont_triph_l+2]#.float() #:: (|W|, |Y1|, l)
# cont_V_prime = torch.einsum('mli,kil->mkl', cont_Y_prime.type(my_tt), cont_Q_l)  # :: (m,n,l)
# cont_M_prime = torch.prod(cont_V_prime, 2) # :: (m,n)
# cont_N_prime = torch.ones((50,))
# torch.matmul(cont_M_prime, acouple_of_pW, out=cont_N_prime) # :: (m, ) #pretty much the same as the cell above and below
# # cont_N_prime = torch.matmul(cont_M_prime, acouple_of_pW) # :: (m, )
# # cont_N_prime = torch.einsum('mn,n->m', cont_M_prime, acouple_of_pW) 
# cont_Z_prime = torch.ones(cont_N_prime.shape, dtype=my_tt)
# torch.div(cont_Z_prime, cont_N_prime, out=cont_Z_prime)
# # cont_Z_prime = 1.0 / cont_N_prime # :: (m, )

In [255]:
%%timeit -r 10 -n 10

#denominator profiling - including time for lexicon prior lookup

cont_Q_l = CMsByLengthByWordformIndex_torch[cont_triph_l+2]#.float() #:: (|W|, |Y1|, l)
cont_V_prime = torch.einsum('mli,kil->mkl', cont_Y_prime.type(my_tt), cont_Q_l)  # :: (m,n,l)
cont_M_prime = torch.prod(cont_V_prime, 2) # :: (m,n)
cont_N_prime = torch.matmul(cont_M_prime, get_pW_t(acoupleof_idx)) # <<<< includes time for lexicon prior lookup
cont_Z_prime = torch.ones(cont_N_prime.shape, dtype=my_tt)
torch.div(cont_Z_prime, cont_N_prime, out=cont_Z_prime)
# cont_Z_prime = 1.0 / cont_N_prime # :: (m, 1)

4.28 ms ± 2.41 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [256]:
%%timeit -r 10 -n 10

#numerator profiling

contact_L_w = cont_Q_l[contact_idx]#.float()
contact_cont_V_prime_w = torch.einsum('mij,ji->mi',cont_Y_prime.type(my_tt), contact_L_w)
contact_cont_O_w = torch.prod(contact_cont_V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
 # U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
contact_cont_U_w = acouple_of_pW[contact_idx] * contact_cont_O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes

50.5 µs ± 4.86 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


***Old* Takeaway**: Sampling and the normalization term (denominator) each take up about half the total time.

***Current* takeaway**: the normalization term takes up nearly all of the calculation time.

## Parallelization over samples and contexts

In [257]:
import psutil

def memAvailable(units='GB'):
    if units == 'GB':
        scale = 1e9
    elif units == 'MB':
        scale = 1e6
    return psutil.virtual_memory().available / scale

def memUsed(units='GB'):
    if units == 'GB':
        scale = 1e9
    elif units == 'MB':
        scale = 1e6
    return psutil.virtual_memory().used / scale

if g:
    total_mem_MB = torch.cuda.get_device_properties(0).total_memory / 1e6

def gpuMem():
#     if g and torch.cuda.is_available():
    info = {'total':total_mem_MB,
            'allocated':torch.cuda.memory_allocated(0) / (10^6),
            'cached':torch.cuda.memory_cached(0) / (10^6)}
    return info

    print('Total Memory: {0}'.format(total_mem_MB) )
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
        
def stampedMemNote(msg='', units='GB', includeGPU=False):
    mem_usage = 'VM used vs. available: {0:.2f}{2} vs. {1:.2f}{2}'.format(memUsed(units), memAvailable(units), units)
    if g and includeGPU:
#         g_info = gpuMem()
#         total, alloc, cached = g_info['total'], g_info['allocated'], g_info['cached']
#         gpu_usage = 'GPU mem allocated, cached, total: {0:.2f}MB vs. {1:.2f}MB vs. {1:.2f}MB'.format(alloc, cached, total)
        pass
    if msg == '':
        stampedNote(mem_usage)
#         if g and includeGPU:
#             print(gpu_usage)
        
    stampedNote(msg)
    print('\t'+mem_usage)
#     if g and includeGPU:
#         print('\t'+gpu_usage)

In [258]:
# adapted from https://stackoverflow.com/a/6909532
import math
def factors(n):
    step = 2 if n%2 else 1
    fs =    set(reduce(list.__add__,
                ([i, n//i] for i in range(1, int(math.sqrt(n))+1, step) if n % i == 0)))
    return np.array(sorted(list(fs)))

def closest_factor(target, factors):
#     fs = np.array(sorted(list(factors)))
    fs = factors
    distances = np.abs( target - fs )
    min_dist = np.min(distances)
    min_dist_idx = np.where( distances == min_dist )[0][0]
    return int(fs[min_dist_idx])

def pXhat0fX0i_pxtc(xhat0f_idx, x0k_CM, m = 50, target_num_batches=30):
    '''
    This function is exactly like 
        pXhat0fX0i_pxt
    except it uses einsum (and batched computation) to calculate 
        p(xhat0f|x0k, c) 
    for *all* contexts c, in target_num_batches batches of contexts
    at a time.
    
    
    If ⋊ x_0 x_1 ... x_i x_k denotes
    the produced prefix r of the speaker's intended wordform w,
     where
       x_i = the last fully produced segment
       k = i+1
       x_k = the next segment
       |⋊ x_0 x_1 ... x_i x_k| = l + 2
       l = i+1 = the length of the produced prefix (not incl. 
                 the left word edge or the next segment)

    then

    x0k_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
     where x0k_CM[:,j] is the distribution p(Y_j|X0^k has been produced)

    xhat0f_idx is the index of a wordform w' s.t. 

    pXhat0fX0i_pxt(xhat0f_idx, x0k_CM) ≈ [p(w'|r)]_c for each context c
    '''
    stampedNote('> Begin calculation.')
    l = x0k_CM.shape[1]
    
    m_C = len(Cs_t) * m
    stampedMemNote('Sampling...')
#     stampedNote('Sampling...')
    # :: (c*m, l, s)
#     Y_prime_C = depthSamplerC_t(x0k_CM, m=m_C)
    Y_prime_C = depthSamplerD_t(x0k_CM, m=m_C, numBatches = 10)
    
    # :: (c, m, l, s)
    stampedMemNote('Splitting by context...')
#     stampedNote('Splitting by context...')
    Y_prime_C_split = torch.stack(torch.chunk(Y_prime_C, len(Cs_t), 0))
#     stampedMemNote('Contexts split.')
#     del Y_prime_C
#     stampedMemNote('Deleted Y_prime_C.')
    Q_l = CMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W|, |Y1|, l)
    
    
    num_contexts = len(Cs_t); num_contexts # c
    target_num_batches = target_num_batches
    num_batches = closest_factor(target_num_batches, factors(num_contexts))
    print(f'Target num batches = {target_num_batches}')
    print(f'Actual num batches = {num_batches}')
    stampedNote('Batching contexts...')
    # (b, c/b, m, l, s)
    Y_prime_C_split_batched = torch.stack(torch.chunk(Y_prime_C_split, 
                                                      num_batches,
                                                      0))
#     stampedMemNote('Contexts batched.')
    
    num_batches = Y_prime_C_split_batched.shape[0] # b
    num_contexts_per_batch = Y_prime_C_split_batched.shape[1] # c/b
    print(f'Contexts per batch = {num_contexts_per_batch}')
    
    stampedMemNote('Calculating denominator batches...')
    Z_prime_C = torch.ones((num_contexts, m), dtype=my_tt)
#     stampedMemNote('Z_prime_C created.')
#     denominator_batches = []
    for b in tqdm(range(num_batches), total=num_batches):
        stampedNote('\t>> Begin batch.')
        stampedMemNote('\t>> V_prime_C_b...')
        # :: (c/b, m, n, l)
        V_prime_C_batch = torch.einsum('cmli,kil->cmkl',
                                       Y_prime_C_split_batched.type(my_tt)[b],
                                       Q_l)
#         stampedMemNote('V_prime_C_b created.')
        
        stampedMemNote('\t>> M_prime_C_b...')
        # :: (c/b, m, n)
        M_prime_C_batch = torch.prod(V_prime_C_batch, 3)
#         stampedMemNote('M_prime_C_b created.')
        del V_prime_C_batch
#         stampedMemNote('V_prime_C_b deleted.')
        
        stampedMemNote('\t>> Creating pW_context_b...')
        context_indices = torch.arange(num_contexts_per_batch * b, num_contexts_per_batch * b + num_contexts_per_batch)
        # :: (n, c/b)
        pW_context_batch = pW_C_torch[:,context_indices]
#         del context_indices
#         stampedMemNote('pW_context_b created.')
        
        stampedMemNote('\t>> N_prime_C_b...')
        # :: (c/b, m)
        N_prime_C_batch = torch.einsum('cmn,nc->cm', M_prime_C_batch, pW_context_batch) # :: (c/b,m)
#         stampedMemNote('N_prime_C_b created.')
        del M_prime_C_batch
#         del pW_context_batch
#         stampedMemNote('M_prime_C_b and pW_context_b deleted.')
#         stampedMemNote('M_prime_C_b deleted.')
        stampedMemNote('\t>> Z_prime_C_b...')
#         Z_prime_C_batch_old = 1.0 / N_prime_C_batch # :: (c/b,m)
#         Z_prime_C_batch_local = torch.ones(N_prime_C_batch.shape, dtype=my_tt)
        my_Z_prime_C_batch_region = Z_prime_C[context_indices]
        torch.div(input=my_Z_prime_C_batch_region,
                  other=N_prime_C_batch,
                  out=my_Z_prime_C_batch_region)
        Z_prime_C[context_indices] = my_Z_prime_C_batch_region
#         torch.div(input=Z_prime_C_batch_local,
#                   other=N_prime_C_batch,
#                   out=Z_prime_C_batch_local)
#         if (not torch.equal(Z_prime_C_batch_local, my_Z_prime_C_batch_region)) or (not torch.equal(my_Z_prime_C_batch_region, Z_prime_C[context_indices])):
#             print(f'Z_prime_C_batch_local.shape = {Z_prime_C_batch_local.shape}')
#             print(f'Z_prime_C[context_indices].shape = {my_Z_prime_C_batch_region.shape}')
#             print( Z_prime_C_batch_local == Z_prime_C[context_indices])
#             print( Z_prime_C_batch_local == my_Z_prime_C_batch_region )
#             return Z_prime_C_batch_local, Z_prime_C[context_indices], my_Z_prime_C_batch_region
#         assert torch.equal(Z_prime_C_batch_local, Z_prime_C[context_indices])
#         del context_indices
#         del M_prime_C_batch
#         del context_indices
#         del pW_context_batch
#         del N_prime_C_batch
#         stampedMemNote('context_indices and N_prime_C_batch deleted.')
        stampedNote('\t>> End batch.')
#         stampedNote('Appending Z_prime_C_b...')
#         denominator_batches.append(Z_prime_C_batch)

#     stampedNote('Consolidating Z_prime_C')
#     Z_prime_C = torch.cat(denominator_batches)
#     Z_prime_C.shape # (c,m)
#     del denominator_batches
    del Y_prime_C_split_batched
#     stampedMemNote('Y_prime_C_split_batched deleted.')
    
    stampedMemNote('Calculating numerator...')
    stampedNote('Calculating V_prime_w_C...')
    xhat0f_L_w = Q_l[xhat0f_idx]#.float() # :: (s, l)
    V_prime_w_C = torch.einsum('cmij,ji->cmi',Y_prime_C_split.type(my_tt), xhat0f_L_w) # :: (c,m,l)
#     stampedMemNote('V_prime_w_C created.')
#     del xhat0f_L_w
#     del Y_prime_C_split
#     stampedMemNote('xhat0f_L_w and Y_prime_C_split deleted.')
    stampedNote('Calculating O_w_C...')
    O_w_C = torch.prod(V_prime_w_C, 2) # :: (c,m) likelihoods of each of the m sampled channel prefixes for each of c contexts
#     stampedMemNote('O_w_C created.')
#     del V_prime_w_C
#     stampedMemNote('V_prime_w_C deleted.')
    stampedNote('Calculating U_w_C...')
    # contact_cont_U_w_ALL = pW_C_torch[contact_idx] * contact_cont_O_w_ALL ## :: (c,m) joint probabilities of xhat0f with each of the m sampled channel prefixes for each of the c contexts
    U_w_C = torch.einsum('c,cm->cm', pW_C_torch[xhat0f_idx], O_w_C) ## :: (c,m) joint probabilities of xhat0f with each of the m sampled channel prefixes for each of the c contexts
#     stampedMemNote('U_w_C created.')
#     del O_w_C
#     stampedMemNote('O_w_C deleted.')
    
    stampedMemNote('Calculating E_w_C...')
#     return torch.dot(Z_prime_C, U_w_C) / m
    E_w_C = torch.einsum('cm,cm->c', Z_prime_C, U_w_C) / m
#     stampedMemNote('E_w_C created.')
#     del Z_prime_C
#     del U_w_C
#     stampedMemNote('Z_prime_C and U_w_C deleted.')
    stampedNote('> End calculation.')
    return E_w_C

In [259]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        7.5G         15G        8.2M        8.5G         23G
Swap:          2.0G        1.3G        723M


In [260]:
factors(len(Cs_t))

array([     1,      5,      7,     35,   3037,  15185,  21259, 106295])

In [261]:
closest_factor(30, factors(len(Cs_t)))
# np.min(closest_factor(10, factors(len(Cs_t))))
# np.where( closest_factor(10, factors(len(Cs_t))) == np.min(closest_factor(10, factors(len(Cs_t)))) )
# np.where( closest_factor(10, factors(len(Cs_t))) == np.min(closest_factor(10, factors(len(Cs_t)))) )[0][0]

35

In [262]:
my_range = torch.arange(10); my_range
my_batches = torch.split(my_range, 3, 0); my_batches
my_starts = torch.tensor(list(map(lambda b: b[0], my_batches)))
my_ends = torch.tensor(list(map(lambda b: b[-1]+1, my_batches)))
my_starts
my_ends
torch.stack([my_starts, my_ends])

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

(tensor([0, 1, 2]), tensor([3, 4, 5]), tensor([6, 7, 8]), tensor([9]))

tensor([0, 3, 6, 9])

tensor([ 3,  6,  9, 10])

tensor([[ 0,  3,  6,  9],
        [ 3,  6,  9, 10]])

In [263]:
def split_dimension(size_or_index_range, target_batch_size, startEndRange=False):
    '''
    This is a helper function for use with torch.split. Given the total size 
    (or a range of indices) of one dimension of a tensor and a target number
    of indices per batch size, this returns a structure describing the indices
    in each batch.
    
    If size_or_index_range is an int it represents the size of a dimension
    to split into batches, where each batch (except possibly the last) is of 
    size target_batch_size.
    
    If size_or_index_range is a 1D tensor of indices, it represents a tensor
    of indices to split into batches, wher each batch (except possibly the last)
    is of size target_batch_size.
    
    If startEndRange is True, this returns a shape (b,2) tensor where b is the
    number of batches and each column contains the start index and end index+1
    of a batch. Note that size_or_index_range must be interpretable as a
    contiguous range of indices.
    
    If startEndRange is False, this returns a tuple of b 1D tensors, where b is
    the number of batches and each tensor contains the indices of a batch.
    
    TODO: generalize for use with whatever numpy's analogue of torch.split is...
    '''
    if type(size_or_index_range) == int:
        index_range = torch.arange(size_or_index_range)
    else:
        index_range = size_or_index_range
    
    index_batches = torch.split(index_range, target_batch_size, 0)
    
    if not startEndRange:
        return index_batches
    
    starts = torch.tensor(list(map(lambda b: b[0], index_batches)))
    ends = torch.tensor(list(map(lambda b: b[-1]+1, index_batches)))
    startEndRange = torch.stack([starts, ends])
    return startEndRange

In [264]:
from torch.utils.dlpack import to_dlpack
from torch.utils.dlpack import from_dlpack

In [265]:
# pW_C_cupy = cupy.array(pW_C_torch.numpy())
# type(pW_C_cupy)
# pW_C_cupy.device()

In [266]:
# del pW_C_cupy

In [267]:
# def denominator_batch_process_c(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices):
# # def denominator_batch_process_g(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, V_prime_ce, N_prime_ce):
#     if benchmark:
#         profilingStamps = True
#     else:
#         profilingStamps = False
    
#     profilingStamps = True
#     paranoid = True
    
#     if profilingStamps:
#         stampedMemNote('\t>> Begin batch.', includeGPU=True)
    
# #     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
# #     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch # :: (c/b, m, l, s)
#     with cupy.cuda.Device(0):
#         Y_prime_ALL_batch = cupy.asarray(Y_prime_ALL_split_batched_batch) # :: (c/b, m, l, s)
    
#     cupy.cuda.Device(0).use()
#     mempool = cupy.get_default_memory_pool()

# #     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
# #     print(f'shape Q_l = {Q_l.shape}')
    
#     if profilingStamps:
#         stampedMemNote('\tV_prime...', includeGPU=True)
#     # ::(c/b, m, n, l)
#     V_prime_ALL_batch = cupy.einsum('cmli,kil->cmkl',
#                                      Y_prime_ALL_batch,
#                                      Q_l)
# #     V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
# #                                      Y_prime_ALL_batch,
# #                                      Q_l)
# #     V_prime_ALL_batch = V_prime_ce(Y_prime_ALL_batch, Q_l, backend='torch')
# #     if paranoid:
# #         hasInf = (V_prime_ALL_batch == float("inf")).any().item()
# #         if hasInf:
# #             print('V_prime_ALL_batch has infinities...')
#     del Y_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tM_prime...', includeGPU=True)
#     # :: (c/b, m, n)
#     M_prime_ALL_batch = cupy.prod(V_prime_ALL_batch, 3)
# #     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
# #     if paranoid:
# #         hasInf = (M_prime_ALL_batch == float("inf")).any().item()
# #         if hasInf:
# #             print('M_prime_ALL_batch has infinities...')
#     del V_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tpW_C...', includeGPU=True)
#     # :: (n, c/b)
# #     pW_context_batch = pW_C_torch[:, batch_context_indices].cuda()
# #     pW_context_batch = cupy.asarray(pW_C_cupy[:, batch_context_indices])
#     pW_context_batch = pW_C_cupy[:, batch_context_indices]
#     del batch_context_indices
    
#     if profilingStamps:
#         stampedMemNote('\tN_prime...', includeGPU=True)
#     # :: (c/b, m)
#     N_prime_ALL_batch = cupy.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
# #     N_prime_ALL_batch = N_prime_ce(M_prime_ALL_batch, pW_context_batch, backend='torch') # :: (c/b,m)
# #     if paranoid:
# #         hasInf = (N_prime_ALL_batch == float("inf")).any().item()
# #         if hasInf:
# #             print('N_prime_ALL_batch has infinities...')
#     del M_prime_ALL_batch
#     del pW_context_batch
    
#     if profilingStamps:
#         stampedMemNote('\tZ_prime...', includeGPU=True)
#     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
# #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
# #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch.type(torch.float64) # :: (c/b,m)
# #     Z_prime_ALL_batch = torch.pow(N_prime_ALL_batch, -1)
# #     Z_prime_ALL_batch = torch.pow(N_prime_ALL_batch.type(torch.float64), -1).type(my_tt)
# #     Z_prime_ALL_batch = torch.reciprocal(N_prime_ALL_batch)
# #     Z_prime_ALL_batch = torch.reciprocal(N_prime_ALL_batch.type(torch.float64)).type(my_tt)
# #     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt, device=gpu)
# #     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=torch.float64, device=gpu)
# #     torch.div(input=Z_prime_ALL_batch,
# # #               other=N_prime_ALL_batch,
# #               other=N_prime_ALL_batch.type(torch.float64),
# #               out=Z_prime_ALL_batch)
# #     Z_prime_ALL_batch = Z_prime_ALL_batch.type(my_tt)
#     if paranoid:
#         hasInf = (Z_prime_ALL_batch.get() == float("inf")).any().item()
#         if hasInf:
#             print('Z_prime_ALL_batch has infinities...')
# #     del M_prime_ALL_batch
# #     del batch_context_indices
# #     del pW_context_batch
#     del N_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\t>> End batch.', includeGPU=True)
    
#     cupy.cuda.Device(0).use()
#     result = Z_prime_ALL_batch.get()
# #     result = Z_prime_ALL_batch.cpu()
#     del Z_prime_ALL_batch
    
#     mempool.free_all_blocks()
# #     torch.cuda.empty_cache()
#     return result

In [268]:
# def denominator_batch_process_g(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, exact_wf=False):
# # def denominator_batch_process_g(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, V_prime_ce, N_prime_ce):
#     if benchmark:
#         profilingStamps = False
#         paranoid = True
#     else:
#         profilingStamps = False
    
# #     profilingStamps = False
#     paranoid = True
# #     mempool = cupy.get_default_memory_pool()
    
#     if profilingStamps:
#         stampedMemNote('\t>> Begin batch.', includeGPU=True)
    
# #     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
#     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch # :: (c/b, m, l, s)
 
#     l = Y_prime_ALL_batch.shape[2]
#     L = l+2

# #     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
# #     print(f'shape Q_l = {Q_l.shape}')
    
#     if profilingStamps:
#         stampedMemNote('\tV_prime...', includeGPU=True)
#     # ::(c/b, m, n, l)
# #     V_prime_ALL_batch = V_prime_ce(Y_prime_ALL_batch, Q_l, backend='torch')
#     V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
#                                      Y_prime_ALL_batch,
#                                      Q_l)
# #     if paranoid:
# #         hasInf = (V_prime_ALL_batch == float("inf")).any().item()
# #         if hasInf:
# #             print('V_prime_ALL_batch has infinities...')
#     del Y_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tM_prime...', includeGPU=True)
#     # :: (c/b, m, n)
# #     M_prime_ALL_batch = cupy.prod(cupy.fromDlpack(to_dlpack(V_prime_ALL_batch)), 3)
# #     M_prime_ALL_batch = from_dlpack(M_prime_ALL_batch.toDlpack())
# #     M_prime_ALL_batch = torch.exp(torch.sum(torch.log(V_prime_ALL_batch), 3))
# #     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch.type(torch.float64), 3).type(my_tt)
#     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
# #     if paranoid:
# #         hasInf = (M_prime_ALL_batch == float("inf")).any().item()
# #         if hasInf:
# #             print('M_prime_ALL_batch has infinities...')
#     del V_prime_ALL_batch
# #     mempool.free_all_blocks()
    
#     if profilingStamps:
#         stampedMemNote('\tpW_C...', includeGPU=True)
#     # :: (n, c/b)
#     if not exact_wf:
#         pW_context_batch = pW_C_torch[:, batch_context_indices].cuda()
#     else:
#         pW_context_batch = torch.index_select(pW_C_torch[l_to_w_idxs[L]],
#                                               1,
#                                               batch_context_indices)
# #         pW_context_batch = pW_C_torch[l_to_w_idxs[L], batch_context_indices]
#         if pW_context_batch.dim() == 1:
# #             torch.unsqueeze(pW_context_batch, -1, out=pW_context_batch)
#             pW_context_batch = torch.unsqueeze(pW_context_batch, 0)
#         pW_context_batch = pW_context_batch.cuda()
#     del batch_context_indices
    
#     if profilingStamps:
#         stampedMemNote('\tN_prime...', includeGPU=True)
#     # :: (c/b, m)
# #     N_prime_ALL_batch = cupy.einsum('cmn,nc->cm', M_prime_ALL_batch, 
# #                                     cupy.fromDlpack(to_dlpack(pW_context_batch))) # :: (c/b,m)
#     N_prime_ALL_batch = torch.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
# #     N_prime_ALL_batch = N_prime_ce(M_prime_ALL_batch, pW_context_batch, backend='torch') # :: (c/b,m)
# #     N_prime_ALL_batch_cupy = N_prime_ALL_batch
# #     del N_prime_ALL_batch
# #     if paranoid:
# #         hasInf = (N_prime_ALL_batch == float("inf")).any().item()
# #         if hasInf:
# #             print('N_prime_ALL_batch has infinities...')
#     del M_prime_ALL_batch
#     del pW_context_batch
    
    
#     if profilingStamps:
#         stampedMemNote('\tZ_prime...', includeGPU=True)
# #     N_prime_ALL_batch_cupy = cupy.asarray(N_prime_ALL_batch.cpu().numpy())
# #     N_prime_ALL_batch_dlpack = to_dlpack(N_prime_ALL_batch)
# #     del N_prime_ALL_batch
# #     N_prime_ALL_batch_cupy = cupy.fromDlpack(N_prime_ALL_batch_dlpack)
# #     del N_prime_ALL_batch_dlpack
# #     Z_prime_ALL_batch_cupy = 1.0 / N_prime_ALL_batch_cupy
# #     Z_prime_ALL_batch_cupy = cupy.reciprocal(N_prime_ALL_batch_cupy)
# #     Z_prime_ALL_batch_cupy = cupy.divide(cupy.ones(N_prime_ALL_batch_cupy.shape, dtype=cupy.float32),
# #                                          N_prime_ALL_batch_cupy)
# #     Z_prime_ALL_batch_cupy = cupy.power(N_prime_ALL_batch_cupy, -1.0 * cupy.ones(N_prime_ALL_batch_cupy.shape))
# #     del N_prime_ALL_batch_cupy
# #     Z_prime_ALL_batch = from_dlpack(Z_prime_ALL_batch_cupy.toDlpack())
# #     del Z_prime_ALL_batch_cupy
# #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
# #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch.type(torch.float64) # :: (c/b,m)
# #     Z_prime_ALL_batch = torch.pow(N_prime_ALL_batch, -1)
# #     Z_prime_ALL_batch = torch.pow(N_prime_ALL_batch.type(torch.float64), -1).type(my_tt)
# #     Z_prime_ALL_batch = torch.reciprocal(N_prime_ALL_batch)
# #     Z_prime_ALL_batch = torch.reciprocal(N_prime_ALL_batch.type(torch.float64)).type(my_tt)
#     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt, device=gpu)
# #     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=torch.float64, device=gpu)
#     torch.div(input=Z_prime_ALL_batch,
#               other=N_prime_ALL_batch,
# #               other=N_prime_ALL_batch.type(torch.float64),
#               out=Z_prime_ALL_batch)
#     Z_prime_ALL_batch = Z_prime_ALL_batch.type(my_tt)
#     if paranoid:
#         hasInf = (Z_prime_ALL_batch == float("inf")).any().item()
#         if hasInf:
#             print('Z_prime_ALL_batch has infinities...')
# #         hasInf = (Z_prime_ALL_batch_cupy == float("inf")).any().item()
# #         if hasInf:
# #             print('Z_prime_ALL_batch_cupy has infinities...')
# #     del Z_prime_ALL_batch_cupy
# #     del M_prime_ALL_batch
# #     del batch_context_indices
# #     del pW_context_batch
#     del N_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\t>> End batch.', includeGPU=True)
#     result = Z_prime_ALL_batch.cpu()
#     del Z_prime_ALL_batch
#     torch.cuda.empty_cache()
# #     mempool.free_all_blocks()
#     return result

# # def denominator_batch_process_oe(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, V_prime_ce, N_prime_ce):
# #     profilingStamps = False
# #     if profilingStamps:
# #         stampedMemNote('\t>> Begin batch.')
    
# #     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
 
# # #     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
# # #     print(f'shape Q_l = {Q_l.shape}')
    
# #     if profilingStamps:
# #         stampedNote('\tV_prime...')
# #     # ::(c/b, m, n, l)
# #     V_prime_ALL_batch = V_prime_ce(Y_prime_ALL_batch, Q_l, backend='torch')
# # #     V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
# # #                                      Y_prime_ALL_batch,
# # #                                      Q_l)
# # #     del Y_prime_ALL_batch
    
# #     if profilingStamps:
# #         stampedMemNote('\tM_prime...')
# #     # :: (c/b, m, n)
# #     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
# #     del V_prime_ALL_batch
    
# #     if profilingStamps:
# #         stampedMemNote('\tpW_C...')
# #     # :: (n, c/b)
# #     pW_context_batch = pW_C_torch[:, batch_context_indices]
# # #     del batch_context_indices
    
# #     if profilingStamps:
# #         stampedNote('\tN_prime...')
# #     # :: (c/b, m)
# #     N_prime_ALL_batch = N_prime_ce(M_prime_ALL_batch, pW_context_batch, backend='torch') # :: (c/b,m)
# # #     N_prime_ALL_batch = torch.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
# #     del M_prime_ALL_batch
# # #     del pW_context_batch
    
# #     if profilingStamps:
# #         stampedMemNote('\tZ_prime...')
# # #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
# #     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt)
# #     torch.div(input=Z_prime_ALL_batch,
# #               other=N_prime_ALL_batch,
# #               out=Z_prime_ALL_batch)
# # #     del M_prime_ALL_batch
# # #     del batch_context_indices
# # #     del pW_context_batch
# # #     del N_prime_ALL_batch
# #     if profilingStamps:
# #         stampedNote('\t>> End batch.')
# #     return Z_prime_ALL_batch

# def denominator_batch_process(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, exact_wf=False):
#     profilingStamps = False
#     if profilingStamps:
#         stampedMemNote('\t>> Begin batch.')
    
#     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
 
#     l = Y_prime_ALL_batch.shape[2]
#     L = l+2

# #     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
# #     print(f'shape Q_l = {Q_l.shape}')
    
#     if profilingStamps:
#         stampedNote('\tV_prime...')
#     # ::(c/b, m, n, l)
#     V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
#                                      Y_prime_ALL_batch,
#                                      Q_l)
# #     del Y_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tM_prime...')
#     # :: (c/b, m, n)
#     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
#     del V_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tpW_C...')
#     # :: (n, c/b)
#     if not exact_wf:
#         pW_context_batch = pW_C_torch[:, batch_context_indices]
#     else:
#         pW_context_batch = pW_C_torch[l_to_w_idxs[L], batch_context_indices]
#         if pW_context_batch.dim() == 1:
# #             torch.unsqueeze(pW_context_batch, -1, out=pW_context_batch)
#             pW_context_batch = torch.unsqueeze(pW_context_batch, 0)
# #     del batch_context_indices
    
#     if profilingStamps:
#         stampedNote('\tN_prime...')
#     # :: (c/b, m)
#     N_prime_ALL_batch = torch.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
#     del M_prime_ALL_batch
# #     del pW_context_batch
    
#     if profilingStamps:
#         stampedMemNote('\tZ_prime...')
# #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
#     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt)
#     torch.div(input=Z_prime_ALL_batch,
#               other=N_prime_ALL_batch,
#               out=Z_prime_ALL_batch)
# #     del M_prime_ALL_batch
# #     del batch_context_indices
# #     del pW_context_batch
# #     del N_prime_ALL_batch
#     if profilingStamps:
#         stampedNote('\t>> End batch.')
#     return Z_prime_ALL_batch

In [269]:
def denominator_batch_process_g(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, exact_wf=False):
# def denominator_batch_process_g(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, V_prime_ce, N_prime_ce):
    if benchmark:
        profilingStamps = False
        paranoid = True
    else:
        profilingStamps = False
    
#     profilingStamps = False
    paranoid = True
#     mempool = cupy.get_default_memory_pool()
    
    if profilingStamps:
        stampedMemNote('\t>> Begin batch.', includeGPU=True)
    
#     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
    Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch # :: (c/b, m, l, s)
 
    l = Y_prime_ALL_batch.shape[2]
    L = l+2

#     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
#     print(f'shape Q_l = {Q_l.shape}')
    
    if profilingStamps:
        stampedMemNote('\tV_prime...', includeGPU=True)
    # ::(c/b, m, n, l)
#     V_prime_ALL_batch = V_prime_ce(Y_prime_ALL_batch, Q_l, backend='torch')
    V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
                                     Y_prime_ALL_batch,
                                     Q_l)
#     if paranoid:
#         hasInf = (V_prime_ALL_batch == float("inf")).any().item()
#         if hasInf:
#             print('V_prime_ALL_batch has infinities...')
    del Y_prime_ALL_batch
    
    if profilingStamps:
        stampedMemNote('\tM_prime...', includeGPU=True)
    # :: (c/b, m, n)
    M_prime_ALL_batch = cupy.prod(cupy.fromDlpack(to_dlpack(V_prime_ALL_batch)), 3)
    M_prime_ALL_batch = from_dlpack(M_prime_ALL_batch.toDlpack())
##     M_prime_ALL_batch = torch.exp(torch.sum(torch.log(V_prime_ALL_batch), 3))
#     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch.type(torch.float64), 3).type(my_tt)
#     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
#     if paranoid:
#         hasInf = (M_prime_ALL_batch == float("inf")).any().item()
#         if hasInf:
#             print('M_prime_ALL_batch has infinities...')
    del V_prime_ALL_batch
#     mempool.free_all_blocks()
    
    if profilingStamps:
        stampedMemNote('\tpW_C...', includeGPU=True)
    # :: (n, c/b)
    if not exact_wf:
        pW_context_batch = pW_C_torch[:, batch_context_indices].cuda()
    else:
        pW_context_batch = torch.index_select(pW_C_torch[l_to_w_idxs[L]],
                                              1,
                                              batch_context_indices)
#         pW_context_batch = pW_C_torch[l_to_w_idxs[L], batch_context_indices]
        if pW_context_batch.dim() == 1:
#             torch.unsqueeze(pW_context_batch, -1, out=pW_context_batch)
            pW_context_batch = torch.unsqueeze(pW_context_batch, 0)
        pW_context_batch = pW_context_batch.cuda()
    del batch_context_indices
    
    if profilingStamps:
        stampedMemNote('\tN_prime...', includeGPU=True)
    # :: (c/b, m)
#     N_prime_ALL_batch = cupy.einsum('cmn,nc->cm', M_prime_ALL_batch, 
#                                     cupy.fromDlpack(to_dlpack(pW_context_batch))) # :: (c/b,m)
    N_prime_ALL_batch = torch.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
#     N_prime_ALL_batch = N_prime_ce(M_prime_ALL_batch, pW_context_batch, backend='torch') # :: (c/b,m)
#     N_prime_ALL_batch_cupy = N_prime_ALL_batch
#     del N_prime_ALL_batch
#     if paranoid:
#         hasInf = (N_prime_ALL_batch == float("inf")).any().item()
#         if hasInf:
#             print('N_prime_ALL_batch has infinities...')
    del M_prime_ALL_batch
    del pW_context_batch
    
    
    if profilingStamps:
        stampedMemNote('\tZ_prime...', includeGPU=True)
##     N_prime_ALL_batch_cupy = cupy.asarray(N_prime_ALL_batch.cpu().numpy())
#     N_prime_ALL_batch_dlpack = to_dlpack(N_prime_ALL_batch)
#     del N_prime_ALL_batch
#     N_prime_ALL_batch_cupy = cupy.fromDlpack(N_prime_ALL_batch_dlpack)
#     del N_prime_ALL_batch_dlpack
#     Z_prime_ALL_batch_cupy = cupy.reciprocal(N_prime_ALL_batch_cupy)
##     Z_prime_ALL_batch_cupy = 1.0 / N_prime_ALL_batch_cupy
##     Z_prime_ALL_batch_cupy = cupy.divide(cupy.ones(N_prime_ALL_batch_cupy.shape, dtype=cupy.float32),
##                                          N_prime_ALL_batch_cupy)
##     Z_prime_ALL_batch_cupy = cupy.power(N_prime_ALL_batch_cupy, -1.0 * cupy.ones(N_prime_ALL_batch_cupy.shape))
##     del N_prime_ALL_batch_cupy
#     Z_prime_ALL_batch = from_dlpack(Z_prime_ALL_batch_cupy.toDlpack())
#     del Z_prime_ALL_batch_cupy
##     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
##     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch.type(torch.float64) # :: (c/b,m)
##     Z_prime_ALL_batch = torch.pow(N_prime_ALL_batch, -1)
##     Z_prime_ALL_batch = torch.pow(N_prime_ALL_batch.type(torch.float64), -1).type(my_tt)
##     Z_prime_ALL_batch = torch.reciprocal(N_prime_ALL_batch)
##     Z_prime_ALL_batch = torch.reciprocal(N_prime_ALL_batch.type(torch.float64)).type(my_tt)
    Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt, device=gpu)
##     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=torch.float64, device=gpu)
    torch.div(input=Z_prime_ALL_batch,
              other=N_prime_ALL_batch,
#               other=N_prime_ALL_batch.type(torch.float64),
              out=Z_prime_ALL_batch)
    Z_prime_ALL_batch = Z_prime_ALL_batch.type(my_tt)
    if paranoid:
        hasInf = (Z_prime_ALL_batch == float("inf")).any().item()
        if hasInf:
            print('Z_prime_ALL_batch has infinities...')
#         hasInf = (Z_prime_ALL_batch_cupy == float("inf")).any().item()
#         if hasInf:
#             print('Z_prime_ALL_batch_cupy has infinities...')
#     del Z_prime_ALL_batch_cupy
#     del M_prime_ALL_batch
#     del batch_context_indices
#     del pW_context_batch
    del N_prime_ALL_batch
    
    if profilingStamps:
        stampedMemNote('\t>> End batch.', includeGPU=True)
    result = Z_prime_ALL_batch.cpu()
    del Z_prime_ALL_batch
    torch.cuda.empty_cache()
#     mempool.free_all_blocks()
    return result

# def denominator_batch_process_oe(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, V_prime_ce, N_prime_ce):
#     profilingStamps = False
#     if profilingStamps:
#         stampedMemNote('\t>> Begin batch.')
    
#     Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
 
# #     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
# #     print(f'shape Q_l = {Q_l.shape}')
    
#     if profilingStamps:
#         stampedNote('\tV_prime...')
#     # ::(c/b, m, n, l)
#     V_prime_ALL_batch = V_prime_ce(Y_prime_ALL_batch, Q_l, backend='torch')
# #     V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
# #                                      Y_prime_ALL_batch,
# #                                      Q_l)
# #     del Y_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tM_prime...')
#     # :: (c/b, m, n)
#     M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
#     del V_prime_ALL_batch
    
#     if profilingStamps:
#         stampedMemNote('\tpW_C...')
#     # :: (n, c/b)
#     pW_context_batch = pW_C_torch[:, batch_context_indices]
# #     del batch_context_indices
    
#     if profilingStamps:
#         stampedNote('\tN_prime...')
#     # :: (c/b, m)
#     N_prime_ALL_batch = N_prime_ce(M_prime_ALL_batch, pW_context_batch, backend='torch') # :: (c/b,m)
# #     N_prime_ALL_batch = torch.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
#     del M_prime_ALL_batch
# #     del pW_context_batch
    
#     if profilingStamps:
#         stampedMemNote('\tZ_prime...')
# #     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
#     Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt)
#     torch.div(input=Z_prime_ALL_batch,
#               other=N_prime_ALL_batch,
#               out=Z_prime_ALL_batch)
# #     del M_prime_ALL_batch
# #     del batch_context_indices
# #     del pW_context_batch
# #     del N_prime_ALL_batch
#     if profilingStamps:
#         stampedNote('\t>> End batch.')
#     return Z_prime_ALL_batch

def denominator_batch_process(Y_prime_ALL_split_batched_batch, Q_l, batch_context_indices, exact_wf=False):
    profilingStamps = False
    if profilingStamps:
        stampedMemNote('\t>> Begin batch.')
    
    Y_prime_ALL_batch = Y_prime_ALL_split_batched_batch.type(my_tt) # :: (c/b, m, l, s)
 
    l = Y_prime_ALL_batch.shape[2]
    L = l+2

#     print(f'shape Y_prime_ALL_batch = {Y_prime_ALL_batch.shape}')
#     print(f'shape Q_l = {Q_l.shape}')
    
    if profilingStamps:
        stampedNote('\tV_prime...')
    # ::(c/b, m, n, l)
    V_prime_ALL_batch = torch.einsum('cmli,kil->cmkl',
                                     Y_prime_ALL_batch,
                                     Q_l)
#     del Y_prime_ALL_batch
    
    if profilingStamps:
        stampedMemNote('\tM_prime...')
    # :: (c/b, m, n)
    M_prime_ALL_batch = torch.prod(V_prime_ALL_batch, 3)
    del V_prime_ALL_batch
    
    if profilingStamps:
        stampedMemNote('\tpW_C...')
    # :: (n, c/b)
    if not exact_wf:
        pW_context_batch = pW_C_torch[:, batch_context_indices]
    else:
        pW_context_batch = pW_C_torch[l_to_w_idxs[L], batch_context_indices]
        if pW_context_batch.dim() == 1:
#             torch.unsqueeze(pW_context_batch, -1, out=pW_context_batch)
            pW_context_batch = torch.unsqueeze(pW_context_batch, 0)
#     del batch_context_indices
    
    if profilingStamps:
        stampedNote('\tN_prime...')
    # :: (c/b, m)
    N_prime_ALL_batch = torch.einsum('cmn,nc->cm', M_prime_ALL_batch, pW_context_batch) # :: (c/b,m)
    del M_prime_ALL_batch
#     del pW_context_batch
    
    if profilingStamps:
        stampedMemNote('\tZ_prime...')
#     Z_prime_ALL_batch = 1.0 / N_prime_ALL_batch # :: (c/b,m)
    Z_prime_ALL_batch = torch.ones(N_prime_ALL_batch.shape, dtype=my_tt)
    torch.div(input=Z_prime_ALL_batch,
              other=N_prime_ALL_batch,
              out=Z_prime_ALL_batch)
#     del M_prime_ALL_batch
#     del batch_context_indices
#     del pW_context_batch
#     del N_prime_ALL_batch
    if profilingStamps:
        stampedNote('\t>> End batch.')
    return Z_prime_ALL_batch

In [270]:
def denominator_calc(CM, m, Q_l, target_contexts_per_batch=500, parallel=False):
    profilingStamps = True
    if profilingStamps:
        stampedMemNote('> Begin calculation.')
    num_contexts = len(Cs_t) #c
#     num_batches = len(Y_prime_ALL_split_batched) #b
#     num_contexts_per_batch = torch.tensor(tuple(map(len, Y_prime_ALL_split_batched_indices))) #c/b
    l = CM.shape[1]
#     print(f'l = {l}')
    
    num_samples = m * num_contexts
    
    if profilingStamps:
        stampedNote('Generating samples...')
    #collect enough sampled channel sequences for all c contexts 
    Y_prime_ALL = depthSamplerD_t(CM, m=num_samples, numBatches = 10)
    
    if profilingStamps:
        stampedNote('Splitting samples by context...')
    #split the samples into stacks of m samples per context
    Y_prime_ALL_split = torch.stack(torch.chunk(Y_prime_ALL, num_contexts, 0))
#     print(f'shape Y_prime_ALL_split = {Y_prime_ALL_split.shape}')
    
    if profilingStamps:
        stampedNote('Splitting contexts into batches ...')
    #split the contexts into batches
#     target_contexts_per_batch = 1000
    Y_prime_ALL_split_batched = torch.split(Y_prime_ALL_split,
                                            target_contexts_per_batch,
                                            0)
    Y_prime_ALL_split_batched_indices = split_dimension(num_contexts, target_contexts_per_batch)
    num_batches = len(Y_prime_ALL_split_batched)
    if profilingStamps:
        print(f'Num batches = {num_batches}')
    
    if profilingStamps:
        stampedMemNote('Processing batches...')
    #process the batches
    if parallel:
        Z_prime_ALL = torch.cat(list( par(delayed(denominator_batch_process)(Y_prime_ALL_split_batched_batch, Q_l, indices)
                                      for Y_prime_ALL_split_batched_batch, indices in zip(Y_prime_ALL_split_batched, 
                                                                                          Y_prime_ALL_split_batched_indices)) ))
    else:
        Z_prime_ALL = torch.cat([denominator_batch_process(Y_prime_ALL_split_batched_batch, Q_l, indices)
                                 for Y_prime_ALL_split_batched_batch, indices in tqdm(zip(Y_prime_ALL_split_batched,
                                                                                          Y_prime_ALL_split_batched_indices),
                                                                                      total=num_batches)]) 
    if profilingStamps:
        stampedMemNote('> Batches processed.')
    return Z_prime_ALL

In [325]:
def pXhat0fX0i_ptc(xhat0f_idx, x0k_CM, m = 50, target_contexts_per_batch=600, parallel=True, use_gpu=False, exact_wf=False):
    '''
    This function is exactly like 
        pXhat0fX0i_pxt
    except it uses einsum (and batched computation) to calculate 
        p(xhat0f|x0k, c) 
    for *all* contexts c, in batches of target_contexts_per_batch
    contexts at a time (default 750).
    
    Currently known optimal target_contexts_per_batch settings:
    Quine CPU/Parallel newdic -> 750
    Solomonoff CPU/Parallel newdic -> 750
    Kotoba CPU/Parallel newdic -> 600-650
    Kotoba GPU newdic -> 300-375
    
    
    If ⋊ x_0 x_1 ... x_i x_k denotes
    the produced prefix r of the speaker's intended wordform w,
     where
       x_i = the last fully produced segment
       k = i+1
       x_k = the next segment
       |⋊ x_0 x_1 ... x_i x_k| = l + 2
       l = i+1 = the length of the produced prefix (not incl. 
                 the left word edge or the next segment)

    then

    x0k_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
     where x0k_CM[:,j] is the distribution p(Y_j|X0^k has been produced)

    xhat0f_idx is the index of a wordform w' s.t. 

    pXhat0fX0i_ptc(xhat0f_idx, x0k_CM) ≈ [p(w'|r)]_c for each context c
    '''
    profilingStamps = True
#     if not benchmark:
    mininterval = 10 #minimum of 10 seconds between tqdm progress updates
    
    if profilingStamps:
        stampedNote('> Begin calculation.')
    l = x0k_CM.shape[1]
    num_contexts = len(Cs_t) #c
    
    if profilingStamps:
        print(f'l = {l}')
        
    if exact_wf:
        if xhat0f_idx not in l_to_w_idxs[x0k_CM.shape[1]+2]:
            return torch.zeros((num_contexts,))
        else:
            L, xhat0f_l_idx = w_idx_to_l_w_l_idx(xhat0f_idx) # L = l+2
    
    m_C = num_contexts * m

    if profilingStamps:
        stampedMemNote('Sampling...')
    # :: (c*m, l, s)
#     Y_prime_C = depthSamplerC_t(x0k_CM, m=m_C)
    Y_prime_C = depthSamplerD_t(x0k_CM, m=m_C, numBatches = 10)
    
    # :: (c, m, l, s)
    if profilingStamps:
        stampedMemNote('Splitting samples by context...')
    Y_prime_C_split = torch.stack(torch.chunk(Y_prime_C, len(Cs_t), 0))
#     stampedMemNote('Contexts split.')
#     del Y_prime_C
#     stampedMemNote('Deleted Y_prime_C.')
    if not exact_wf:
        Q_l = CMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W|, |Y1|, l)
    else:
        Q_l = exactCMsByLengthByWordformIndex_torch[l+2]
    if use_gpu:
        Q_l_g = Q_l.cuda()
    
    
    if profilingStamps:
        stampedNote('Splitting contexts into batches ...')
    #split the contexts into batches
    Y_prime_C_split_batched = torch.split(Y_prime_C_split,
                                          target_contexts_per_batch,
                                          0)
    Y_prime_C_split_batched_indices = split_dimension(num_contexts, target_contexts_per_batch)
    num_batches = len(Y_prime_C_split_batched)
    if profilingStamps:
        print(f'Num batches = {num_batches}')
    
#     if profilingStamps:
#         stampedMemNote('Pre-calculating contract expressions for each batch...')
#     #calculate contract expressions for each batch
#     main_batch_size = len(Y_prime_C_split_batched_indices[0])
#     final_batch_size = len(Y_prime_C_split_batched_indices[-1])
#     batch_sizes = tuple(map(len, Y_prime_C_split_batched_indices))
#     n = pW_C_torch.shape[0]
#     s = Y_prime_C.shape[2]
    
#     V_prime_index_str = 'cmli,kil->cmkl'
#     Y_prime_ALL_batch_shape = lambda contexts_per_batch: (contexts_per_batch, m, l, s) # (c/b, m, l, s)
#     Q_l_batch_shape = tuple(Q_l.shape) # (n, s, l) = (|W|, |Y1|, l)
#     V_prime_ALL_shape_for_batch = lambda contexts_per_batch: (Y_prime_ALL_batch_shape(contexts_per_batch), Q_l_batch_shape)
#     V_prime_ALL_shape_main_batch = Y_prime_ALL_batch_shape(main_batch_size), Q_l_batch_shape
#     V_prime_ALL_shape_final_batch = Y_prime_ALL_batch_shape(final_batch_size), Q_l_batch_shape
#     V_prime_ALL_shapes_by_batch_size = {main_batch_size:V_prime_ALL_shape_main_batch,
#                                         final_batch_size:V_prime_ALL_shape_main_batch}
# #     V_prime_ALL_shapes_by_batch = tuple(map(V_prime_ALL_shape_for_batch, batch_sizes))
#     V_prime_contract_expr_by_batch_size = {main_batch_size:oe.contract_expression(V_prime_index_str, *(V_prime_ALL_shapes_by_batch_size[main_batch_size])),
#                                            final_batch_size:oe.contract_expression(V_prime_index_str, *(V_prime_ALL_shapes_by_batch_size[final_batch_size]))}
# #     V_prime_contract_exprs_by_batch = tuple(map(lambda shapes: oe.contract_expression(V_prime_index_str, *shapes),
# #                                                 V_prime_ALL_shapes_by_batch))    
#     N_prime_index_str = 'cmn,nc->cm'
#     M_prime_ALL_batch_shape = lambda contexts_per_batch: (contexts_per_batch, m, n) # (c/b, m, n)
#     pW_context_batch_shape = lambda contexts_per_batch: (n, contexts_per_batch) # (n, c/b)
#     N_prime_shape_for_batch = lambda contexts_per_batch: (M_prime_ALL_batch_shape(contexts_per_batch), pW_context_batch_shape(contexts_per_batch))
#     N_prime_shapes_by_batch_size = {main_batch_size:N_prime_shape_for_batch(main_batch_size),
#                                     final_batch_size:N_prime_shape_for_batch(final_batch_size)}
# #     N_prime_shapes_by_batch = tuple(map(N_prime_shape_for_batch, batch_sizes))
#     N_prime_contract_expr_by_batch_size = {main_batch_size:oe.contract_expression(N_prime_index_str, *(N_prime_shapes_by_batch_size[main_batch_size])),
#                                            final_batch_size:oe.contract_expression(N_prime_index_str, *(N_prime_shapes_by_batch_size[final_batch_size]))}
# #     N_prime_contract_exprs_by_batch = tuple(map(lambda shapes: oe.contract_expression(N_prime_index_str, *shapes),
# #                                                 N_prime_shapes_by_batch))
    
#     with cupy.cuda.Device(0):
#         Q_l_g = cupy.array(Q_l.numpy())
#     Y_prime_C_split_batched = tuple(map(lambda batch: batch.numpy().astype(np.float32),
#                                         Y_prime_C_split_batched))
# #     cupy.array(Y_prime_C_split_batched.numpy().astype(np.float32))
# #     Y_prime_C_split_batched = cupy.array(Y_prime_C_split_batched.numpy().astype(np.float32))
#     Y_prime_C_split_batched_indices = tuple(map(lambda index_batch: cupy.array(index_batch.numpy()),
#                                                Y_prime_C_split_batched_indices))
    
    if profilingStamps:
        stampedMemNote('Processing batches...')
    #process the batches
    if parallel:
#         print('Parallelizing across CPUs cores...')
        Z_prime_C = torch.cat(list( par(delayed(denominator_batch_process)(Y_prime_C_split_batched_batch, Q_l, indices, exact_wf=exact_wf)
                                        for Y_prime_C_split_batched_batch, indices in zip(Y_prime_C_split_batched,
                                                                                          Y_prime_C_split_batched_indices)) ))

#         Z_prime_C = torch.cat(list( par(delayed(denominator_batch_process_oe)(Y_prime_C_split_batched_batch, Q_l, indices, 
#                                                                               V_prime_contract_expr_by_batch_size[len(indices)], 
#                                                                               N_prime_contract_expr_by_batch_size[len(indices)])
#                                         for Y_prime_C_split_batched_batch, indices in zip(Y_prime_C_split_batched,
#                                                                                           Y_prime_C_split_batched_indices)) ))
#         Z_prime_C = torch.cat(list( par(delayed(denominator_batch_process_oe)(Y_prime_C_split_batched_batch, Q_l, indices, V_ce, N_ce)
#                                         for Y_prime_C_split_batched_batch, indices, V_ce, N_ce in zip(Y_prime_C_split_batched,
#                                                                                                       Y_prime_C_split_batched_indices,
#                                                                                                       V_prime_contract_exprs_by_batch,
#                                                                                                       N_prime_contract_exprs_by_batch)) ))
    elif use_gpu:
#         print('Using the GPU...')
#         Z_prime_C = torch.cat([torch.tensor(cupy.asnumpy(denominator_batch_process_c(Y_prime_C_split_batched_batch, Q_l_g, indices)))
#                        for Y_prime_C_split_batched_batch, indices in tqdm(zip(Y_prime_C_split_batched,
#                                                                               Y_prime_C_split_batched_indices),
#                                                                           total=num_batches,
#                                                                           mininterval=mininterval)]) 
        Z_prime_C = torch.cat([denominator_batch_process_g(Y_prime_C_split_batched_batch.type(my_tt).cuda(), Q_l_g, indices, exact_wf=exact_wf)
                       for Y_prime_C_split_batched_batch, indices in tqdm(zip(Y_prime_C_split_batched,
                                                                              Y_prime_C_split_batched_indices),
                                                                          total=num_batches,
                                                                          mininterval=mininterval,
                                                                          unit_scale=target_contexts_per_batch,
                                                                          unit="calc")]) 
#         Z_prime_C = torch.cat([denominator_batch_process_g(Y_prime_C_split_batched_batch.type(my_tt).cuda(), Q_l_g, indices, 
#                                                             V_prime_contract_expr_by_batch_size[len(indices)], 
#                                                             N_prime_contract_expr_by_batch_size[len(indices)])
#                        for Y_prime_C_split_batched_batch, indices in tqdm(zip(Y_prime_C_split_batched,
#                                                                               Y_prime_C_split_batched_indices),
#                                                                           total=num_batches)]) 
    else:
#         print('Serially on the CPU...')
        Z_prime_C = torch.cat([denominator_batch_process(Y_prime_C_split_batched_batch, Q_l, indices, exact_wf=exact_wf)
                               for Y_prime_C_split_batched_batch, indices in tqdm(zip(Y_prime_C_split_batched,
                                                                                      Y_prime_C_split_batched_indices),
                                                                                  total=num_batches,
                                                                                  mininterval=mininterval,
                                                                                  unit_scale=target_contexts_per_batch,
                                                                                  unit="calc")]) 

#         Z_prime_C = torch.cat([denominator_batch_process_oe(Y_prime_C_split_batched_batch, Q_l, indices, 
#                                                             V_prime_contract_expr_by_batch_size[len(indices)], 
#                                                             N_prime_contract_expr_by_batch_size[len(indices)])
#                                for Y_prime_C_split_batched_batch, indices in tqdm(zip(Y_prime_C_split_batched,
#                                                                                       Y_prime_C_split_batched_indices),
#                                                                                   total=num_batches)]) 
#         Z_prime_C = torch.cat([denominator_batch_process_oe(Y_prime_C_split_batched_batch, Q_l, indices, V_ce, N_ce)
#                                for Y_prime_C_split_batched_batch, indices, V_ce, N_ce in tqdm(zip(Y_prime_C_split_batched,
#                                                                                                   Y_prime_C_split_batched_indices,
#                                                                                                   V_prime_contract_exprs_by_batch,
#                                                                                                   N_prime_contract_exprs_by_batch),
#                                                                                                  total=num_batches)]) 
    if profilingStamps:
        stampedMemNote('> Batches processed.')
    if use_gpu:
        torch.cuda.empty_cache()
    
    if profilingStamps:
        stampedMemNote('Calculating numerator...')
        stampedNote('Calculating V_prime_w_C...')
    if not exact_wf:
        xhat0f_L_w = Q_l[xhat0f_idx]#.float() # :: (s, l)
    else:
        xhat0f_L_w = Q_l[xhat0f_l_idx]#.float() # :: (s, l)
#     V_prime_w_C = oe.contract('cmij,ji->cmi',Y_prime_C_split.type(my_tt), xhat0f_L_w, backend='torch') # :: (c,m,l)
    V_prime_w_C = torch.einsum('cmij,ji->cmi',Y_prime_C_split.type(my_tt), xhat0f_L_w) # :: (c,m,l)
#     stampedMemNote('V_prime_w_C created.')
#     del xhat0f_L_w
#     del Y_prime_C_split
#     stampedMemNote('xhat0f_L_w and Y_prime_C_split deleted.')
    if profilingStamps:
        stampedNote('Calculating O_w_C...')
    O_w_C = torch.prod(V_prime_w_C, 2) # :: (c,m) likelihoods of each of the m sampled channel prefixes for each of c contexts
#     stampedMemNote('O_w_C created.')
#     del V_prime_w_C
#     stampedMemNote('V_prime_w_C deleted.')
    if profilingStamps:
        stampedNote('Calculating U_w_C...')
    if not exact_wf:
        U_w_C = torch.einsum('c,cm->cm', pW_C_torch[xhat0f_idx], O_w_C) ## :: (c,m) joint probabilities of xhat0f with each of the m sampled channel prefixes for each of the c contexts
    else:
        U_w_C = torch.einsum('c,cm->cm', pW_C_torch[xhat0f_l_idx], O_w_C) ## :: (c,m) joint probabilities of xhat0f with each of the m sampled channel prefixes for each of the c contexts
    
#     stampedMemNote('U_w_C created.')
#     del O_w_C
#     stampedMemNote('O_w_C deleted.')
    
    if profilingStamps:
        stampedMemNote('Calculating E_w_C...')
#     return torch.dot(Z_prime_C, U_w_C) / m
    E_w_C = torch.einsum('cm,cm->c', Z_prime_C, U_w_C) / m
#     stampedMemNote('E_w_C created.')
#     del Z_prime_C
#     del U_w_C
#     stampedMemNote('Z_prime_C and U_w_C deleted.')
    if profilingStamps:
        stampedNote('> End calculation.')
    return E_w_C

In [272]:
random_long_word = choice(list({w for w in Ws if len(ds2t(w)) == max(wordlengthsInclEdges)}))
random_long_word
rlw_idx = Ws_t.index(random_long_word)

'⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉'

In [273]:
len(ds2t(random_long_word))
rlw_slice = pXhat0fX0i_ptc(rlw_idx, CMsByLengthByWordformIndex_torch[len(ds2t(random_long_word))][rlw_idx],
                           m=n, target_contexts_per_batch=50, parallel=l, use_gpu=g)
rlw_slice.dtype

19

> Begin calculation. @ 22:07:53
l = 17
Sampling... @ 22:07:53
	VM used vs. available: 8.06GB vs. 25.16GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    5.4s remaining:   12.6s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    6.5s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    7.5s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.1s finished


Splitting samples by context... @ 22:08:04
	VM used vs. available: 11.65GB vs. 21.57GB


  0%|          | 0/2126 [00:00<?, ?it/s]

...

 94%|█████████▍| 1995/2126 [01:40<00:06, 19.94it/s]

Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...


100%|██████████| 2126/2126 [01:46<00:00, 19.91it/s]


> Batches processed. @ 22:10:01
	VM used vs. available: 17.05GB vs. 16.16GB
Calculating numerator... @ 22:10:01
	VM used vs. available: 17.05GB vs. 16.16GB
Calculating V_prime_w_C... @ 22:10:01
Calculating O_w_C... @ 22:10:10
Calculating U_w_C... @ 22:10:10
Calculating E_w_C... @ 22:10:10
	VM used vs. available: 17.46GB vs. 15.75GB
> End calculation. @ 22:10:10


torch.float32

In [274]:
len(ds2t(Ws_t[0]))
idealize_slice2 = pXhat0fX0i_ptc(0, CMsByLengthByWordformIndex_torch[len(ds2t(Ws_t[0]))][0],
                                m=n, target_contexts_per_batch=150, parallel=l, use_gpu=g)
idealize_slice2.dtype

9

> Begin calculation. @ 22:10:10
l = 7
Sampling... @ 22:10:10
	VM used vs. available: 10.05GB vs. 23.16GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    3.5s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    3.9s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    4.4s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.0s finished


Splitting samples by context... @ 22:10:16
	VM used vs. available: 11.52GB vs. 21.69GB


  0%|          | 0/709 [00:00<?, ?it/s]

Splitting contexts into batches ... @ 22:10:17
Num batches = 709
Processing batches... @ 22:10:17
	VM used vs. available: 12.97GB vs. 20.24GB


100%|██████████| 709/709 [00:47<00:00, 15.09it/s]


> Batches processed. @ 22:11:04
	VM used vs. available: 12.97GB vs. 20.24GB
Calculating numerator... @ 22:11:04
	VM used vs. available: 12.97GB vs. 20.24GB
Calculating V_prime_w_C... @ 22:11:04
Calculating O_w_C... @ 22:11:08
Calculating U_w_C... @ 22:11:08
Calculating E_w_C... @ 22:11:08
	VM used vs. available: 13.15GB vs. 20.06GB
> End calculation. @ 22:11:08


torch.float32

In [275]:
(idealize_slice2 == float('inf')).any()
max(idealize_slice2)
min(idealize_slice2)

tensor(0, dtype=torch.uint8)

tensor(0.9651)

tensor(9.8052e-06)

In [276]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        9.4G         20G         14M        1.7G         21G
Swap:          2.0G        1.3G        723M


In [277]:
len(Cs_t)
len(Cs_t) * 50

106295

5314750

In [278]:
if benchmark:
    wlr_n = np.array(sorted(list(wordlengthsInclEdges)))
    benchmark_lengths = torch.tensor(list(wlr_n[ np.where(wlr_n % 4 == 0)[0]  ]) + [max(wordlengthsInclEdges)])
    benchmark_wordform_idxs = tuple([Ws_t.index(choice(list(wordformsOfLength(l, Ws_t, True))))
                                     for l in benchmark_lengths])
    benchmark_wordforms = tuple([Ws_t[w_idx] for w_idx in benchmark_wordform_idxs])
    dict(zip(list(map(lambda t: t.item(), benchmark_lengths)), 
             benchmark_wordforms))
    benchmark_CMs = tuple([CMsByLengthByWordformIndex_torch[len(ds2t(w))][Ws_t.index(w)]
                           for w in benchmark_wordforms])

In [279]:
if benchmark:
    # iterate through batch sizes, leaving large batch sizes for last, 
    # since they're most likely to cause crashes and this way we collect more useful data first...
    for batch_size in [50, 100, 200, 300, 500, 600, 750, 1000, 1250, 1500]:
        print(f"### Batch size: {batch_size}")
        for b_l, w_idx, CM in zip(benchmark_lengths, benchmark_wordform_idxs, benchmark_CMs):
            stampedMemNote(f'## L={b_l}, b={batch_size}', includeGPU=True)
            post = pXhat0fX0i_ptc(w_idx, CM, m=n, target_contexts_per_batch=batch_size, parallel=l, use_gpu=g, exact_wf=False)
            print('')
        print('')

In [280]:
if benchmark:
    # batch size 50, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 40.2s = 106295c/40.2 = 2644.15cps
    # peak memory usage is about 1.0GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=50, parallel=False, use_gpu=True)

In [281]:
if benchmark:
    # batch size 200, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 32.1s = 106295c/32.1 = 3311.37cps
    # peak GPU memory usage ~2.0 GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=200, parallel=False, use_gpu=True)

In [282]:
if benchmark:
    # batch size 350, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 28.7s = 106295c/28.7s = 3703.66cps
    # peak GPU memory usage ~3.1 GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=350, parallel=False, use_gpu=True)

In [283]:
if benchmark:
    # batch size 375, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 28.6s = 106295c/28.6 = 3716.61cps
    # peak GPU memory usage ~3.3 GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=375, parallel=False, use_gpu=True)

In [284]:
if benchmark:
    # batch size 375, kotoba, gpu, float32, opt_einsum
    # 5s on sample generation, 1s to split into batches of contexts
    # 103s = 106295c/103s = 1032cps
    # peak GPU memory usage ~? GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=375, parallel=False, use_gpu=True)

In [285]:
if benchmark:
    # batch size 400, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 94s = 106295c/94s = 1130.80cps
    # peak GPU memory usage ~3.1 GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=400, parallel=False, use_gpu=True)

In [286]:
if benchmark:
    # batch size 500, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 93.0s = 106295c/93.0s = 1142.96cps
    # peak GPU memory usage ~4.2 GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=500, parallel=False, use_gpu=True)

In [287]:
if benchmark:
    # batch size 750, kotoba, gpu, float32
    # 5s on sample generation, 1s to split into batches of contexts
    # 252s = 106295c/252s = 421.81cps
    # peak GPU memory usage ~5.9 GB / 8GB
    pContact_cont_C = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=750, parallel=False, use_gpu=True)

In [288]:
if benchmark:
    #batch size 500, quine, serial
    # 40s on sample generation, 1s to split into batches of contexts / float64 / quine
    # 6s on sample generation, <1s to split into batches of contexts / float32 / kotoba
    # ≈10s per batch, 213 batches = 35m4s = 106295c/2104s = 50.52 cps / float64 / quine
    # ≈5s per batch, 213 batches = 17m47s = 106295c/1067s = 99.62 cps / float32 / quine
    # 1.16s per batch, 213 batches = 247.08s = 4m7s = 106295c/247s = 430.34 cps / float32 / kotoba
    #  - pytorch does make use of all available cpus (for a significant stretch) 
    #    of each batch step, and the memory overhead is low enough to run on kotoba,
    #    but it's clearly possible (serial processing w/ batch size 3000 only takes 90s for all 106295c)
    #      - parallelization over (smaller? larger?) batches 
    #    would waste less time
    cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 500)

In [289]:
if benchmark:
    #batch size 500, quine, parallel
    # 30s on sample generation, 1s to split into batches of contexts
    # 13m39s = 819s = 106295c/819s = 129.79 cps / float64 / quine
    # 4-5s per batch, 213 batches ≈10^3s to do ≈10^5c = ≈10^2 cps / float64 / quine
    #   = about twice as fast as serial processing with same batch size
    # cpu utilization is more consistent per core but far from full utilization
    # 11m46s = 706s = 106295c/706s = 150.56 cps / float32 / quine
    # 3m59s = 239s = 106295c/239s = 444.75 cps / float32 / kotoba / peak 25GB RAM usage
    cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 500, parallel=True)

In [290]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        9.4G         20G         14M        1.7G         21G
Swap:          2.0G        1.3G        723M


In [291]:
if benchmark:
    #batch size 650, quine, parallel
    # 24s on sample generation, 1s to split into batches of contexts / float64 / quine
    # 12m21s = 741s = 106295c/741s = 143.45 cps / float64 / quine
    # 9m14s = 554s = 106295c/554s = 191.87 cps / float32 / quine
    # ?m?s = ?s = 106295c/? = ? cps / float32 / kotoba / peak ?GB RAM usage = too high...
    cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 650, parallel=True)

In [288]:
#batch size 750, quine, serial, oe test (reused paths, no storage of constant Q_l, efficient reused path expr calc)
cont_Z_prime_ALL_p = pXhat0fX0i_ptc(contact_idx, cont_CM, m=50, target_contexts_per_batch=750, parallel=False)

> Begin calculation. @ 19:24:47
Sampling... @ 19:24:47
	VM used vs. available: 19.74GB vs. 144.22GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    6.5s remaining:   15.1s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    7.6s remaining:    7.6s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.0s remaining:    3.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.3s finished


Splitting samples by context... @ 19:25:05
	VM used vs. available: 19.77GB vs. 144.20GB



  0%|          | 0/142 [00:00<?, ?it/s]

Splitting contexts into batches ... @ 19:25:06
Num batches = 142
Pre-calculating contract expressions for each batch... @ 19:25:06
	VM used vs. available: 20.39GB vs. 143.58GB
Processing batches... @ 19:25:06
	VM used vs. available: 20.39GB vs. 143.58GB



  6%|▋         | 9/142 [11:36<2:55:52, 79.34s/it]

KeyboardInterrupt: 

In [435]:
#batch size 750, quine, parallel
# 19s on sample generation, 1s to split into batches of contexts
# 10m37s = 637s = 106295c/637s = 166.87 cps / float64
# 8m38s = 478s = 106295c/?s = 222.37 cps / float32
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 750, parallel=True)

> Begin calculation. @ 09:35:39
	VM used vs. available: 21.11GB vs. 142.86GB
Generating samples... @ 09:35:39


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    7.0s remaining:   16.2s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    8.0s remaining:    8.0s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    8.6s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.2s finished


Splitting samples by context... @ 09:36:03
Splitting contexts into batches ... @ 09:36:04
Num batches = 142
Processing batches... @ 09:36:04
	VM used vs. available: 21.11GB vs. 142.86GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 110 out of 142 | elapsed:  6.4min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done 125 out of 142 | elapsed:  7.2min remaining:   58.6s
[Parallel(n_jobs=-1)]: Done 140 out of 142 | elapsed:  8.0min remaining:    6.9s
[Parallel(n_jobs=-1)]: Done 142 out of 142 | elapsed:  8.1min finished


> Batches processed. @ 09:44:17
	VM used vs. available: 21.13GB vs. 142.85GB


In [436]:
#batch size 850, quine, parallel
# 21s on sample generation, 1s to split into batches of contexts
# 10m46s = 646s = 106295c/646s = 164.54 cps / float64
# 8m1s = 481s = 106295c/481s = 220.99 cps / float32
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 850, parallel=True)

> Begin calculation. @ 09:44:17
	VM used vs. available: 21.11GB vs. 142.86GB
Generating samples... @ 09:44:17


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    8.1s remaining:   18.9s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    8.8s remaining:    8.8s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   10.0s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.2s finished


Splitting samples by context... @ 09:44:41
Splitting contexts into batches ... @ 09:44:42
Num batches = 126
Processing batches... @ 09:44:42
	VM used vs. available: 21.11GB vs. 142.86GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  92 out of 126 | elapsed:  5.6min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done 105 out of 126 | elapsed:  6.4min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done 118 out of 126 | elapsed:  7.0min remaining:   28.6s
[Parallel(n_jobs=-1)]: Done 126 out of 126 | elapsed:  7.4min finished


> Batches processed. @ 09:52:17
	VM used vs. available: 21.11GB vs. 142.86GB


In [292]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        9.4G         20G         14M        1.7G         21G
Swap:          2.0G        1.3G        723M


In [ ]:
# batch size 1000, quine, parallel
# 30s sample generation
# peak memory usage is too high
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 1000, parallel=True)

In [25]:
#batch size 100, quine, parallel
# ?s on sample generation, 1s to split into batches of contexts
# ?m?s = ?s = 106295c/?s = ?.? cps
# ?s per batch, ? batches ≈?s to do ≈10^5c = ≈? cps
#   = about ? as fast as ? processing with batch size ?
# cpu utilization is ? per core but ? ? full utilization
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 100, parallel=True)

NameError: name 'denominator_calc' is not defined

In [441]:
#batch size 1000, quine, parallel
# 30s sample generation
# peak memory usage is too high
# cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 1000, parallel=True)

Generating samples... @ 21:16:00


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    8.1s remaining:   18.9s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    9.2s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.8s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   18.8s finished


Splitting samples by context... @ 21:16:30
Splitting contexts into batches ... @ 21:16:31
Num batches = 107
Processing batches... @ 21:16:31


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.7min
Process ForkPoolWorker-591:
Process ForkPoolWorker-595:
Process ForkPoolWorker-590:
Process ForkPoolWorker-584:
Process ForkPoolWorker-580:
Process ForkPoolWorker-585:
Process ForkPoolWorker-583:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.r

  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/site-packages/joblib/pool.py", line 149, in get
    return recv()
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/multiprocessing/connection.py", line 251, in recv
    return _ForkingPickler.loads(buf.getbuffer())
  File "/home/AD/emeinhar/anaconda3/envs/anvil/lib/python3.6/site-packages/torch/storage.py", line 125, in _load_from_bytes
    def _load_from_bytes(b):
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
#batch size 50, quine, parallel
# 30s sample generation
# 
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 50, parallel=True)

In [444]:
#batch size 2000, quine, serial
# 30s sample generation
# 34s/batch -> 54 batches = 1836s = 106295c/1836s = 57.90 cps
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 2000, parallel=False)

Generating samples... @ 21:23:08


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    6.7s remaining:   15.6s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    7.4s remaining:    7.4s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    8.2s remaining:    3.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.6s finished


Splitting samples by context... @ 21:23:32


  0%|          | 0/54 [00:00<?, ?it/s]

Splitting contexts into batches ... @ 21:23:34
Num batches = 54
Processing batches... @ 21:23:34


 19%|█▊        | 10/54 [05:37<24:36, 33.56s/it]

KeyboardInterrupt: 

In [ ]:
#batch size 5000, quine, serial
# 17s sample generation (?????)
# mem usage beforehand ≈16GB
# each batch initially requires ≈+70GB 
# 53m30s = 3810s -> 106295c/3810s = 27.90 cps
cont_Z_prime_ALL_p = denominator_calc(cont_CM, 50, cont_Q_l, 5000, parallel=False)

## Exact/full wordform-length variant functions

These variant versions of the functions from section 5.2 are used for Case 3 calculations.

In [293]:
def w_l_idx_to_w_idx(l, w_l_idx):
    w = Ws_l_t[l][w_l_idx]
    w_idx = Ws_t.index(w)
    return w_idx

l_to_w_idxs = {l:torch.tensor([w_l_idx_to_w_idx(l, w_l_idx)
                               for w_l_idx in torch.arange(len(Ws_l_t[l]))]).type(torch.long)
               for l in wordlengthsInclEdges}

In [294]:
l_to_w_idxs[4]

tensor([  35, 2283, 4298, 7418, 7724])

In [295]:
pW_C_torch.shape

torch.Size([9172, 106295])

In [296]:
pW_C_torch[l_to_w_idxs[4],0]

tensor([7.5595e-02, 1.9239e-06, 3.3584e-06, 5.0495e-08, 1.6083e-02])

In [297]:
def get_pW_t_l(l, c_idx):
    return pW_C_torch[l_to_w_idxs[l], c_idx]

In [298]:
def w_idx_to_l_w_l_idx(w_idx):
    w = Ws_t[w_idx]
    w_t = ds2t(w)
    l = len(w_t)
    w_l_idx = Ws_l_t[l].index(w)
    return (l, w_l_idx)

w_l_idx_to_w_idx(4, 0)
w_idx_to_l_w_l_idx(35)

35

(4, 0)

In [299]:
def pXhat0fX0f_pxt(xhat0f_idx, x0f_CM, c_idx, m = 50):
    '''
    If ⋊ x_0 x_1 ... x_i x_k denotes
    the produced prefix r of the speaker's intended wordform w,
     where
       x_i = the last fully produced segment
       k = i+1
       x_k = the next segment
       |⋊ x_0 x_1 ... x_i x_k| = l + 2
       l = i+1 = the length of the produced prefix (not incl. 
                 the left word edge or the next segment)
    and this function concerns the special case where always
    i = f wrt the source sequence.
    
    then

    x0f_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
     where x0f_CM[:,j] is the distribution p(Y_j|X0^f has been produced)

    xhat0f_idx is the index of a wordform w' s.t. 
        pXhat0fX0f_pxt(xhat0f_idx, x0f_CM) ≈ p(w'|w)
    '''
    shape_info = False
#     pW = get_pW_t_l(l, c_idx)
#     if x0k_CM is None and x0k is None:
#         raise Exception('Must specify one of x0k_CM or x0k.')

    # Computation proceeds in two steps:
    # calculate the denominator p(y0i) for m = 50 sampled y0i's
    # calculate the numerator p(y0i|w')p(w') for m = 50 sampled y0i's

    if xhat0f_idx not in l_to_w_idxs[x0f_CM.shape[1]+2]:
        return torch.tensor([0.]) # <<<< this is one of the non-cosmetic (variable-renaming) differences from the main version of this function

    L, xhat0f_l_idx = w_idx_to_l_w_l_idx(xhat0f_idx) # L = l+2

#     xhat0f_idx = Wmap[xhat0f]

#     l = len(ds2t(x0i))
#     x0k_CM = CMsByPrefixIndex[prefixMap[x0i]]

#     my_Q_l = CMsByLengthByWordformIndex[l - 2]

    #Collect m samples from p(Y0i|X0k) = p(Y0i|r), 
    # where each sample is an l-length stack of one-hot vectors
    # where each one-hot vector corresponds to a channel symbol:
    Y_prime = depthSamplerC_t(x0f_CM, m)#.float() #:: (m, l, |Y1|)
#         if x0k_CM is not None:
#             Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#             if x0k is not None:
# #                 print('foo')
#                 print(type(x0k_CM))
#                 print(x0k_CM.shape)
#                 generated_CM = sample_pY0iX0k(x0k, debug=True)[1]
#                 print(type(generated_CM))
#                 print(generated_CM.shape)
#                 generated_CM_t = torch.tensor(generated_CM)
#                 assert torch.allclose(x0k_CM, torch.tensor(sample_pY0iX0k(x0k, debug=True)[1]))
# #                 print('bar')
#         else:
#             Y_prime = torch.tensor(list(map(y0kOHmap, [sample_pY0iX0k(x0k) for each in range(m)])))
#         print('baz')
    l = Y_prime.shape[1]# + 1
    if shape_info:
        print(f'|Y_prime| = {Y_prime.shape} = (m, l, |Y1|)')
        
    pW = get_pW_t_l(L, c_idx) # (|W_l| = n_L) # <<<< this is one of the non-cosmetic (variable-renaming) differences from the main version of this function
    
    #Grab ALL the relevant-prefix-length channel matrices 
    # for every segmental wordform in the lexicon:
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 3]#.float()
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 1]#.float() #:: (|W_l|, |Y1|, l)
    my_Q_l = exactCMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W_l|, |Y1|, l) # <<<< this is one of the non-cosmetic (variable-renaming) differences from the main version of this function

    if shape_info:
        print(f'|Q_l| = {my_Q_l.shape} = (|W_l| = n_L, |Y1|, l)')

#         print(f'{Y_prime.dtype}, {my_Q_l.dtype}')

    # NORMALIZATION
    V_prime = torch.einsum('mli,kil->mkl', Y_prime.type(my_tt), my_Q_l)  # :: (m,n_L,l)
#         print(f"|V'| = {V_prime.shape} = (m, n_L, l)")
    M_prime = torch.prod(V_prime, 2) # :: (m,n_L)
#         print(f"|M'| = {M_prime.shape} = (m, n_l)")
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, ) <- prior probabilities of each of the m sampled channel prefixes
    N_prime = torch.matmul(M_prime, pW)
#         print(f"|N'| = {N_prime.shape} = (m, )")
#     Z_prime = 1.0 / N_prime # :: (m, )
    Z_prime = torch.ones(N_prime.shape, dtype=my_tt)
    torch.div(Z_prime,
              N_prime,
              out=Z_prime)
#         print(f"|Z'| = {Z_prime.shape} = (m, )")
    if shape_info:
        print(f"|V'| = {V_prime.shape} = (m, n_L, l)")
        print(f"|M'| = {M_prime.shape} = (m, n_L)")
        print(f"|N'| = {N_prime.shape} = (m, )")
        print(f"|Z'| = {Z_prime.shape} = (m, )")

    # NUMERATOR
    L_w = my_Q_l[xhat0f_l_idx]#.float()
#     L_w = my_Q_l[xhat0f_idx]#.float()
    V_prime_w = torch.einsum('mij,ji->mi',Y_prime.type(my_tt), L_w)
#         print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
    O_w = torch.prod(V_prime_w, 1) # :: (m,) likelihoods of each of the m sampled channel prefixes
#         print(f"|O_w| = {O_w.shape} = (m, )")
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,) joint probabilities of xhat0f with each of the m sampled channel prefixes
    U_w = pW[xhat0f_l_idx] * O_w ## :: (m,) joint probabilities of xhat0f with each of the m sampled channel prefixes
#     U_w = pW[xhat0f_idx] * O_w ## :: (m,) joint probabilities of xhat0f with each of the m sampled channel prefixes
#         print(f"|U_w| = {U_w.shape} = (m, )")
    if shape_info:
        print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
        print(f"|O_w| = {O_w.shape} = (m, )")
        print(f"|U_w| = {U_w.shape} = (m, )")


    if shape_info:
        E = torch.dot(Z_prime, U_w) / m
        print(f"|E| = scalar: type(E) = {type(E)}")
    return torch.dot(Z_prime, U_w) / m

def pXhat0fX0f_pxtn(xhat0f_idx, x0f_CM, c_idx, m = 50):
    return pXhat0fX0f_pxt(xhat0f_idx, x0f_CM, c_idx, m = 50).numpy()

def wordformTox0fCM(w=None, w_idx=None):
#     if w is None and w_idx is None and w_l_idx is None:
#         raise Exception('At least one argument must be specified.')
    if w is not None:
        w_t = ds2t(w)
        w_idx = Ws_t.index(w)
    else:
        w = Ws_t[w_idx]
        w_t = ds2t(w)
    
    total_x0f_length = len(w_t) #includes both edge symbols
#     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
#     word_length_noRE = word_length_noLE - 1
#     offset = 2
#     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
#     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
#     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
#     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
    w_l_idx = Ws_l_t[total_x0f_length].index(w)
    x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0f_length][w_l_idx]
    return x0f_cm

# for convenience
def pXhat0fX0f_pxtn_conv(w_hat, w, c, m = 50):
    w_idx = Ws_t.index(w_hat)

#     r_idx = Ps_t.index(r)
    source_wf = list(wordsWithPrefix(w, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    total_x0f_length = len(ds2t(w))# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    w_l_idx = Ws_l_t[total_x0f_length].index(w)
    x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0f_length][w_l_idx]
    
    c_idx = Cs_t.index(c)
    
    return pXhat0fX0f_pxtn(w_idx, x0f_cm, c_idx, m=m)

In [300]:
if benchmark:
    phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
                           '⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
                           'a couple of')

In [301]:
if benchmark:
    pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
                   CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
                   Cs_t.index('a couple of'),
                   m = 200)

In [302]:
if benchmark:
    pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
                   CMsByLengthByWordformIndex_torch[9][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
                   Cs_t.index('a couple of'),
                   m = 50000)

In [303]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        9.4G         20G         14M        1.7G         21G
Swap:          2.0G        1.3G        723M


In [304]:
if benchmark:
    pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
                   CMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
                   Cs_t.index('a couple of'),
                   m = 50000)

In [305]:
wordlengthsInclEdges
len(wordlengthsInclEdges)
len([None for each in range(min(wordlengthsInclEdges))])
len(exactCMsByLengthByWordformIndex_torch)

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}

15

4

20

In [306]:
exactCMsByLengthByWordformIndex_torch[10].shape
exactCMsByLengthByWordformIndex_torch[10][Ws_l_t[10].index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')].shape
exactCMsByLengthByWordformIndex_torch[10][Ws_l_t[10].index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')]

torch.Size([1052, 39, 8])

torch.Size([39, 8])

tensor([[0.0439, 0.0181, 0.0091, 0.0090, 0.2037, 0.0093, 0.0111, 0.0000],
        [0.0070, 0.0137, 0.0091, 0.0090, 0.0075, 0.0093, 0.0111, 0.0000],
        [0.0255, 0.0072, 0.0091, 0.0090, 0.0188, 0.0093, 0.0111, 0.0000],
        [0.0070, 0.0071, 0.0090, 0.0171, 0.0074, 0.0092, 0.0108, 0.0000],
        [0.0068, 0.0069, 0.0170, 0.0087, 0.0072, 0.0090, 0.0105, 0.0000],
        [0.0070, 0.0072, 0.0091, 0.0090, 0.0710, 0.0351, 0.0111, 0.0000],
        [0.0068, 0.0070, 0.0089, 0.0088, 0.0073, 0.0091, 0.0108, 0.0000],
        [0.0070, 0.0072, 0.0091, 0.0090, 0.0075, 0.0178, 0.0111, 0.0000],
        [0.0068, 0.0071, 0.0090, 0.0089, 0.0074, 0.0176, 0.0111, 0.0000],
        [0.0070, 0.0072, 0.0091, 0.0090, 0.0075, 0.0093, 0.0111, 0.0000],
        [0.0052, 0.0062, 0.0078, 0.0077, 0.0064, 0.0080, 0.0111, 0.0000],
        [0.6525, 0.0071, 0.0090, 0.0089, 0.0074, 0.5897, 0.0403, 0.0000],
        [0.0255, 0.0072, 0.0091, 0.0090, 0.0075, 0.0093, 0.0111, 0.0000],
        [0.0070, 0.0071, 0.0090, 0.008

In [307]:
len(ds2t('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')) - 2

8

In [308]:
if benchmark:
    %timeit pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), exactCMsByLengthByWordformIndex_torch[10][Ws_l_t[10].index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], Cs_t.index('a couple of'), m = 50)

In [309]:
if benchmark:
    %timeit pXhat0fX0f_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), exactCMsByLengthByWordformIndex_torch[10][Ws_l_t[10].index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], Cs_t.index('a couple of'), m = 50)

In [310]:
if benchmark:
    len(ds2t('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')) - 2
    pXhat0fX0f_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
                   exactCMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
                   Cs_t.index('a couple of'),
                   m = 50000)

In [311]:
torch.equal(exactCMsByLengthByWordformIndex_torch[10][Ws_l_t[10].index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], 
            CMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')])

True

In [312]:
n_test_cases = 1000
random_wordform_idxs = choices(list(range(len(Ws_t))), k=n_test_cases)
random_context_idxs = choices(list(range(len(Cs_t))), k=n_test_cases)

In [313]:
%%timeit -n 100 -r 10

rand_w_idx = choice(random_wordform_idxs)
rand_c_idx = choice(random_context_idxs)

# 10.5ms on kotoba
pXhat0fX0i_pxt(rand_w_idx, wordformTox0fCM(w_idx=rand_w_idx), rand_c_idx, m = 50)

10.1 ms ± 765 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [314]:
%%timeit -n 100 -r 10

rand_w_idx = choice(random_wordform_idxs)
rand_c_idx = choice(random_context_idxs)

# ≈130-150ms on new sidious (pre-sampling+Z_prime change)
# ≈132ms on quine
# prev 10.9ms on kotoba
# 2.75ms on kotoba
pXhat0fX0f_pxt(rand_w_idx, wordformTox0fCM(w_idx=rand_w_idx), rand_c_idx, m = 50)

2.83 ms ± 119 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [315]:
%%timeit -n 100 -r 10

rand_w_idx = choice(random_wordform_idxs)
rand_c_idx = choice(random_context_idxs)

# ≈130-150ms on new sidious (pre-sampling+Z_prime change)
# ≈132ms on quine
# prev 10.9ms on kotoba
# 2.65ms on kotoba
pXhat0fX0f_pxt(rand_w_idx, wordformTox0fCM(w_idx=rand_w_idx), rand_c_idx, m = 50)

2.92 ms ± 104 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [316]:
del random_wordform_idxs
del random_context_idxs

In [323]:
exactLengthFreqs = {l:len({w for w in Ws_t if len(ds2t(w)) == l})
                    for l in wordlengthsInclEdges}
exactLengthFreqs
mostFreq_length_freq = max(exactLengthFreqs.values())
mostFreq_length = [l for l in wordlengthsInclEdges if exactLengthFreqs[l] == mostFreq_length_freq]
mostFreq_length
mostFreq_length = mostFreq_length[0]

{4: 5,
 5: 173,
 6: 1266,
 7: 1790,
 8: 1646,
 9: 1328,
 10: 1052,
 11: 828,
 12: 508,
 13: 329,
 14: 150,
 15: 66,
 16: 24,
 17: 6,
 19: 1}

[7]

In [318]:
# there are infinities even when n_l is exactly 1 and batch sizes are fairly small...
len(ds2t(random_long_word))
rlw_slice_x = pXhat0fX0i_ptc(rlw_idx, CMsByLengthByWordformIndex_torch[len(ds2t(random_long_word))][rlw_idx],
                             m=n, target_contexts_per_batch=350, parallel=l, use_gpu=g, exact_wf=True)
rlw_slice_x.dtype

19

> Begin calculation. @ 22:11:29
l = 17
Sampling... @ 22:11:29
	VM used vs. available: 10.08GB vs. 23.13GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    5.2s remaining:   12.1s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    6.3s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    7.4s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    9.0s finished


Splitting samples by context... @ 22:11:40
	VM used vs. available: 13.67GB vs. 19.54GB


  0%|          | 0/304 [00:00<?, ?it/s]

Splitting contexts into batches ... @ 22:11:42
Num batches = 304
Processing batches... @ 22:11:42
	VM used vs. available: 17.20GB vs. 16.01GB
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...


100%|██████████| 304/304 [00:10<00:00, 30.32it/s]


> Batches processed. @ 22:11:52
	VM used vs. available: 17.20GB vs. 16.02GB
Calculating numerator... @ 22:11:52
	VM used vs. available: 17.20GB vs. 16.02GB
Calculating V_prime_w_C... @ 22:11:52
Calculating O_w_C... @ 22:12:01
Calculating U_w_C... @ 22:12:01
Calculating E_w_C... @ 22:12:01
	VM used vs. available: 17.56GB vs. 15.65GB
> End calculation. @ 22:12:01


torch.float32

In [319]:
# pick a word of length 7, check if you get infinities at various batch sizes

In [327]:
benchmark = True

In [330]:
if benchmark:
    wlr_n = np.array(sorted(list(wordlengthsInclEdges)))
    benchmark_lengths = torch.tensor(list(wlr_n[ np.where(wlr_n % 4 == 0)[0]  ]) + [sorted(list(wordlengthsInclEdges))[-2]] + [max(wordlengthsInclEdges)])
    benchmark_wordform_idxs = tuple([Ws_t.index(choice(list(wordformsOfLength(l, Ws_t, True))))
                                     for l in benchmark_lengths])
    benchmark_wordforms = tuple([Ws_t[w_idx] for w_idx in benchmark_wordform_idxs])
    dict(zip(list(map(lambda t: t.item(), benchmark_lengths)), 
             benchmark_wordforms))
    benchmark_CMs = tuple([exactCMsByLengthByWordformIndex_torch[len(ds2t(w))][w_idx_to_l_w_l_idx(Ws_t.index(w))[1]]
                           for w in benchmark_wordforms])

{4: '⋊.aɪ.⋉.⋉',
 8: '⋊.n.ɛ.k.s.t.⋉.⋉',
 12: '⋊.æ.k.tʃ.u.æ.l.ɪ.t.i.⋉.⋉',
 16: '⋊.m.ə.t.ɪ.ɹ.i.ə.l.ɪ.s.t.ɪ.k.⋉.⋉',
 19: '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉'}

In [333]:
if benchmark: #RESUME
    # iterate through batch sizes, leaving large batch sizes for last, 
    # since they're most likely to cause crashes and this way we collect more useful data first...
    for batch_size in [50, 100, 200, 300, 500, 600, 750, 1000, 1250, 1500]:
        print(f"### Batch size: {batch_size}")
        for b_l, w_idx, CM in zip(benchmark_lengths, benchmark_wordform_idxs, benchmark_CMs):
            stampedMemNote(f'## L={b_l}, b={batch_size}, |W_L| = {exactLengthFreqs[b_l.item()]}', includeGPU=True)
            post = pXhat0fX0i_ptc(w_idx, CM, m=n, target_contexts_per_batch=batch_size, parallel=l, use_gpu=g, exact_wf=True)
            print('')
        print('')

### Batch size: 50
## L=4, b=50, |W_L| = 5 @ 23:54:08
	VM used vs. available: 8.21GB vs. 25.01GB
> Begin calculation. @ 23:54:08
l = 2
Sampling... @ 23:54:08
	VM used vs. available: 8.21GB vs. 25.01GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    2.5s remaining:    5.8s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    2.6s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    2.8s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.9s finished


Splitting samples by context... @ 23:54:12
	VM used vs. available: 8.63GB vs. 24.59GB


  0%|          | 0/106300 [00:00<?, ?calc/s]

Splitting contexts into batches ... @ 23:54:12
Num batches = 2126
Processing batches... @ 23:54:12
	VM used vs. available: 9.04GB vs. 24.18GB


...

Splitting samples by context... @ 00:36:52
	VM used vs. available: 11.87GB vs. 21.29GB


  0%|          | 0/106500 [00:00<?, ?calc/s]

Splitting contexts into batches ... @ 00:36:54
Num batches = 71
Processing batches... @ 00:36:54
	VM used vs. available: 15.39GB vs. 17.76GB
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...
Z_prime_ALL_batch has infinities...


100%|██████████| 106500/106500 [00:09<00:00, 10713.50calc/s]


> Batches processed. @ 00:37:04
	VM used vs. available: 15.39GB vs. 17.76GB
Calculating numerator... @ 00:37:04
	VM used vs. available: 15.39GB vs. 17.76GB
Calculating V_prime_w_C... @ 00:37:04
Calculating O_w_C... @ 00:37:13
Calculating U_w_C... @ 00:37:13
Calculating E_w_C... @ 00:37:13
	VM used vs. available: 15.75GB vs. 17.40GB
> End calculation. @ 00:37:13




In [334]:
benchmark = False

In [358]:
# Volume organization 3
est_time_per_w_c = 2.75 #ms
len(Ws_t)
len(Cs_t)
n_calcs = len(Ws_t) * len(Cs_t)
"{0:,} = {0:.2E} calculations".format(n_calcs)

serial_computation_time_est = est_time_per_w_c * len(Ws_t) * len(Cs_t) #ms
est_s = serial_computation_time_est / 1e3
est_m = est_s / 60
est_h = est_m / 60
est_d = est_h / 24

est_at_scales = (est_s, est_m, est_h, est_d)

toSN = lambda n: "{0:.2E}".format(n)

tuple(map(toSN,
          est_at_scales))

est_cps = n_calcs / est_s
"est. {0:,} = {0:.2E} cps".format(est_cps)

9172

106295

'974,937,740 = 9.75E+08 calculations'

('2.68E+06', '4.47E+04', '7.45E+02', '3.10E+01')

'est. 363.6363636363636 = 3.64E+02 cps'

## Calculation volume organization for Case 1

See Section 6.2 for explanation of the "cases".

### Slices: Calculate results for groups of reconstructed wordforms by (input prefix, context)

In [359]:
len(Ps_t)

42231

In [360]:
def retrieveKcousins(r, maxK):
    word_cousin_indices = union([set(list(cousin_mats[k][Ps_t.index(r),:].nonzero()[0])) for k in range(maxK)])
    word_cousins = list(map(lambda w_idx: Ws_t[w_idx], word_cousin_indices))
    return word_cousins

In [361]:
if r or not e:
    retrieveKcousins('⋊.k.ɑ.n.t', 2)

In [362]:
if r or not e:
    cousin_mats[0].shape
    cousin_mats[0][Ps_t.index('⋊.k.ɑ.n.t'), Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')] #should be 1

In [363]:
wordsWithPrefix('⋊.k.ɑ.n.t', Ws)

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉'}

In [364]:
def get_nnz_word(my_k, j):
    return cousin_mats[my_k][:,j].nnz

if r or not e:
    cousin_mats[1].shape
    word_nnz_counts_by_k = {my_k:par(delayed(get_nnz_word)(my_k, j) for j in range(cousin_mats[my_k].shape[1]))
                            for my_k in range(5)}

In [365]:
from statistics import mean, median

In [366]:
if r or not e:
    for my_k in range(5):
        print(f"max number of {my_k}-cousins = {max(word_nnz_counts_by_k[my_k])}")
        print(f"median number of {my_k}-cousins = {median(word_nnz_counts_by_k[my_k])}")
        print(f"mean number of {my_k}-cousins = {mean(word_nnz_counts_by_k[my_k])}")
        print(' ')

In [367]:
def get_nnz_prefix(my_k, i):
    return cousin_mats[my_k][i,:].nnz

if r or not e:
    cousin_mats[1].shape
    prefix_nnz_counts_by_k = {my_k:par(delayed(get_nnz_prefix)(my_k, i) for i in range(cousin_mats[my_k].shape[0]))
                              for my_k in range(5)}

In [368]:
if r or not e:
    for my_k in range(5):
        print(f"max number of {my_k}-cousins = {max(prefix_nnz_counts_by_k[my_k])}")
        print(f"median number of {my_k}-cousins = {median(prefix_nnz_counts_by_k[my_k])}")
        print(f"mean number of {my_k}-cousins = {mean(prefix_nnz_counts_by_k[my_k])}")
        print(' ')

In [369]:
if r or not e:
    cousin_mats[1].shape

In [370]:
def to_prefix_idx(w):
    return Ps_t.index(w)

prefix_indices_of_full_wordforms = par(delayed(to_prefix_idx)(w) for w in Ws)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0023s.) Setting batch_size=176.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0763s.) Setting batch_size=922.
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 9172 out of 9172 | elapsed:    0.4s finished


In [371]:
def to_word_idx(p):
    return Ws_t.index(p)

def get_wordform_idx(prefix_idx):
    return to_word_idx(Ps_t[prefix_idx])

# get_wordform_idx = {prefix_idx:to_word_idx(Ps_t[prefix_idx])
#                     for prefix_idx in prefix_indices_of_full_wordforms}

In [372]:
Ps_t[8370]
get_wordform_idx(8370)
Ws_t[get_wordform_idx(8370)]

'⋊.h.aɪ.k.⋉.⋉'

1928

'⋊.h.aɪ.k.⋉.⋉'

In [373]:
if r and not e:
    cousin_mats[k].shape

In [374]:
def getKcousin_idxs_of_wordform(w_idx, maxK, asType=None, onlyWordCousins = False, getPrefixIndices=True):
    kcousin_indices = union([set(list(cousin_mats[k][:,w_idx].nonzero()[0])) for k in range(maxK)])
    if onlyWordCousins:
    # top line is ≈100x slower
#         kcousin_indices = {prefix_idx for prefix_idx in kcousin_indices if prefix_idx in prefix_indices_of_full_wordforms}
        kcousin_indices = set.intersection(kcousin_indices, prefix_indices_of_full_wordforms)
    if not getPrefixIndices:
        kcousin_indices = set(map(get_wordform_idx, kcousin_indices))
    
    if asType == 'ndarray':
        kcousin_indices = np.array(sorted(list(kcousin_indices)), dtype='int32')
        return kcousin_indices
    elif asType == 'torch':
        kcousin_indices = torch.tensor(sorted(list(kcousin_indices)), dtype=torch.int32)
        return kcousin_indices
    else:
        return kcousin_indices
    
def getKcousin_idxs_of_prefix(p_idx, maxK, asType=None):
    kcousin_indices = union([set(list(cousin_mats[k][p_idx].nonzero()[0])) for k in range(maxK)])
    
    if asType == 'ndarray':
        kcousin_indices = np.array(sorted(list(kcousin_indices)), dtype='int32')
        return kcousin_indices
    elif asType == 'torch':
        kcousin_indices = torch.tensor(sorted(list(kcousin_indices)), dtype=torch.int32)
        return kcousin_indices
    else:
        return kcousin_indices

In [375]:
if r and not e:
    random_source_wordform
    getKcousin_idxs_of_wordform(Ws_t.index(random_source_wordform), 2, 'ndarray', True, False)

In [376]:
if r and not e:
    random_source_prefix
    getKcousin_idxs_of_prefix(Ps_t.index(random_source_prefix), 2, 'ndarray')

In [377]:
def pXhat0fX0i_pxtn_by_rc(produced_prefix_idx, x0k_cm, c_idx, m = 50, asType='ndarray', parallel=False):
    kCousin_idxs = getKcousin_idxs_of_prefix(produced_prefix_idx, k, 'ndarray')
    
    if x0k_cm is None:
        x0k_cm = prefixTox0kCM(Ps_t[produced_prefix_idx])
    
    if parallel:
        estimates = par(delayed(pXhat0fX0i_pxtn)(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs)
        estimates = np.array(list(estimates))
    else:
        estimates = np.array([pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs])

    if asType == 'ndarray':
        return kCousin_idxs, estimates
    elif asType == 'torch':
#         return torch.tensor(kCousin_idxs), torch.tensor(estimates)
        return torch.from_numpy(kCousin_idxs), torch.from_numpy(estimates)
    else:
        return tuple(kCousin_idxs), tuple(estimates)

def pXhat0fX0i_pxtn_by_wc(produced_wordform_idx, x0k_cm, c_idx, m = 50, asType='ndarray', parallel=False):
    equivalent_prefix_idx = Ps_t.index(Ws_t[produced_wordform_idx])
    return pXhat0fX0i_pxtn_by_rc(equivalent_prefix_idx, x0k_cm, c_idx, m, asType, parallel)

In [378]:
random_source_prefix
random_source_prefix_idx = Ps_t.index(random_source_prefix)
random_source_prefix_x0kcm = prefixTox0kCM(random_source_prefix)

'⋊.d.eɪ.k.ɑ'

In [379]:
if r and not e:
    print(f'x0k = {random_source_prefix}')
    print(f'c = {random_context}')
    cousin_idxs, post_probs = pXhat0fX0i_pxtn_by_rc(random_source_prefix_idx, random_source_prefix_x0kcm, random_context_idx)
    print(f'({k}-cousin W-hat, p(W-hat|X0k = {random_source_prefix}, C = {random_context}))')
    [(Ws_t[cousin_idx], post_probs[i]) for i, cousin_idx in enumerate(cousin_idxs)]

### Blocks: Calculate results for groups of input prefixes by context

In [380]:
#FIXME

## Calculation volume organization 2

See Section 6.2 for explanation of the "cases".

In [381]:
#FIXME

## Calculation volume organization 3

See Section 6.2 for explanation of the "cases".

### Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $W$ and for a single given $c$ 

In [382]:
{w for w in Ws_t if ds2t(w)[0] != leftEdge or ds2t(w)[-1] != rightEdge}

set()

In [383]:
Ws_t.index('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉')
len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉')) #19

6304

19

In [384]:
len(CMsByLengthByWordformIndex_torch)
CMsByLengthByWordformIndex_torch[19][6304].shape
CMsByLengthByWordformIndex_torch[19][6304]

20

torch.Size([39, 17])

tensor([[0.0070, 0.0366, 0.0071, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075, 0.0070, 0.0000],
        [0.0070, 0.0089, 0.0071, 0.0071, 0.0092, 0.0335, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075, 0.0070, 0.0000],
        [0.0070, 0.0089, 0.0135, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075, 0.0070, 0.0000],
        [0.0070, 0.0088, 0.0070, 0.0070, 0.0091, 0.0174, 0.0113, 0.0092, 0.0069,
         0.0090, 0.0157, 0.0088, 0.0071, 0.0070, 0.0142, 0.0068, 0.0000],
        [0.0068, 0.0086, 0.0068, 0.0068, 0.0088, 0.0169, 0.0110, 0.0089, 0.0068,
         0.0088, 0.0153, 0.0086, 0.0069, 0.0068, 0.0072, 0.0066, 0.0000],
        [0.0070, 0.0550, 0.0071, 0.2747, 0.0092, 0.1082, 0.0114, 0.0093, 0.0070,
         0.0091, 0.1078, 0.0089, 0.7089, 0.0071, 0.1108, 0.0070, 0.0000],
        [0.0068, 0.0087, 0.0069, 0.0069, 0.0090, 0.0171, 0.0111, 0.009

In [385]:
len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉').shape

16

14

torch.Size([39, 14])

17

15

...

16

torch.Size([39, 16])

19

17

torch.Size([39, 17])

In [386]:
len(ds2t("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n"))
prefixTox0kCM("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n").shape
prefixTox0kCM("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n")

17

torch.Size([39, 15])

tensor([[0.0070, 0.0366, 0.0071, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0089, 0.0071, 0.0071, 0.0092, 0.0335, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0089, 0.0135, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0088, 0.0070, 0.0070, 0.0091, 0.0174, 0.0113, 0.0092, 0.0069,
         0.0090, 0.0157, 0.0088, 0.0071, 0.0070, 0.0142],
        [0.0068, 0.0086, 0.0068, 0.0068, 0.0088, 0.0169, 0.0110, 0.0089, 0.0068,
         0.0088, 0.0153, 0.0086, 0.0069, 0.0068, 0.0072],
        [0.0070, 0.0550, 0.0071, 0.2747, 0.0092, 0.1082, 0.0114, 0.0093, 0.0070,
         0.0091, 0.1078, 0.0089, 0.7089, 0.0071, 0.1108],
        [0.0068, 0.0087, 0.0069, 0.0069, 0.0090, 0.0171, 0.0111, 0.0090, 0.0068,
         0.0089, 0.0155, 0.0087, 0.0070, 0.0069, 0.0073],
        [0.0070, 0.0089, 0.

In [387]:
# CMsByLengthByWordformIndex_torch
# X0f_CMs_by_wordform_index = [wordformTox0kCM(w, w_idx) for w_idx, w in enumerate(Ws_t)]
# X0f_CMs_by_wordform_index_torch = torch.tensor([wordformTox0kCM(w, w_idx) for w_idx, w in enumerate(Ws_t)])
# X0f_CMs_by_wordform_index_torch.shape

In [388]:
def p_wstar_wstar_pxtn_by_wc(produced_wordform_idx, x0f_cm, c_idx, m = 50):
    return pXhat0fX0f_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)
#     return pXhat0fX0i_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)

def p_wstar_wstar_pxtn_by_c(c_idx, m, asType='ndarray', parallel=True):
    if parallel:
        estimates = np.array(list(par(delayed(p_wstar_wstar_pxtn_by_wc)(w_idx, 
                                                                        wordformTox0fCM(w_idx=w_idx),
                                                                        c_idx,
                                                                        m)
                                      for w_idx in range(len(Ws_t)))))
    else:
        estimates = np.array([p_wstar_wstar_pxtn_by_wc(w_idx, 
                                                       wordformTox0fCM(w_idx=w_idx),
                                                       c_idx,
                                                       m)
                             for w_idx in np.arange(len(Ws_t))])
    
    if asType =='ndarray':
        return estimates
    elif asType == 'torch':
        return torch.tensor(estimates)
    else:
        return tuple(estimates)
    
def p_wstar_wstar_pxt_by_wc(produced_wordform_idx, x0f_cm, c_idx, m = 50):
    return pXhat0fX0f_pxt(produced_wordform_idx, x0f_cm, c_idx, m = m)
#     return pXhat0fX0i_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)

def p_wstar_wstar_pxt_by_c(c_idx, m, asType='torch', parallel=True):
    if parallel:
        estimates = torch.tensor(list(par(delayed(p_wstar_wstar_pxt_by_wc)(w_idx, 
                                                                           wordformTox0fCM(w_idx=w_idx),
                                                                           c_idx,
                                                                           m)
                                          for w_idx in range(len(Ws_t)))))
    else:
        estimates = torch.tensor([p_wstar_wstar_pxt_by_wc(w_idx,
                                                          wordformTox0fCM(w_idx=w_idx),
                                                          c_idx,
                                                          m)
                                  for w_idx in np.arange(len(Ws_t))])
    
    if asType =='ndarray':
        return estimates.numpy()
    elif asType == 'torch':
        return estimates
    else:
        return tuple(estimates)

In [389]:
random_source_wordform
random_source_wordform_idx = Ws_t.index(random_source_wordform)
random_source_wordform_x0f_cm = wordformTox0fCM(random_source_wordform, random_source_wordform_idx)
random_source_wordform_x0f_cm.shape
random_source_wordform_x0f_cm

'⋊.ɹ.ɛ.p.t.ɪ.l.i.ə.n.⋉.⋉'

torch.Size([39, 10])

tensor([[0.0070, 0.0085, 0.0090, 0.0090, 0.0082, 0.0075, 0.0070, 0.0204, 0.0111,
         0.0000],
        [0.0070, 0.0085, 0.0090, 0.0090, 0.0157, 0.0075, 0.0070, 0.0107, 0.0111,
         0.0000],
        [0.0070, 0.0085, 0.0226, 0.0090, 0.0082, 0.0075, 0.0070, 0.0107, 0.0111,
         0.0000],
        [0.0070, 0.0084, 0.0089, 0.0088, 0.0081, 0.0074, 0.0069, 0.0106, 0.0108,
         0.0000],
        [0.0068, 0.0082, 0.0087, 0.0086, 0.0079, 0.0072, 0.0068, 0.0103, 0.0105,
         0.0000],
        [0.0070, 0.0422, 0.0090, 0.0090, 0.0387, 0.0075, 0.0070, 0.0560, 0.0111,
         0.0000],
        [0.0068, 0.0083, 0.0088, 0.0087, 0.0080, 0.0073, 0.0068, 0.0104, 0.0108,
         0.0000],
        [0.0070, 0.0085, 0.0090, 0.0090, 0.0082, 0.0075, 0.0070, 0.0107, 0.0111,
         0.0000],
        [0.0068, 0.0084, 0.0089, 0.0088, 0.0081, 0.0074, 0.0069, 0.0106, 0.0111,
         0.0000],
        [0.0070, 0.0085, 0.0090, 0.0090, 0.0206, 0.0143, 0.7380, 0.0107, 0.0111,
         0.0000],
        [0

In [390]:
J

-1

In [391]:
! free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        7.7G         18G        9.8M        4.6G         23G
Swap:          2.0G        1.0G        1.0G


In [419]:
if benchmark:
    random_context_idxs = set(map(lambda c: Cs_t.index(c),
                                  choices(Cs_t, k=1000)))

In [395]:
if benchmark:
    # random_c_idx1 = choice(random_context_idxs); Cs_t[random_c_idx1]
    random_c_idx1 = Cs_t.index('yeah philadelphia')

In [396]:
if benchmark:
    # 1m56s = 116s = 9172c/116s = 79.069 cps / kotoba
    # 14s = 9172/14s = 655.14 cps / kotoba
    random_slice1 = p_wstar_wstar_pxtn_by_c(random_c_idx1, m=50, asType='ndarray', parallel=True)
    random_slice1.shape
    random_slice1.nbytes / 1e6
    random_slice1.nbytes / 1e9
    del random_slice1

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1494s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0464s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1274s.) Setting batch_size=50.
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 556 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 1306 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 2056 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 2906 tasks 

(9172,)

0.036688

3.6688e-05

In [412]:
if benchmark:
    # random_c_idx2 = choice(random_context_idxs); Cs_t[random_c_idx2]
    random_c_idx2 = Cs_t.index('<rem> little')

In [413]:
if benchmark:
    # 1m49s = 109s = 9172c/109s = 84.15 cps / kotoba
    # 12.4s = 9172c/12.4s = 739.68 cps / kotoba
    random_slice2 = p_wstar_wstar_pxtn_by_c(random_c_idx2, m=50, asType='ndarray', parallel=True)
    random_slice2.shape
    random_slice2.nbytes / 1e6
    random_slice2.nbytes / 1e9
    del random_slice2

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0125s.) Setting batch_size=30.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1638s.) Setting batch_size=72.
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 414 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 744 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 1680 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.0065s.) Setting batch_size=36.
[Parallel(n_jobs=-1)]: Done 2616 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.4426s.) Setting batch_size=18.
[Parallel(n_jobs=-1)]: Done 3624 tasks      | elapsed:    5.7s
[Parallel(n_job

(9172,)

0.036688

3.6688e-05

In [414]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        7.8G         18G        9.8M        4.6G         23G
Swap:          2.0G        1.0G        1.0G


In [415]:
if benchmark:
    J = 4

In [421]:
if benchmark:
    random_c_idx3 = choice(list(random_context_idxs)); Cs_t[random_c_idx3]

'ultimately'

In [423]:
if benchmark:
    random_slice3 = p_wstar_wstar_pxtn_by_c(random_c_idx3, m=50, asType='ndarray', parallel=True)
    random_slice3.shape
    random_slice3.nbytes / 1e6
    random_slice3.nbytes / 1e9
    del random_slice3

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0105s.) Setting batch_size=38.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 518 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 936 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 1430 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1924 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 2494 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 3064 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 3710 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 4356 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 5078 tasks      | elapsed:    7.0s
[Parallel(n_job

(9172,)

0.036688

3.6688e-05

In [403]:
if benchmark:
    J = 8

In [404]:
if benchmark:
    random_c_idx4 = choice(random_context_idxs); Cs_t[random_c_idx4]

In [405]:
if benchmark:
    random_slice4 = p_wstar_wstar_pxtn_by_c(random_c_idx4, m=50, asType='ndarray', parallel=True)
    random_slice4.shape
    random_slice4.nbytes / 1e6
    random_slice4.nbytes / 1e9
    del random_slice4

In [406]:
if benchmark:
    J = 16

In [407]:
if benchmark:
    random_c_idx5 = choice(random_context_idxs); Cs_t[random_c_idx5]

In [408]:
if benchmark:
    random_slice5 = p_wstar_wstar_pxtn_by_c(random_c_idx5, m=50, asType='ndarray', parallel=True)
    random_slice5.shape
    random_slice5.nbytes / 1e6
    random_slice5.nbytes / 1e9
    del random_slice5

In [409]:
if benchmark:
    random_c_idx6 = random_c_idx5

    random_slice6 = p_wstar_wstar_pxt_by_c(random_c_idx6, m=50, asType='torch', parallel=True)
    random_slice6.shape
    torch_nbytes(random_slice6) / 1e6
    torch_nbytes(random_slice6) / 1e9
    del random_slice6

In [410]:
if benchmark:
    random_c_idx6 = random_c_idx5

    random_slice6 = p_wstar_wstar_pxt_by_c(random_c_idx6, m=50, asType='torch', parallel=True)
    random_slice6.shape
    torch_nbytes(random_slice6) / 1e6
    torch_nbytes(random_slice6) / 1e9
    del random_slice6

In [411]:
if benchmark:
    J = -1

### Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $c$

In [ ]:
# pXhat0fX0i_ptc(xhat0f_idx, x0k_CM, m = 50, target_contexts_per_batch=750, parallel=True)

# Calculate distribution

## Number of calculations necessary under different calculation criteria

In general, the full distribution that could be calculated is $$p(\widehat{X}_0^f | X_0^k, C) = p(\widehat{W} | R, C)$$

In [327]:
len(Ws_t)
len(Ps_t)
len(Cs_t)
"{0:,}".format( len(Ws_t) * len(Ps_t) * len(Cs_t) )
"{0:.2E}".format( len(Ws_t) * len(Ps_t) * len(Cs_t) )

9172

42231

106295

'41,172,595,697,940'

'4.12E+13'

For the chosen value of $k$, there are this many $(w,r)$ pairs to be calculated (for every $c$):

In [328]:
# cousin_mats[k].shape
# sum([cousin_mats[k][i,:].nnz for i in range(len(Ps))])

In [329]:
if r and not e:
    print(f'k = {k}')

    cousin_mats[k].shape

    def calc_count(i):
        return cousin_mats[k][i,:].nnz

    num_seq_pairs_case1 = sum(list(par(delayed(calc_count)(i) for i in range(len(Ps)))))
    num_calcs_case1 = len(Cs_t) * num_seq_pairs_case1

    print('|(w,r)| pairs to do calculations for every c = {0:,}'.format( num_seq_pairs_case1 ))
    print("|(w,r)| pairs there would be without the k-edit-distance-restriction = {0:,}".format( len(Ws) * len(Ps) ))
    print(f'|Cs| = {len(Cs_t)}')
    print('Est. of total number of calculations to do = {0:,} = {0:.2E}'.format(num_calcs_case1))

If $r$ is `False`, then the dimensions and number of calculations reduce to:

In [330]:
len(Ws_t)
len(Ws_t)
len(Cs_t)
"{0:,}".format( len(Ws_t) * len(Ws_t) * len(Cs_t) )
"{0:.2E}".format( len(Ws_t) * len(Ws_t) * len(Cs_t) )

9172

9172

106295

'8,942,128,951,280'

'8.94E+12'

If $r$ is `False`, then for the chosen value of $k$, there are this many $(w',w)$ pairs to be calculated (for every $c$):

In [331]:
if r or not e:
    print(f'k = {k}')

    cousin_mats[k].shape

    def calc_count(i):
        if Ps_t[i] in Ws:
            return cousin_mats[k][i,:].nnz
        else:
            return 0

    num_seq_pairs_case2 = sum(list(par(delayed(calc_count)(i) for i in range(len(Ps)))))
    num_calcs_case2 = len(Cs_t) * num_seq_pairs_case2

    print("|(w',w)| pairs to do calculations for every c = {0:,}".format( num_seq_pairs_case2 ))
    print("|(w',w)| pairs there would be without the k-edit-distance-restriction = {0:,}".format( len(Ws) * len(Ws) ))
    print(f'|Cs| = {len(Cs_t)}')
    print('Est. of total number of calculations to do = {0:,} = {0:.2E}'.format(num_calcs_case2))

If $r$ is `False` and $e$ is `True`, then the dimensions and number of calculations reduce to:

In [332]:
len(Ws)
len(Cs_t)
num_calcs_case3 = len(Ws_t) * len(Cs_t)
"{0:,}".format( num_calcs_case3 )
"{0:.2E}".format( num_calcs_case3 )

9172

106295

'974,937,740'

'9.75E+08'

## Determine dimensions and cells to be calculated vs. ignored

In [333]:
assert (k != 0 and r) or (k != 0 and (not r) and (not e)) or (k == 0 and (not r) and e) or (k > 0 and (not r) and e)

### Option: Only calculate full source wordforms rather than all source prefixes?

In [334]:
print(f"r = {r}")

if r:
    source_sequences = Ps_t
    print("Performing calculations for all source prefixes.")
else:
    source_sequences = Ws_t
    print("Performing calculations only for all complete source wordforms (and *not* all prefixes thereof).")

Ss_t = source_sequences    

print(f"|source_sequences| = {len(source_sequences)}")

r = False
Performing calculations only for all complete source wordforms (and *not* all prefixes thereof).
|source_sequences| = 9172


### Suboption: Given $\neg r$ and a full source wordform $w^*$, only calculate posterior probability $p(\widehat{W} = w^* | W = w^*, C = c)$, $\forall w^*$?

In [335]:
print(f"¬r ∧ e = {not r} ∧ {e} = {(not r) and e}")

if (not r) and e:
    print("Performing calculations only for those cases where the reconstructed wordform exactly matches the source wordform.")
    
else:
    print("Calculations not restricted only to exact reconstructed wordform matches.")

¬r ∧ e = True ∧ True = True
Performing calculations only for those cases where the reconstructed wordform exactly matches the source wordform.


### Suboption: Restrict calculations only to (source sequence, reconstruction sequence) pairs within edit distance $k$?

In [336]:
print(f"k = {k}")

if r or ((not r) and (not e)):
    print(f"Restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance {k}.")
elif (not r) and e:
    print(f"¬r ∧ e is equivalent to restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance 0.")
else:
    print("No restrictions on (source sequence, reconstructed sequence) pairs to be calculated.")

k = 2
¬r ∧ e is equivalent to restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance 0.


### Altogether...

In [337]:
o

'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

In [338]:
if r and (0 < k <= 4):
    case = 1    
    myShape = (len(Ws_t), len(Ps-t), len(Cs_t))
    my_num_calcs = num_calcs_case1
    
    my_type = f'pW_RC_k{k}'
    status = '_unnormalized'

if (not r) and (0 < k <= 4) and (not e):
    case = 2
    myShape = (len(Ws_t), len(Ws_t), len(Cs_t))
    my_num_calcs = num_calcs_case2
    
    my_type = f'pW_WC_k{k}'
    status = '_unnormalized'
    
if (not r) and e:
    case = 3
#     myShape = (len(Ws), len(Ws), len(Cs))
    myShape = (len(Ws_t), len(Cs_t))
    my_num_calcs = num_calcs_case3

    my_type = f'pW_WC_e'
    status = ''

print(f'Case:\n\t{case}')
print(f'Shape of matrix:\n\t{myShape}')
print('Num elements:')
print("\t{0:,} = {0:.2E}".format(prod(myShape)))
print('Num calculations:')
print("\t{0:,} = {0:.2E}".format(my_num_calcs))
print('Est. size in GB of result:')
print("\t{0:,} = {0:.2E}".format(my_num_calcs * 8 / 1e9))
print("Density:")
print("\t{0:,}".format(my_num_calcs / prod(myShape)))    

print('Result suffix:')
my_fn_suffix = my_type + status
print(f"\t{my_fn_suffix}")

Case:
	3
Shape of matrix:
	(9172, 106295)
Num elements:
	974,937,740 = 9.75E+08
Num calculations:
	974,937,740 = 9.75E+08
Est. size in GB of result:
	7.79950192 = 7.80E+00
Density:
	1.0
Result suffix:
	pW_WC_e


In [339]:
abort_threshold_GB = 250
size_est_GB = (my_num_calcs * 8 / 1e9)
assert size_est_GB <= abort_threshold_GB, "Estimated size of resulting matrix is ≥ {1} GB:\n\tEst. size = {0:,}GB = {0:.2E}GB\n\tAborting calculation.".format(size_est_GB, abort_threshold_GB)

In [340]:
# if exists(o + my_fn_suffix):
#     unnormalized_posterior = np.load(o + my_fn_suffix + '.npy',
#                                      allow_pickle = False)
# assert not exists(o + my_fn_suffix + '.npy'), 'Unnormalized distribution already found at filepath \n\t{0}'.format( o + my_fn_suffix + '.npy' )

In [341]:
# unnormalized_posterior_torch = np.zeros(myShape)

## Case 1: $r \land (0 < k \leq 4)$

## Case 2: $\neg r \land (0 < k \leq 4) \land \neg e$

## Case 3: $\neg r \land e$

In [342]:
myShape
# blockShape = (myShape[0], 1) #every context is a slice
blockShape = (1, myShape[1]) #every source wordform is a slice
blockShape

(9172, 106295)

(1, 106295)

In [343]:
myShape #dim of eventual tensor representing desired portion of distribution

# segWordDomain = (wStart, wEnd); segWordDomain
segWordDomain = (0, myShape[0]-1); segWordDomain
contextDomain = (0, myShape[1]-1); contextDomain

(9172, 106295)

(0, 9171)

(0, 106294)

In [344]:
segWordDomainStr = '_' + 'w' + '.' + str(wStart) + '-' + str(wEnd); segWordDomainStr
domainRegionStr = segWordDomainStr; domainRegionStr

'_w.0-9171'

'_w.0-9171'

In [345]:
# dbName = my_fn_suffix + domainRegionStr; dbName
dbName = my_fn_suffix# + domainRegionStr; dbName
dbName

'pW_WC_e'

In [346]:
dom = tiledb.Domain(tiledb.Dim(name="segWord", domain=segWordDomain, tile=blockShape[0], dtype=np.uint32),
                    tiledb.Dim(name="context", domain=contextDomain, tile=blockShape[1], dtype=np.uint32))

schema = tiledb.ArraySchema(domain=dom, sparse=False,
                            attrs=[tiledb.Attr(name=dbName, dtype=np.float32)])

In [347]:
o
my_fn_suffix
# domainRegionStr
dbName
o + '.' + dbName
array_name = o + '.' + dbName

'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

'pW_WC_e'

'pW_WC_e'

'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC_e'

In [348]:
if tiledb.object_type(array_name) == "array":
        print(f"Array with name\n\t{array_name}\nalready exists.")
else:
    tiledb.DenseArray.create(array_name, schema)
    print(f'Array with name\n\t{array_name}\ncreated.')

Array with name
	LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC_e
already exists.


In [349]:
# rm -r 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC_e'

In [350]:
# will also work if if given a range of indices and a range of slices...
# def slice_writer(slice_context_index, context_slice, A):
# # #     with tiledb.SparseArray(arr_name, mode='w') as A:
#         A[:,slice_context_index] = context_slice

def slice_writer(slice_word_index, word_slice, A):
# #     with tiledb.SparseArray(arr_name, mode='w') as A:
        A[slice_word_index,:] = word_slice

In [351]:
def tensor_reader(indices):
    with tiledb.DenseArray(array_name, mode='r') as A:
        return A[indices][dbName]

In [352]:
config = tiledb.Config()
# config["sm.consolidation.steps"] = 10
config["sm.consolidation.steps"] = 600
config["sm.consolidation.step_min_frags"] = 2
config["sm.consolidation.step_max_frags"] = 20
# tiledb.consolidate(array_name, config)

In [353]:
# if cStart != '' and cEnd != '':
#     print(f'Context block start and end given: ({cStart}, {cEnd})')
#     blockRange = np.arange(cStart, cEnd)
#     assert 0 <= cStart <= len(Cs_t), f'0<= cStart <= len(Cs_t) does not hold: {cStart} vs. {len(Cs_t)}'
#     assert 0 <= cEnd <= len(Cs_t), f'0<= cEnd <= len(Cs_t) does not hold: {cEnd} vs. {len(Cs_t)}'
# else:
#     print(f'Calculating all indices from 0 to len(Cs_t) = {len(Cs_t)}')
#     blockRange = np.arange(len(Cs_t))


# for c_idx in tqdm(blockRange):
#     context_slice = p_wstar_wstar_pxtn_by_c(c_idx, m=50, asType='ndarray', parallel=True)

#     with tiledb.SparseArray(array_name, mode='w') as A:
#         slice_writer(c_idx, context_slice, A)
        
#     if c_idx % 100 == 0:
#         tiledb.consolidate(config, uri=array_name)

In [354]:
if wStart != '' and wEnd != '':
    print(f'Word block start and end given: ({wStart}, {wEnd})')
    blockRange = np.arange(wStart, wEnd)
    assert 0 <= wStart < len(Ws_t), f'0<= wStart < len(Ws_t) does not hold: {wStart} vs. {len(Ws_t)}'
    assert 0 <= wEnd < len(Ws_t), f'0<= wEnd < len(Ws_t) does not hold: {wEnd} vs. {len(Ws_t)}'
else:
    print(f'Calculating all indices from 0 to len(Ws_t)-1 = {len(Ws_t)}-1 = {len(Ws_t)-1}')
    blockRange = np.arange(len(Ws_t))

Word block start and end given: (0, 9171)


In [462]:
def uncalculated_words_by_context(context_id, wordRange=None):
    if wordRange is None:
        wordRange = blockRange
        
    with tiledb.DenseArray(array_name, mode='r') as A:
        nanMask = np.isnan(A[:,context_id][dbName])
#         print(nanMask)
#         print(nanMask.shape)
        uncalculated_indices = np.arange(len(Ws_t))[ nanMask ]
#         uncalculated_indices_s = set(uncalculated_indices)
#         uncalculated_indices_in_range = np.array(sorted(list( uncalculated_indices_s & set(wordRange) )))
        uncalculated_indices_in_range = np.intersect1d(uncalculated_indices, wordRange)
        return uncalculated_indices_in_range

def uncalculated_contexts_by_word(word_id, contextRange=None):
    if contextRange is None:
        contextRange = np.arange(len(Cs_t)) #case 3-specific assumption
        
    with tiledb.DenseArray(array_name, mode='r') as A:
        nanMask = np.isnan(A[word_id,:][dbName])
#         print(nanMask)
#         print(nanMask.shape)
        uncalculated_indices = np.arange(len(Cs_t))[ nanMask ]
#         uncalculated_indices_s = set(uncalculated_indices)
#         uncalculated_indices_in_range = np.array(sorted(list( uncalculated_indices_s & set(contextRange) )))
        uncalculated_indices_in_range = np.intersect1d(uncalculated_indices, contextRange)
        return uncalculated_indices_in_range
    
def is_completely_calculated(word_id=None, context_id=None):
    if word_id is not None:
        return np.array_equal( uncalculated_contexts_by_word(word_id), 
                               np.array([], dtype=np.float32))
    if context_id is not None:
        return np.array_equal( uncalculated_contexts_by_context(context_id),
                               np.array([], dtype=np.float32))
    raise Exception('At least one argument must be non-null.')
    
def is_completely_calculated_w(w):
    return is_completely_calculated(w)

def is_completely_calculated_c(c): 
    return is_completely_calculated(context_id=c)
    
def get_completed_indices(dim='words', rangeOfInterest=None):
    if dim == 'words':
        if rangeOfInterest is None:
            rangeOfInterest = np.arange(len(Ws_t))
        return np.array([w_idx
                         for w_idx in tqdm(rangeOfInterest, total=len(rangeOfInterest))
                         if is_completely_calculated(word_id=w_idx)])
#         selection_mask = par(delayed(is_completely_calculated)(w_idx)
#                              for w_idx in rangeOfInterest)
#         return rangeOfInterest[ np.array(selection_mask) ] 
    if dim == 'contexts':
        if rangeOfInterest is None:
            rangeOfInterest = np.arange(len(Cs_t))
        return np.array([c_idx
                         for c_idx in tqdm(rangeOfInterest, total=len(rangeOfInterest))
                         if is_completely_calculated(context_id=c_idx)])
#         selection_mask = par(delayed(is_completely_calculated_c)(c_idx)
#                             for c_idx in rangeOfInterest)
#         return rangeOfInterest[ np.array(selection_mask) ]
    raise Exception(f"dim must be one of {'words', 'contexts'}, got {dim} instead")
                    
def get_incomplete_indices(dim='words', rangeOfInterest=None):
    if dim == 'words':
        if rangeOfInterest is None:
            rangeOfInterest = np.arange(len(Ws_t))
    if dim == 'contexts':
        if rangeOfInterest is None:
            rangeOfInterest = np.arange(len(Cs_t))
    
    completed_indices = get_completed_indices(dim=dim, rangeOfInterest=rangeOfInterest)
    
    return np.array([i for i in rangeOfInterest if i not in completed_indices])

In [435]:
np.array([], dtype=np.float32)

array([], dtype=float32)

In [433]:
len(Ws_t)
uncalculated_words_by_context(context_id=6703)
# uncalculated_indices(context_id=np.array([6703, 6704]))
uncalculated_contexts_by_word(word_id=6703)

9172

array([ 273,  274,  275, ..., 9168, 9169, 9170])

array([     0,      1,      2, ..., 106292, 106293, 106294])

In [461]:
get_incomplete_indices(rangeOfInterest=blockRange)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [463]:
from copy import deepcopy

In [464]:
if d:
    #   identify indices that have not been completely calculated
    print("Identifying incompletely calculated indices in block range...")
    print("\t(Partially calculated indices will still be overwritten.)")
    old_block_range = deepcopy(blockRange)
    old_block_range_l = len(old_block_range)
    blockRange = get_incomplete_indices(rangeOfInterest=blockRange)
    new_block_range_l = len(blockRange)
    print(f"Original block range size = {old_block_range_l}\nNew block range size = {new_block_range_l}")
    completed_indices = np.array([i for i in old_block_range if not i in blockRange])
    if completed_indices.shape[0] > 0:
        print(f"|completed indices| = {completed_indices.shape[0]}")
        print("completed indices = {completed_indices}")
else:
    print("All indices in blockRange will be overwritten.")
# else:
#     #erase and start over
#     tiledb.remove(array_name) # <<<< this would destroy EVERYTHING
#     tiledb.DenseArray.create(array_name, schema)
#     print(f'Array with name\n\t{array_name}\ncreated.')


  1%|          | 51/9171 [00:00<00:18, 504.36it/s]

Identifying incompletely calculated indices in block range...



 18%|█▊        | 1688/9171 [00:17<01:32, 80.96it/s]


 33%|███▎      | 3020/9171 [00:34<01:15, 81.21it/s]


 47%|████▋     | 4349/9171 [00:50<00:59, 81.14it/s]


 62%|██████▏   | 5681/9171 [01:07<00:42, 82.33it/s]


 76%|███████▋  | 7013/9171 [01:23<00:26, 81.42it/s]


 91%|█████████ | 8345/9171 [01:39<00:10, 81.73it/s]


100%|██████████| 9171/9171 [01:49<00:00, 83.50it/s]

Original block range size = 9171
New block range size = 8898
|completed indices| = 273
completed indices = {completed_indices}


In [465]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            31G        8.0G         19G        9.5M        3.4G         22G
Swap:          2.0G        1.1G        948M


In [371]:
def logCondition(w_idx):
    if len(blockRange) < 50:
        return True
    elif len(blockRange) < 100:
        return w_idx % 10 == 0
#     elif len(blockRange) < 250:
    else:
        return w_idx % 25

with tiledb.DenseArray(array_name, mode='w') as A:
    for w_idx in tqdm(blockRange, total=len(blockRange)):
#         word_slice = p_wstar_wstar_pxtn_by_c(c_idx, m=50, asType='ndarray', parallel=True)
        
        if logCondition(w_idx):
            print('')
            stampedMemNote(f'w_idx = {w_idx}')
            stampedNote('Calculating word slice...')

        word_slice = pXhat0fX0i_ptc(w_idx, wordformTox0kCM(w_idx=w_idx), 
                                    m = n, target_contexts_per_batch=b, 
                                    parallel=l, use_gpu=g).numpy()
        torch.cuda.empty_cache()
        
        if logCondition(w_idx):
            stampedMemNote('\tWriting to disk...')
        slice_writer(w_idx, word_slice, A)
        if logCondition(w_idx):
            stampedMemNote('\tWrote to disk.')
        
        if logCondition(w_idx) and consolidation_enabled:
            stampedMemNote('\tConsolidating...')
            tiledb.consolidate(uri=array_name, config = config)
            stampedMemNote('\tConsolidated.')

  0%|          | 0/9171 [00:00<?, ?it/s]

> Begin calculation. @ 17:59:53
l = 7
Sampling... @ 17:59:53
	VM used vs. available: 6.69GB vs. 26.52GB


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    3.8s remaining:    8.9s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    4.3s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    4.7s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.4s finished


Splitting samples by context... @ 17:59:59
	VM used vs. available: 8.19GB vs. 25.02GB



  0%|          | 0/426 [00:00<?, ?it/s]

...

Splitting samples by context... @ 18:38:40
	VM used vs. available: 9.55GB vs. 23.66GB



  0%|          | 0/426 [00:00<?, ?it/s]

Splitting contexts into batches ... @ 18:38:40
Num batches = 426
Processing batches... @ 18:38:40
	VM used vs. available: 10.38GB vs. 22.83GB



 35%|███▍      | 147/426 [00:10<00:18, 14.70it/s]

KeyboardInterrupt: 

In [466]:
args = {
    'x':x,
    'd':d,
    'n':n,
    'k':k,
    'b':b,
    'l':l,
    'g':g,
    'r':r,
    'e':e,
    'wStart':wStart,
    'wEnd':wEnd
}

dim_md = {'W':{'from fp':f"from W dimension of {p}",
               'changes':'sorted full wordforms',
               'size':len(Ws_t)},
          'C':{'from fp':s,
               'changes':'sorted contexts',
               'size':len(Cs_t)}}
exportMatrixMetadata(md_fp=array_name + '_metadata.json', 
                     matrix_fp=array_name, 
                     matrix=None, 
                     dim_md=dim_md, 
                     step_name='Step 5b', 
                     nb_name='Calculate segmental posterior given segmental wordform + context', 
                     other_md = args)

ValueError: dictionary update sequence element #0 has length 10; 2 is required

In [ ]:
myShape

In [ ]:
first_word_slice = tensor_reader(0)

In [ ]:
blockRange

In [ ]:
first_word_slice.shape
non_inf_indices = np.where(first_word_slice < np.inf)[0]
non_inf_indices
len(non_inf_indices)
non_inf_probs = first_word_slice[ non_inf_indices ]#.sum(); non_inf_counts
non_inf_probs

In [ ]:
idealize_slice = pXhat0fX0i_ptc(0, CMsByLengthByWordformIndex_torch[len(ds2t(Ws_t[0]))][0],
                                m=n, target_contexts_per_batch=b, parallel=l, use_gpu=g, exact_wf=True)
idealize_slice.dtype
type(idealize_slice)
np.array_equal(first_word_slice, idealize_slice)

In [ ]:
Cs_t[0]
pXhat0fX0f_pxt(0, 
               CMsByLengthByWordformIndex_torch[len(ds2t(Ws_t[0]))][0], 
               0, m=n)
idealize_slice.shape
idealize_slice[0]

In [ ]:
idealize_CM = CMsByLengthByWordformIndex_torch[len(ds2t(Ws_t[0]))][0]
idealize_slice_slow = par(delayed(pXhat0fX0f_pxt)(0,
                                                  idealize_CM,
                                                  c_idx,
                                                  m = n)
                          for c_idx in torch.arange(len(Cs_t)))

In [ ]:
Ws_t[0]
for c_idx in non_inf_indices:
    Cs_t[c_idx]

In [ ]:
idealize_slice2 = pXhat0fX0i_ptc(0, CMsByLengthByWordformIndex_torch[len(ds2t(Ws_t[0]))][0],
                                m=n, target_contexts_per_batch=b, parallel=l, use_gpu=g)
idealize_slice2.dtype
type(idealize_slice2)
np.array_equal(first_word_slice, idealize_slice2)

In [ ]:
if g:
    torch.cuda.max_memory_allocated(gpu) / 1e6